# 0. Setup

In [1]:
# !pip install pandas==1.0.5

In [2]:
import numpy as np
import pandas as pd
import os.path
import math
import re
import random

In [3]:
root_path = os.path.dirname(os.getcwd())
ingredients_data_raw = pd.read_csv(os.path.join(root_path, 'DATA/ingredients_data.csv'))
ingredients_data = ingredients_data_raw.replace(float('nan'), '')
stir_fry_data_impractical = ingredients_data[ingredients_data['stir_fry']=='y']
stir_fry_data_all = stir_fry_data_impractical[(stir_fry_data_impractical['stir_fry_umbrella'] != 'y') & (stir_fry_data_impractical['redirect'] != 'y')]
stir_fry_data = stir_fry_data_all[stir_fry_data_all['stir_fry_yes'] == 'y']

stir_fry_data_basic = stir_fry_data[stir_fry_data['stir_fry_basic'] == 'y']
# stir_fry_data_current = stir_fry_data[stir_fry_data['2020_7_5'] == 'y']
# stir_fry_data_with_umbrella = ingredients_data[(ingredients_data['stir_fry_umbrella'] != 'y') & (ingredients_data['stir_fry'] == 'y')]
# stir_fry_data = stir_fry_data_basic
# stir_fry_data = stir_fry_data_current
# stir_fry_data = stir_fry_data_with_umbrella
stir_fry_data_is_basic = False

stir_fry_data.reset_index(drop=True, inplace=True)

In [4]:
root_path = os.path.dirname(os.getcwd())
ingredients_data_raw = pd.read_csv(os.path.join(root_path, 'DATA/ingredients_data.csv'))
ingredients_data = ingredients_data_raw.replace(float('nan'), '')

stir_fry_data = ingredients_data[(ingredients_data['stir_fry_yes'] == 'y') & (ingredients_data['stir_fry_umbrella'] != 'y') & (ingredients_data['redirect'] != 'y')]
stir_fry_data.reset_index(drop=True, inplace=True)

# 1. Matching "pairs with" terms to ingredient names
### (doesn't need to be run regularly) (except the first bit?)

In [5]:
def get_terms_from_pairs_with(pairs_with):
    if str(pairs_with) == 'nan':
        return []
    else:
        return [term.strip() for term in pairs_with.split('\n\n') if term.strip() != '']

# break entries in column that has 'pairs with' strings into lists of ingredient terms
ingredient_pairs_with_terms = stir_fry_data['pairs_with'].apply(get_terms_from_pairs_with)

In [634]:
# # create list of all terms, ignoring case and excluding duplicates
# all_terms = list(set(ingredient_pairs_with_terms.sum()))
# all_terms_lower = list(set([term.lower() for term in ingredient_pairs_with_terms.sum()]))

In [635]:
# # !pip install inflect
# import inflect
# p = inflect.engine()

In [636]:
# %%time

# salad_matches = pd.read_csv(os.path.join(root_path, 'DATA/term_name_matches_specific.csv'))
    
# def get_tokens(phrase):
#     tokens = [token.strip() for token in re.split('\(|\)|,|e\.g\.|esp\.|and|—|or|aka|see|see also|;|and\/or|\*', phrase)]
# #     print(tokens)
#     tokens = [p.singular_noun(token) or token for token in tokens if token != '']
# #     print(tokens)
#     return tokens

# def get_mark(name, term):
# #     print()
# #     print('NAME', name)
# #     print('TERM', term)
#     try:
#         salad_match = salad_matches[term][salad_matches['name'] == name].iloc[0]
# #         print('SALAD MATCH', salad_match)
#     except:
# #         print('NO SALAD MATCH')
#         salad_match = None 
#     if salad_match:
#         if str(salad_match) in ['0', 'nan']:
# #             print('BAD SALAD MATCH')
#             return ''
#         else:
# #             print('GOOD SALAD MATCH', salad_match)
# #             print('NAME', name)
# #             print('TERM', term)
# #             print()
#             return salad_match
    
#     name_tokens = [token.lower() for token in get_tokens(name)]  
#     term_tokens_mixed = get_tokens(term)
#     term_tokens = [token.lower() for token in term_tokens_mixed]
    
#     primary_name = re.split('\(|—|e\.g\.', name)[0].strip()
#     primary_name_split = primary_name.split(', ')
#     single_comma_primary_name = len(primary_name_split) == 2
    
# #     print()
#     if 'e.g.' in term:
#         if single_comma_primary_name:
#             if name_tokens[0] == term_tokens[0]:
#                 if name_tokens[1] in term_tokens: # specific name in e.g. term
# #                     print('NAME TOKENS In TERM TOKENS')
#                     term_i = term_tokens.index(name_tokens[1])
#                     if term_tokens_mixed[term_i] == term_tokens_mixed[term_i].upper():
#                         match = 'D'
#                     else:
#                         if term_tokens_mixed[0] == term_tokens_mixed[0].upper():
#                             match = 'D'
#                         else:
#                             match = 'd'
#                 else: # name matches only generic (pre e.g.) part of term
#                     if term_tokens_mixed[0] == term_tokens_mixed[0].upper():
#                         match = 'C'
#                     else:
#                         match = 'c'
#             else:
#                 match = ''
#         else:
#             if name_tokens[0] == term_tokens[0]: # name matches term before e.g.
#                 if term_tokens_mixed[0] == term_tokens_mixed[0].upper():
#                     match = 'C'
#                 else:
#                     match = 'c'
#             elif name_tokens[0] in term_tokens[1:]: # name matches term after e.g.
#                 term_i = term_tokens.index(name_tokens[0])
#                 if term_tokens_mixed[term_i] == term_tokens_mixed[term_i].upper():
#                     match = 'D'
#                 else:
#                     if term_tokens_mixed[0] == term_tokens_mixed[0].upper():
#                         match = 'D'
#                     else:
#                         match = 'd'
#             else:
#                 match = ''
#     else:
#         if single_comma_primary_name:
#             if ' '.join(primary_name_split).lower() in term.lower() or primary_name.lower() in term.lower(): # if primary_name is in term, any order
#                 if term_tokens_mixed[0] == term_tokens_mixed[0].upper():
#                     match = 'D'
#                 else:
#                     match = 'd'
#             elif name_tokens[0] == term_tokens[0]: # if first part of primary name matches first token in term
#                 if term_tokens_mixed[0] == term_tokens_mixed[0].upper():
#                     match = 'C'
#                 else:
#                     match = 'c'
#             else:
#                 match = ''
#         else:
#             if name_tokens[0] in term_tokens: # if non-comma name anywhere in non-e.g. term_tokens
#                 if term_tokens_mixed[0] == term_tokens_mixed[0].upper():
#                     match = 'D'
#                 else:
#                     match = 'd'
#             elif len(set(name_tokens).intersection(set(term_tokens))) > 0: # if there are any common tokens
#                 if term_tokens_mixed[0] == term_tokens_mixed[0].upper():
#                     match = 'C'
#                 else:
#                     match = 'c'
#             else:
#                 match = ''
                
#     if match == '':
#         n_common = len(set(name_tokens).intersection(set(term_tokens)))
#         if n_common != 0:
#             match = n_common
#     return match

In [637]:
# %%time

# # doing it this way so I can add 'print' to monitor progress
# mark_data = []
# for name in stir_fry_data['name']:
#     print(name)
#     mark_data.append([get_mark(name, term) for term in all_terms])

# term_name_marks = pd.DataFrame(mark_data, columns = all_terms)
# term_name_marks['name'] = pd.Series(stir_fry_data['name'].values.tolist())

# term_name_marks.to_csv(os.path.join(root_path, 'DATA/stir_fry_term_name_marks.csv'), index=False)

# 2. Extracting "pairs with" data

In [7]:
# term_name_matches_raw = pd.read_csv(os.path.join(root_path, 'DATA/stir_fry_term_name_matches_plus.csv'))
# term_name_matches = term_name_matches_raw.replace(['0', '1', '2', '3', '4', '5', 0, 1, 2, 3, 4, 5, float('nan')], '')

In [8]:
# %%time

# # CREATE PAIRING DATA MATRIX (names x names)
# # takes a few minutes

# pairing_data = pd.DataFrame({
#     'name': stir_fry_data['name'],
#     'pairs_with_terms': ingredient_pairs_with_terms
# })

# for name in stir_fry_data['name']:
#     pairing_data[name] = pd.Series(['']*len(stir_fry_data['name']))

# def get_pairs_with_names(row):
#     lower_category_names = []
#     lower_direct_names = []
#     upper_category_names = []
#     upper_direct_names = []
#     for term in row['pairs_with_terms']:
#         if term in term_name_matches.columns.values.tolist():
#             lower_category_names += term_name_matches[term_name_matches[term] == 'c']['name'].values.tolist()
#             lower_direct_names += term_name_matches[term_name_matches[term] == 'd']['name'].values.tolist()
#             upper_category_names += term_name_matches[term_name_matches[term] == 'C']['name'].values.tolist()
#             upper_direct_names += term_name_matches[term_name_matches[term] == 'D']['name'].values.tolist()
#         else:
#             pass
#             print(term)
    
#     for lower_category_name in lower_category_names:
#         row[lower_category_name] = 'c'
#     for lower_direct_name in lower_direct_names:
#         row[lower_direct_name] = 'd'
#     for upper_category_name in upper_category_names:
#         row[upper_category_name] = 'C'
#     for upper_direct_name in upper_direct_names:
#         row[upper_direct_name] = 'D'

#     return row

# pairing_data = pairing_data.apply(get_pairs_with_names, axis=1)
# pairing_data.replace(float('nan'), '', inplace=True)

(Native and North) American cuisines
BAKED GOODS, e.g., breads, cakes, cookies, pastries, pies, scones
butter, e.g., brown
cereals, breakfast, e.g., granola
cheese, e.g., blue, goat, ricotta
cherries, esp. dried
ice cream, e.g., butter pecan
PIES, e.g., pecan, sweet potato
purees, vegetable
waffles
wheat germ
bibimbap
cereals, hot breakfast, e.g., with fruit and nuts
“meatballs” (e.g., brown rice + onion + parsley + walnuts)
soups, e.g., tomato
sprouts, e.g., bean, pea
sweeteners, e.g., mirin, brown sugar
milk, e.g., nondairy (almond, coconut, rice)
cheese, e.g., blue, mozzarella
citrus, e.g., grapefruit, lemon, lime, orange, tangerine; juice, zest
eggs, e.g., frittatas, hard-boiled, tortillas
OILS, e.g., nut, olive, walnut
salad dressings, e.g., sherry vinaigrette
SALADS, e.g., fruit, vegetable
sauces, e.g., butter
SPANISH CUISINE
vinegar, other, e.g., balsamic, red wine, white wine
CPU times: user 27.2 s, sys: 7.79 ms, total: 27.2 s
Wall time: 27.2 s


In [9]:
# %%time

# # SYNC PAIRING DATA MATRIX (make sure [a][b] agrees with [b][a])

# for index_1, name_1 in enumerate(pairing_data['name'].values.tolist()):
#     for index_2, name_2 in enumerate(pairing_data['name'].values.tolist()):
#         value_1 = pairing_data[name_1][index_2]
#         value_2 = pairing_data[name_2][index_1]
        
#         if name_1 == name_2:
#             proper_value = ''
#         elif value_1 == 'D' or value_2 == 'D':
#             proper_value = 'D'
#         elif value_1 == 'C' or value_2 == 'C':
#             proper_value = 'C'
#         elif value_1 == 'd' or value_2 == 'd':
#             proper_value = 'd'
#         elif value_1 == 'c' or value_2 == 'c':
#             proper_value = 'c'
#         else:
#             proper_value = ''
        
#         pairing_data[name_1][index_2] = proper_value

        
#         pairing_data[name_2][index_1] = proper_value

CPU times: user 6.25 s, sys: 152 ms, total: 6.41 s
Wall time: 6.27 s


In [10]:
# %%time

# # REPRESENT PAIRING DATA AS LISTS

# def get_pairs_with_names(row):
#     lower_category_name = pairing_data[pairing_data[row['name']] == 'c']['name'].values.tolist()
#     lower_direct_name = pairing_data[pairing_data[row['name']] == 'd']['name'].values.tolist()
#     upper_category_name = pairing_data[pairing_data[row['name']] == 'C']['name'].values.tolist()
#     upper_direct_name = pairing_data[pairing_data[row['name']] == 'D']['name'].values.tolist()
    
#     row['lower_category_names'] = pairing_data[pairing_data[row['name']] == 'c']['name'].values.tolist()
#     row['lower_direct_names'] = pairing_data[pairing_data[row['name']] == 'd']['name'].values.tolist()
#     row['upper_category_names'] = pairing_data[pairing_data[row['name']] == 'C']['name'].values.tolist()
#     row['upper_direct_names'] = pairing_data[pairing_data[row['name']] == 'D']['name'].values.tolist()
#     row['lower_names'] = row['lower_category_names'] + row['lower_direct_names']
#     row['upper_names'] = row['upper_category_names'] + row['upper_direct_names']
#     row['all_names'] = row['lower_names'] + row['upper_names']
    
# #     row['lower_category_pairs_with_names'] = list(set([lower_category_name for lower_category_name in row['lower_category_names'] if lower_category_name != row['name']]))
# #     row['lower_direct_pairs_with_names'] = list(set([lower_direct_name for lower_direct_name in row['lower_direct_names'] if lower_direct_name != row['name']]))
# #     row['upper_category_pairs_with_names'] = list(set([upper_category_name for upper_category_name in row['upper_category_names'] if upper_category_name != row['name']]))
# #     row['upper_direct_pairs_with_names'] = list(set([upper_direct_name for upper_direct_name in row['upper_direct_names'] if upper_direct_name != row['name']]))
# #     row['lower_pairs_with_names'] = list(set(row['lower_category_pairs_with_names'] + row['lower_direct_pairs_with_names']))
# #     row['upper_pairs_with_names'] = list(set(row['upper_category_pairs_with_names'] + row['upper_direct_pairs_with_names']))
# #     row['all_pairs_with_names'] = list(set(row['lower_pairs_with_names'] + row['upper_pairs_with_names']))
    
#     row['lc_sorted_pairs'] = [tuple(sorted((row['name'], other_name,))) for other_name in row['lower_category_names']]
#     row['ld_sorted_pairs'] = [tuple(sorted((row['name'], other_name,))) for other_name in row['lower_direct_names']]
#     row['uc_sorted_pairs'] = [tuple(sorted((row['name'], other_name,))) for other_name in row['upper_category_names']]
#     row['ud_sorted_pairs'] = [tuple(sorted((row['name'], other_name,))) for other_name in row['upper_direct_names']]
#     row['l_sorted_pairs'] = [tuple(sorted((row['name'], other_name,))) for other_name in row['lower_names']]
#     row['u_sorted_pairs'] = [tuple(sorted((row['name'], other_name,))) for other_name in row['upper_names']]
#     row['a_sorted_pairs'] = [tuple(sorted((row['name'], other_name,))) for other_name in row['all_names']]
    
#     return row

# pairing_data = pairing_data.apply(get_pairs_with_names, axis=1)

CPU times: user 4.6 s, sys: 7.94 ms, total: 4.6 s
Wall time: 4.61 s


In [11]:
# small enough to git commit
# pairing_data.to_pickle(os.path.join(root_path, 'DATA/stir_fry_pairing_data.pickle'))
pairing_data = pd.read_pickle(os.path.join(root_path, 'DATA/stir_fry_pairing_data.pickle'))

# 3. Creating "clashes with" data \[gonna start out without this one, for stir fry\]
### (but still run through, to add it to data just in case?)

In [12]:
# name_name_clashes_blank = stir_fry_data[['name', 'protein_cheese_sub', 'stir_fry_allium', 'fruit', 'veg']].copy()
# names = stir_fry_data['name'].values.tolist()

# for i, col_name in enumerate(names):
#     name_name_clashes_blank[col_name] = pd.Series(['x']*(i+1) + ['']*(len(names)-(i+1)))

# name_name_clashes_blank.to_csv(os.path.join(root_path, 'DATA/name_name_clashes_blank.csv'), index=False)

In [13]:
# name_name_clashes_input = pd.read_csv(os.path.join(root_path, 'DATA/name_name_clashes_input.csv'))
# name_name_clashes_input = name_name_clashes_input.replace([float('nan'), 'x'], '')

In [14]:
# columns = ['name'] + name_name_clashes_input['name'].values.tolist()
# name_name_clashes_input = name_name_clashes_input[columns]

# def get_clashes_with_data(row):
#     data = pd.Series([])
#     name = row['name']
#     data['name'] = name
    
#     lower_names = name_name_clashes_input['name'][name_name_clashes_input[name] == 'y'].values.tolist()
#     upper_names = name_name_clashes_input['name'][name_name_clashes_input[name] == 'Y'].values.tolist()
#     if 'y' in lower_names:
#         print(lower_names)
    
#     for name in name_name_clashes_input['name']:
#         if row[name] == 'y':
#             lower_names.append(name)
#         elif row[name] == 'Y':
#             upper_names.append(name)
    
#     lower_names = list(set(lower_names))
#     upper_names = list(set(upper_names))
    
#     data['lower_clashes_with_names'] = lower_names
#     data['upper_clashes_with_names'] = upper_names
#     data['all_clashes_with_names'] = list(set(lower_names + upper_names)) # shouldn't be overlap here, but hey
    
#     data['lower_clashes_with_pairs'] = [tuple(sorted([name, lower_name])) for lower_name in data['lower_clashes_with_names']]
#     data['upper_clashes_with_pairs'] = [tuple(sorted([name, lower_name])) for lower_name in data['upper_clashes_with_names']]
#     data['all_clashes_with_pairs'] = [tuple(sorted([name, lower_name])) for lower_name in data['all_clashes_with_names']]
    
#     return data

# clashes_with_data = name_name_clashes_input.apply(get_clashes_with_data, axis=1)

/home/russell/flavor_tool/venv/lib/python3.6/site-packages/ipykernel_launcher.py:5: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  """


In [15]:
# # SYNC PAIRING DATA MATRIX (make sure [a][b] agrees with [b][a])

# name_name_clashes_synced = name_name_clashes_input.copy()

# for index_1, name_1 in enumerate(name_name_clashes_input['name'].values.tolist()):
#     for index_2, name_2 in enumerate(name_name_clashes_input['name'].values.tolist()):
#         value_1 = name_name_clashes_input[name_1][index_2]
#         value_2 = name_name_clashes_input[name_2][index_1]
        
#         if name_1 == name_2:
#             proper_value = ''
#         elif value_1 == 'Y' or value_2 == 'Y':
#             proper_value = 'Y'
#         elif value_1 == 'y' or value_2 == 'y':
#             proper_value = 'y'
#         else:
#             proper_value = ''
        
#         name_name_clashes_synced[name_1][index_2] = proper_value
#         name_name_clashes_synced[name_2][index_1] = proper_value

In [16]:
# # add name_name_clashes_synced to clashes_with_data
# clashes_with_data[name_name_clashes_synced['name'].tolist()] = name_name_clashes_synced[name_name_clashes_synced['name'].tolist()].copy()

In [12]:
# clashes_with_data.to_pickle(os.path.join(root_path, 'DATA/stir_fry_clashes_with_data.pickle'))
clashes_with_data = pd.read_pickle(os.path.join(root_path, 'DATA/stir_fry_clashes_with_data.pickle'))

# 4. Adding umbrella data
- Because if 'onion' umbrella pairs with eg. olive oil, all onions should be paired with olive oil as at least c/C.
- So what I'm doing here is
    - matching umbrella-names' terms to regular ol names
    - then marking all ingredients that fall under a specific umbrella
    - then for each reg-term for each umbrella-name, I'll add a c/C connection value if existing connection is lower
- and let the records show I'm deciding that
    - if onions D pair with olive oil, but white onions c pair with olive oil, then imma overwrite c with D
    - and if onions c pair, white onions D pair I'd write D for white onions
    - (so I'm going with the bigger mark, just like when syncing)
### (also, the first part of this doesn't need to be run regularly)

In [20]:
# umbrella_data = ingredients_data[(ingredients_data['stir_fry_yes'] == 'y') & (ingredients_data['stir_fry_useful_umbrella'] == 'y') & (ingredients_data['redirect'] != 'y')]
# umbrella_data.reset_index(drop=True, inplace=True)

# stir_fry_data_with_umbrella = ingredients_data[(ingredients_data['stir_fry_yes'] == 'y') & (ingredients_data['redirect'] != 'y')]
# stir_fry_data_with_umbrella.reset_index(drop=True, inplace=True)

In [23]:
# # break entries in column that has 'pairs with' strings into lists of ingredient terms
# umbrella_ingredient_pairs_with_terms = umbrella_data['pairs_with'].apply(get_terms_from_pairs_with)

In [23]:
# # create list of all terms, ignoring case and excluding duplicates
# umbrella_all_terms = list(set(umbrella_ingredient_pairs_with_terms.sum()))
# umbrella_all_terms_lower = list(set([term.lower() for term in umbrella_ingredient_pairs_with_terms.sum()]))

In [24]:
# # !pip install inflect
# import inflect
# p = inflect.engine()

In [25]:
# salad_matches = pd.read_csv(os.path.join(root_path, 'DATA/term_name_matches_specific.csv'))
# stir_fry_matches = pd.read_csv(os.path.join(root_path, 'DATA/stir_fry_term_name_matches_plus.csv'))
    
# def get_tokens(phrase):
#     tokens = [token.strip() for token in re.split('\(|\)|,|e\.g\.|esp\.|and|—|or|aka|see|see also|;|and\/or|\*', phrase)]
#     tokens = [p.singular_noun(token) or token for token in tokens if token != '']
#     return tokens

# def get_mark(name, term):
#     try:
#         stir_fry_match = stir_fry_matches[term][stir_fry_matches['name'] == name].iloc[0]
#     except:
#         stir_fry_match = None 
#     if stir_fry_match:
#         if str(stir_fry_match) in ['0', 'nan']:
#             return ''
#         else:
#             print('FOUND stir fry match')
#             return stir_fry_match
    
#     try:
#         salad_match = salad_matches[term][salad_matches['name'] == name].iloc[0]
#     except:
#         salad_match = None 
#     if salad_match:
#         if str(salad_match) in ['0', 'nan']:
#             return ''
#         else:
#             print('FOUND salad match')
#             return salad_match
    
#     name_tokens = [token.lower() for token in get_tokens(name)]  
#     term_tokens_mixed = get_tokens(term)
#     term_tokens = [token.lower() for token in term_tokens_mixed]
    
#     primary_name = re.split('\(|—|e\.g\.', name)[0].strip()
#     primary_name_split = primary_name.split(', ')
#     single_comma_primary_name = len(primary_name_split) == 2
    
# #     print()
#     if 'e.g.' in term:
#         if single_comma_primary_name:
#             if name_tokens[0] == term_tokens[0]:
#                 if name_tokens[1] in term_tokens: # specific name in e.g. term
# #                     print('NAME TOKENS In TERM TOKENS')
#                     term_i = term_tokens.index(name_tokens[1])
#                     if term_tokens_mixed[term_i] == term_tokens_mixed[term_i].upper():
#                         match = 'D'
#                     else:
#                         if term_tokens_mixed[0] == term_tokens_mixed[0].upper():
#                             match = 'D'
#                         else:
#                             match = 'd'
#                 else: # name matches only generic (pre e.g.) part of term
#                     if term_tokens_mixed[0] == term_tokens_mixed[0].upper():
#                         match = 'C'
#                     else:
#                         match = 'c'
#             else:
#                 match = ''
#         else:
#             if name_tokens[0] == term_tokens[0]: # name matches term before e.g.
#                 if term_tokens_mixed[0] == term_tokens_mixed[0].upper():
#                     match = 'C'
#                 else:
#                     match = 'c'
#             elif name_tokens[0] in term_tokens[1:]: # name matches term after e.g.
#                 term_i = term_tokens.index(name_tokens[0])
#                 if term_tokens_mixed[term_i] == term_tokens_mixed[term_i].upper():
#                     match = 'D'
#                 else:
#                     if term_tokens_mixed[0] == term_tokens_mixed[0].upper():
#                         match = 'D'
#                     else:
#                         match = 'd'
#             else:
#                 match = ''
#     else:
#         if single_comma_primary_name:
#             if ' '.join(primary_name_split).lower() in term.lower() or primary_name.lower() in term.lower(): # if primary_name is in term, any order
#                 if term_tokens_mixed[0] == term_tokens_mixed[0].upper():
#                     match = 'D'
#                 else:
#                     match = 'd'
#             elif name_tokens[0] == term_tokens[0]: # if first part of primary name matches first token in term
#                 if term_tokens_mixed[0] == term_tokens_mixed[0].upper():
#                     match = 'C'
#                 else:
#                     match = 'c'
#             else:
#                 match = ''
#         else:
#             if name_tokens[0] in term_tokens: # if non-comma name anywhere in non-e.g. term_tokens
#                 if term_tokens_mixed[0] == term_tokens_mixed[0].upper():
#                     match = 'D'
#                 else:
#                     match = 'd'
#             elif len(set(name_tokens).intersection(set(term_tokens))) > 0: # if there are any common tokens
#                 if term_tokens_mixed[0] == term_tokens_mixed[0].upper():
#                     match = 'C'
#                 else:
#                     match = 'c'
#             else:
#                 match = ''
                
#     if match == '':
#         n_common = len(set(name_tokens).intersection(set(term_tokens)))
#         if n_common != 0:
#             match = n_common
#     return match

In [26]:
# umbrella_mark_data = []
# for name in stir_fry_data['name']:
#     print(name)
#     umbrella_mark_data.append([get_mark(name, term) for term in umbrella_all_terms])

# umbrella_term_name_marks = pd.DataFrame(umbrella_mark_data, columns = umbrella_all_terms)
# umbrella_term_name_marks['name'] = pd.Series(stir_fry_data['name'].values.tolist())

# umbrella_term_name_marks.to_csv(os.path.join(root_path, 'DATA/stir_fry_useful_umbrella_term_name_marks.csv'), index=False)

In [18]:
# umbrella_term_name_matches_raw = pd.read_csv(os.path.join(root_path, 'DATA/stir_fry_umbrella_term_name_matches.csv'))
# umbrella_term_name_matches = umbrella_term_name_matches_raw.replace(['0', '1', '2', '3', '4', '5', 0, 1, 2, 3, 4, 5, float('nan')], '')

In [24]:
# %%time
# # CREATE PAIRING DATA MATRIX (names x names)

# umbrella_pairing_data = pd.DataFrame({
#     'name': umbrella_data['name'],
#     'pairs_with_terms': umbrella_ingredient_pairs_with_terms
# })

# for name in stir_fry_data['name']:
#     umbrella_pairing_data[name] = pd.Series(['']*len(umbrella_data))

# def get_pairs_with_names(row):
#     lower_category_names = []
#     lower_direct_names = []
#     upper_category_names = []
#     upper_direct_names = []
#     for term in row['pairs_with_terms']:
#         if term in umbrella_term_name_matches.columns.values.tolist():
#             lower_category_names += umbrella_term_name_matches[umbrella_term_name_matches[term] == 'c']['name'].values.tolist()
#             lower_direct_names += umbrella_term_name_matches[umbrella_term_name_matches[term] == 'd']['name'].values.tolist()
#             upper_category_names += umbrella_term_name_matches[umbrella_term_name_matches[term] == 'C']['name'].values.tolist()
#             upper_direct_names += umbrella_term_name_matches[umbrella_term_name_matches[term] == 'D']['name'].values.tolist()
#         else:
#             pass
#             print(term)
    
#     for lower_category_name in lower_category_names:
#         row[lower_category_name] = 'c'
#     for lower_direct_name in lower_direct_names:
#         row[lower_direct_name] = 'd'
#     for upper_category_name in upper_category_names:
#         row[upper_category_name] = 'C'
#     for upper_direct_name in upper_direct_names:
#         row[upper_direct_name] = 'D'

#     return row

# umbrella_pairing_data = umbrella_pairing_data.apply(get_pairs_with_names, axis=1)
# umbrella_pairing_data.replace(float('nan'), '', inplace=True)

CPU times: user 2.69 s, sys: 0 ns, total: 2.69 s
Wall time: 2.68 s


In [29]:
# # Creating template to match ingredients to their umbrellas
# umbrella_covered_blanks = pd.DataFrame({
#     umbrella_name: ['']*len(stir_fry_data)
# for umbrella_name in umbrella_data['name']})
# umbrella_covered_blanks['name'] = stir_fry_data['name']
# umbrella_covered_blanks.to_csv(os.path.join(root_path, 'DATA/stir_fry_umbrella_covered_blanks.csv'), index=False)

In [25]:
# umbrella_covered_matches_raw = pd.read_csv(os.path.join(root_path, 'DATA/stir_fry_umbrella_covered_matches.csv'))
# umbrella_covered_matches = umbrella_covered_matches_raw.replace([float('nan')], '')
# def get_covered_list(umbrella_name):
#     return umbrella_covered_matches[umbrella_covered_matches[umbrella_name] == 'y']['name'].tolist()

In [26]:
# %%time
# # 40s

# pairing_data_with_umbrella_matches = pairing_data.copy()

# for umbrella_name in umbrella_data['name']: # eg. onions
#     for covered_name in get_covered_list(umbrella_name): # eg. white onion
# #         print(umbrella_name, covered_name)
#         for stir_fry_name in stir_fry_data['name']: # eg. olive oil
#             umbrella_match_value = umbrella_pairing_data[umbrella_pairing_data['name'] == umbrella_name][stir_fry_name].iloc[0] # eg. D (because onions match with OLIVE OIL)
# #             print(umbrella_match_value, umbrella_name, stir_fry_name)
#             stir_fry_match_value = pairing_data[pairing_data['name'] == covered_name][stir_fry_name].iloc[0] # eg. 'c' (because white onions match with oil)
# #             print(umbrella_match_value, stir_fry_match_value)
        
#             if umbrella_match_value == 'D' or stir_fry_match_value == 'D':
#                 proper_match_value = 'D'
#             elif umbrella_match_value == 'C' or stir_fry_match_value == 'C':
#                 proper_match_value = 'C'
#             elif umbrella_match_value == 'd' or stir_fry_match_value == 'd':
#                 proper_match_value = 'd'
#             elif umbrella_match_value == 'c' or stir_fry_match_value == 'c':
#                 proper_match_value = 'c'
#             else:
#                 proper_match_value = ''
                
#             pairing_data_with_umbrella_matches.loc[pairing_data_with_umbrella_matches['name'] == covered_name, stir_fry_name] = proper_match_value
#             pairing_data_with_umbrella_matches.loc[pairing_data_with_umbrella_matches['name'] == stir_fry_name, covered_name] = proper_match_value

CPU times: user 38.8 s, sys: 44 ms, total: 38.9 s
Wall time: 38.9 s


In [27]:
# %%time

# # REPRESENT PAIRING DATA AS LISTS

# def get_pairs_with_names(row):
#     lower_category_name = pairing_data_with_umbrella_matches[pairing_data_with_umbrella_matches[row['name']] == 'c']['name'].values.tolist()
#     lower_direct_name = pairing_data_with_umbrella_matches[pairing_data_with_umbrella_matches[row['name']] == 'd']['name'].values.tolist()
#     upper_category_name = pairing_data_with_umbrella_matches[pairing_data_with_umbrella_matches[row['name']] == 'C']['name'].values.tolist()
#     upper_direct_name = pairing_data_with_umbrella_matches[pairing_data_with_umbrella_matches[row['name']] == 'D']['name'].values.tolist()
    
#     row['lower_category_names'] = pairing_data_with_umbrella_matches[pairing_data_with_umbrella_matches[row['name']] == 'c']['name'].values.tolist()
#     row['lower_direct_names'] = pairing_data_with_umbrella_matches[pairing_data_with_umbrella_matches[row['name']] == 'd']['name'].values.tolist()
#     row['upper_category_names'] = pairing_data_with_umbrella_matches[pairing_data_with_umbrella_matches[row['name']] == 'C']['name'].values.tolist()
#     row['upper_direct_names'] = pairing_data_with_umbrella_matches[pairing_data_with_umbrella_matches[row['name']] == 'D']['name'].values.tolist()
#     row['lower_names'] = row['lower_category_names'] + row['lower_direct_names']
#     row['upper_names'] = row['upper_category_names'] + row['upper_direct_names']
#     row['all_names'] = row['lower_names'] + row['upper_names']
    
# #     row['lower_category_pairs_with_names'] = list(set([lower_category_name for lower_category_name in row['lower_category_names'] if lower_category_name != row['name']]))
# #     row['lower_direct_pairs_with_names'] = list(set([lower_direct_name for lower_direct_name in row['lower_direct_names'] if lower_direct_name != row['name']]))
# #     row['upper_category_pairs_with_names'] = list(set([upper_category_name for upper_category_name in row['upper_category_names'] if upper_category_name != row['name']]))
# #     row['upper_direct_pairs_with_names'] = list(set([upper_direct_name for upper_direct_name in row['upper_direct_names'] if upper_direct_name != row['name']]))
# #     row['lower_pairs_with_names'] = list(set(row['lower_category_pairs_with_names'] + row['lower_direct_pairs_with_names']))
# #     row['upper_pairs_with_names'] = list(set(row['upper_category_pairs_with_names'] + row['upper_direct_pairs_with_names']))
# #     row['all_pairs_with_names'] = list(set(row['lower_pairs_with_names'] + row['upper_pairs_with_names']))
    
#     row['lc_sorted_pairs'] = [tuple(sorted((row['name'], other_name,))) for other_name in row['lower_category_names']]
#     row['ld_sorted_pairs'] = [tuple(sorted((row['name'], other_name,))) for other_name in row['lower_direct_names']]
#     row['uc_sorted_pairs'] = [tuple(sorted((row['name'], other_name,))) for other_name in row['upper_category_names']]
#     row['ud_sorted_pairs'] = [tuple(sorted((row['name'], other_name,))) for other_name in row['upper_direct_names']]
#     row['l_sorted_pairs'] = [tuple(sorted((row['name'], other_name,))) for other_name in row['lower_names']]
#     row['u_sorted_pairs'] = [tuple(sorted((row['name'], other_name,))) for other_name in row['upper_names']]
#     row['a_sorted_pairs'] = [tuple(sorted((row['name'], other_name,))) for other_name in row['all_names']]
    
#     return row

# pairing_data_with_umbrella_matches = pairing_data_with_umbrella_matches.apply(get_pairs_with_names, axis=1)

CPU times: user 2.07 s, sys: 0 ns, total: 2.07 s
Wall time: 2.07 s


In [13]:
# pairing_data_with_umbrella_matches.to_pickle(os.path.join(root_path, 'DATA/stir_fry_pairing_data_with_umbrella_matches.pickle'))
pairing_data_with_umbrella_matches = pd.read_pickle(os.path.join(root_path, 'DATA/stir_fry_pairing_data_with_umbrella_matches.pickle'))

# 5. Creating connection data \[without clashes data, for now\]

In [31]:
# %%time
# # 23s

# # CREATING CONNECTION DATA AND ADDING IT TO stir_fry DATA (& getting stir_fry_data in order)
# # takes a minute
# stir_fry_names = stir_fry_data['name'].values.tolist()
# for col_name in stir_fry_names:
#     col_values = []
#     for row_name in stir_fry_names:
#         pairing_value = pairing_data_with_umbrella_matches[col_name][pairing_data_with_umbrella_matches['name'] == row_name].iloc[0]
# #         clashing_value = clashes_with_data[col_name][clashes_with_data['name'] == row_name].iloc[0]

# #         if clashing_value == 'Y':
# #             col_values.append('N')
# #         elif clashing_value == 'y':
# #             col_values.append('n')
# #         elif pairing_value == 'D':
# #             col_values.append('D')
# #         elif pairing_value == 'C':
# #             col_values.append('C')
# #         elif pairing_value == 'd':
# #             col_values.append('d')
# #         elif pairing_value == 'c':
# #             col_values.append('c')
# #         else:
# #             col_values.append('')
#         col_values.append(pairing_value)

#     stir_fry_data[col_name] = pd.Series(col_values)
    
# # just in case
# stir_fry_data.sort_values('name', inplace=True)
# stir_fry_data.reset_index(drop=True, inplace=True)

/home/russell/flavor_tool/venv/lib/python3.6/site-packages/ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


CPU times: user 27.1 s, sys: 11.7 ms, total: 27.1 s
Wall time: 27.1 s


/home/russell/flavor_tool/venv/lib/python3.6/site-packages/ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [14]:
# stir_fry_data.to_pickle(os.path.join(root_path, 'DATA/stir_fry_data_with_names_and_umbrella_matches.pickle'))
stir_fry_data = pd.read_pickle(os.path.join(root_path, 'DATA/stir_fry_data_with_names_and_umbrella_matches.pickle'))

# 6. Creating flavor tool data

In [33]:
# stir_fry_flavor_data = stir_fry_data[['staple', 'not_vegan', 'gluten', 'flavoring_sweet', 'flavoring_fresh', 'flavoring_wet', 'flavoring_concentrate', 'flavoring', 'flavoring_dry', 'protein', 'protein_cheese_sub', 'protein_milk_sub', 'protein_meat_sub', 'protein_bean', 'veg', 'veg_leafy', 'grain', 'grain_flour', 'fat_oil', 'oil', 'fat', 'fruit', 'fruit_berry', 'stir_fry', 'stir_fry_basic', 'stir_fry_yes', 'stir_fry_early', 'stir_fry_mid', 'stir_fry_late', 'stir_fry_garnish', 'stir_fry_umbrella', 'protein_nut_seed', 'protein_nut', 'protein_seed', 'redirect', 'salty', 'sour', 'spicy', 'bitter', 'savory', 'sweet','name', 'flavor', 'volume', 'pairs_with', 'phonetic', 'techniques', 'dishes', 'tip', 'possible_substitutes', 'flavor_affinities', 'nutritional_profile', 'season', 'botanical_relatives', 'protein_content', 'what_they_are', 'brands', 'vegan_substitutes', 'vegan_brands', 'stir_fry_protein', 'stir_fry_protein_nut_seed', 'stir_fry_protein_nut', 'stir_fry_protein_seed', 'stir_fry_flavoring', 'stir_fry_oil', 'stir_fry_fat', 'stir_fry_fat_oil', 'stir_fry_fruit', 'stir_fry_allium', 'stir_fry_protein_bean', 'stir_fry_veg', 'stir_fry_grain', 'stir_fry_salt', 'stir_fry_pepper', 'stir_fry_vinegar', 'strong', 'stir_fry_mushroom', 'stir_fry_main', 'stir_fry_useful_umbrella'] + stir_fry_data['name'].tolist()].copy()
# stir_fry_flavor_data['id'] = range(len(stir_fry_flavor_data))

# selected_pairing_data = pairing_data_with_umbrella_matches[['name', 'pairs_with_terms', 'lower_category_names', 'lower_direct_names', 'upper_category_names', 'upper_direct_names', 'lower_names', 'upper_names', 'all_names', 'lc_sorted_pairs', 'ld_sorted_pairs', 'uc_sorted_pairs', 'ud_sorted_pairs', 'l_sorted_pairs', 'u_sorted_pairs', 'a_sorted_pairs']]
# stir_fry_flavor_data = stir_fry_flavor_data.merge(selected_pairing_data, how='inner', on='name')

# # selected_clashes_with_data = clashes_with_data[['name', 'lower_clashes_with_names', 'upper_clashes_with_names', 'all_clashes_with_names', 'lower_clashes_with_pairs', 'upper_clashes_with_pairs', 'all_clashes_with_pairs']]
# # stir_fry_flavor_data = stir_fry_flavor_data.merge(selected_clashes_with_data, how='inner', on='name')

# stir_fry_flavor_data.replace(float('nan'), '', inplace=True)
# stir_fry_flavor_data.set_index('name', drop=False, inplace=True)
# stir_fry_flavor_data.rename_axis('name_index', inplace=True)

In [34]:
# %%time
# # 13s

# # ADD STRENGTH VALUES
# ref_flavor_data = stir_fry_flavor_data.copy()
# for i_1, row_1 in ref_flavor_data.iterrows():
#     for i_2, row_2 in ref_flavor_data.iterrows():
#         pairs_with_value = ref_flavor_data[row_1['name']][row_2['name']] or '_'
#         if row_1['strong'] in ['y', 'Y'] and row_2['strong'] in ['y', 'Y']:
#             strength_value = 'S'
#         elif row_1['strong'] in ['y', 'Y'] or row_2['strong'] in ['y', 'Y']:
#             strength_value = 's'
#         else:
#             strength_value = '_'
            
#         stir_fry_flavor_data[row_1['name']][row_2['name']] = pairs_with_value + strength_value

/home/russell/flavor_tool/venv/lib/python3.6/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


CPU times: user 13.1 s, sys: 40 ms, total: 13.1 s
Wall time: 13.1 s


In [15]:
# stir_fry_flavor_data.to_pickle(os.path.join(root_path, 'DATA/stir_fry_flavor_data_with_umbrella.pickle'))
# stir_fry_flavor_data.to_pickle(os.path.join(root_path, '../data/stir_fry_flavor_data_with_umbrella.pickle'))
stir_fry_flavor_data = pd.read_pickle(os.path.join(root_path, 'DATA/stir_fry_flavor_data_with_umbrella.pickle'))

# 7. stir_fry recipe generators

## 7.1. Flavor tool generator

In [50]:
# !pip install networkx==2.4
# !pip install pyvis

import networkx as nx
from pyvis import network as net

You should consider upgrading via the '/home/russell/flavor_tool/venv/bin/python -m pip install --upgrade pip' command.


ModuleNotFoundError: No module named 'networkx'

In [40]:
# THE PLAN
    # is to let 'flavor balance bonus' take care of salt, vinegar, fruit etc. selection
    # the only exception is 1 picked salt in every dish. salt also appears in flavorings, so I'll discourage doubling but not forbid it.

    # Not gonna worry about balancing food groups too much.
        # factor for protein balance? (YES, THIS)
        # or selected protein? (0-3?)
            # but without food group balance OR texture balance, only regulator would be pairs w
    # oh! and maybe a hard limit on flavorings?
        # or a bonus.. but that'd reward a lot, and I really only want a few flavorings per dish? (and for sure a few flavorings)
        # HARD LIMIT IT IS
            # but I think I'll draw from other_flavorings after all, if flavorings and salt are the only other distinct groups
     # and I think fat/oil, like salt, is one of those things where you gotta add a certain amt, but if there's a lot you can just add a little
        # so that's a distinct category, then
        
# UNKNOWNS
    # 7 or 8 max? and will I need to reward balance?
    # special allium category? or just reward alliums? or just onions?
    
n_locked = random.randrange(0, 1)
locked = stir_fry_data.sample(n_locked)
# locked_proteins = locked[locked['protein'] == 'y']
# locked_protein_nut_seeds = locked[locked['stir_fry_protein_nut_seed'] == 'y']
# locked_protein_nuts = locked[locked['stir_fry_protein_nut'] == 'y']
# locked_protein_seeds = locked[locked['stir_fry_protein_seed'] == 'y']
# locked_alliums = locked[locked['stir_fry_allium'] == 'y']
# locked_flavorings = locked[locked['stir_fry_flavoring'] == 'y']
locked_fat_oils = locked[locked['stir_fry_fat_oil'] == 'y']
# locked_fats = locked[locked['stir_fry_fat'] == 'y']
# locked_oils = locked[locked['stir_fry_oil'] == 'y']
# locked_fruits = locked[locked['stir_fry_fruit'] == 'y']
# locked_protein_beans = locked[locked['stir_fry_protein_bean'] == 'y']
# locked_veg = locked[locked['stir_fry_veg'] == 'y']
# locked_grains = locked[locked['stir_fry_grain'] == 'y']
locked_salts = locked[locked['stir_fry_salt'] == 'y']
# locked_peppers = locked[locked['stir_fry_pepper'] == 'y']
# locked_vinegars = locked[locked['stir_fry_vinegar'] == 'y']
locked_other_flavorings = locked[(locked['stir_fry_flavoring'] == 'y') & (locked['stir_fry_salt'] != 'y')]
locked_foodstuffs = locked[(locked['stir_fry_fat_oil'] != 'y') & (locked['stir_fry_salt'] != 'y') & (locked['stir_fry_flavoring'] != 'y')]

the_rest = stir_fry_data[~stir_fry_data['name'].isin(locked['name'])]
# the_rest_proteins = the_rest[the_rest['protein'] == 'y']
# the_rest_protein_nut_seeds = the_rest[the_rest['stir_fry_protein_nut_seed'] == 'y']
# the_rest_protein_nuts = the_rest[the_rest['stir_fry_protein_nut'] == 'y']
# the_rest_protein_seeds = the_rest[the_rest['stir_fry_protein_seed'] == 'y']
# the_rest_alliums = the_rest[the_rest['stir_fry_allium'] == 'y']
# the_rest_flavorings = the_rest[the_rest['stir_fry_flavoring'] == 'y']
the_rest_fat_oils = the_rest[the_rest['stir_fry_fat_oil'] == 'y']
# the_rest_fats = the_rest[the_rest['stir_fry_fat'] == 'y']
# the_rest_oils = the_rest[the_rest['stir_fry_oil'] == 'y']
# the_rest_fruits = the_rest[the_rest['stir_fry_fruit'] == 'y']
# the_rest_protein_beans = the_rest[the_rest['stir_fry_protein_bean'] == 'y']
# the_rest_veg = the_rest[the_rest['stir_fry_veg'] == 'y']
# the_rest_grains = the_rest[the_rest['stir_fry_grain'] == 'y']
the_rest_salts = the_rest[the_rest['stir_fry_salt'] == 'y']
# the_rest_peppers = the_rest[the_rest['stir_fry_pepper'] == 'y']
# the_rest_vinegars = the_rest[the_rest['stir_fry_vinegar'] == 'y']
the_rest_other_flavorings = the_rest[(the_rest['stir_fry_flavoring'] == 'y') & (the_rest['stir_fry_salt'] != 'y')]
the_rest_foodstuffs = the_rest[(the_rest['stir_fry_fat_oil'] != 'y') & (the_rest['stir_fry_salt'] != 'y') & (the_rest['stir_fry_flavoring'] != 'y')]

n_gen_salts = max(1 - len(locked_salts), 0)
n_gen_fat_oils = max(1 - len(locked_fat_oils), 0)
n_gen_other_flavorings_min = max(1 - len(locked_other_flavorings), 0)
n_gen_other_flavorings_max = max(3 - len(locked_other_flavorings), 0)
n_gen_foodstuffs_min = max(3 - len(locked_foodstuffs), 0)
n_gen_foodstuffs_max = max(7 - len(locked_foodstuffs), 0)

In [41]:
# TODO: account for if connected subgraph is impossible

top_score = 0
for try_i in range(100):
    n_subgraphs = 2
    while n_subgraphs > 1: # keep shuffling until you get a well connected graph
        n_gen_other_flavorings = min(random.randrange(n_gen_other_flavorings_min, n_gen_other_flavorings_max+1), len(the_rest_other_flavorings))
        n_gen_foodstuffs = min(random.randrange(n_gen_foodstuffs_min, n_gen_foodstuffs_max+1), len(the_rest_foodstuffs))
        
        selected_salts = locked_salts.append(the_rest_salts.sample(n_gen_salts))
        selected_fat_oils = locked_fat_oils.append(the_rest_fat_oils.sample(n_gen_fat_oils))
        selected_other_flavorings = locked_other_flavorings.append(the_rest_other_flavorings.sample(n_gen_other_flavorings))
        selected_foodstuffs = locked_foodstuffs.append(the_rest_foodstuffs.sample(n_gen_foodstuffs))
        selected_ingredients = selected_salts.append(selected_fat_oils).append(selected_other_flavorings).append(selected_foodstuffs)
        selected_names = selected_ingredients['name'].values.tolist()

        lower_category_pairs = []
        lower_direct_pairs = []
        upper_category_pairs = []
        upper_direct_pairs = []

        # finicky but pretty fast
        for i, col_name in enumerate(selected_names):
            for j, row_name in enumerate(selected_names[i+1:]):
                connection = selected_ingredients[col_name].tolist()[i+1+j] # this is what is finicky
                if connection == 'c':
                    lower_category_pairs.append((col_name, row_name,))
                elif connection == 'd':
                    lower_direct_pairs.append((col_name, row_name,))
                elif connection == 'C':
                    upper_category_pairs.append((col_name, row_name,))
                elif connection == 'D':
                    upper_direct_pairs.append((col_name, row_name,))
        lower_pairs = lower_category_pairs + lower_direct_pairs
        upper_pairs = upper_category_pairs + upper_direct_pairs
        all_pairs = lower_pairs + upper_pairs
        G = nx.Graph()
        G.add_nodes_from(selected_names)
        G.add_edges_from(lower_category_pairs, length=2)
        G.add_edges_from(lower_direct_pairs, length=1.5)
        G.add_edges_from(upper_category_pairs, length=1.2)
        G.add_edges_from(upper_direct_pairs, length=1)
        n_subgraphs = len(list(nx.connected_component_subgraphs(G)))
    score = 0

# PAIRING BONUS ============================================================================================
# ranges from roughly (0 to 1) * 3, tho could be a lil over or under that range
    average_shortest_path_length = nx.average_shortest_path_length(G, weight='length')
    average_shortest_path_score = 1 / average_shortest_path_length * 4 - 1
#     print(average_shortest_path_score)
    score += average_shortest_path_score * 3

# FLAVOR BALANCE BONUS =============================================================================================
# ranges from roughly (0 to 1) * 1 (could be a lil over/under)
    n_sweet_lower = (selected_ingredients['sweet'] == 'y').sum()
    n_sweet_upper = (selected_ingredients['sweet'] == 'Y').sum()
    n_salty_lower = (selected_ingredients['salty'] == 'y').sum()
    n_salty_upper = (selected_ingredients['salty'] == 'Y').sum()
    n_sour_lower = (selected_ingredients['sour'] == 'y').sum()
    n_sour_upper = (selected_ingredients['sour'] == 'Y').sum()
    n_savory_lower = (selected_ingredients['savory'] == 'y').sum()
    n_savory_upper = (selected_ingredients['savory'] == 'Y').sum()
    n_bitter_lower = (selected_ingredients['bitter'] == 'y').sum()
    n_bitter_upper = (selected_ingredients['bitter'] == 'Y').sum()
    n_spicy_lower = (selected_ingredients['spicy'] == 'y').sum()
    n_spicy_upper = (selected_ingredients['spicy'] == 'Y').sum()

    # each varies from roughly .5 to 1 (normalized to the average flavor score)
    sweet_score = (n_sweet_lower/2 + n_sweet_upper)/6
    salty_score = (n_salty_lower/2 + n_salty_upper)/4
    sour_score = (n_sour_lower/2 + n_sour_upper)/2.5
    savory_score = (n_savory_lower/2 + n_savory_upper)/3
    bitter_score = (n_bitter_lower/2 + n_bitter_upper)/3
    spicy_score = (n_spicy_lower/2 + n_spicy_upper)/3

    flavor_balance_score = 5 / (1 + abs(1-sweet_score) + abs(1-salty_score) + abs(1-sour_score) + abs(1-savory_score) + abs(1-spicy_score)) - .9
#     print(flavor_balance_score)
    score += flavor_balance_score
#     print()

# will bias toward larger stir_frys, slightly
# PROTEIN BONUS =========================================================================================
# ranges from roughly (0 to 1) * .5 (mostly balanced on its own)
    n_protein = (selected_ingredients['stir_fry_protein'] == 'y').sum()
    
    # /2 for steep diminishing returns (?)
    protein_score = (n_protein/2)**.5 * .75
#     print(protein_score)
    score += protein_score * .5
    
    if score > top_score:
        top_average_shortest_path_score = average_shortest_path_score
        top_flavor_balance_score = flavor_balance_score
        top_protein_score = protein_score
        top_score = score
        top_lc_pairs = lower_category_pairs
        top_ld_pairs = lower_direct_pairs
        top_uc_pairs = upper_category_pairs
        top_ud_pairs = upper_direct_pairs
        top_selected_ingredients = selected_ingredients
print('RAW PAIRING BONUS', top_average_shortest_path_score)
print('RAW FLAVOR BALANCE BONUS', top_flavor_balance_score)
print('RAW PROTEIN BONUS', top_protein_score)
print('SCORE', top_score)

top_net = net.Network(notebook=True)

nodes = top_selected_ingredients['name'].tolist()

def get_color(row):
#     return 'grey'
    if row['stir_fry_veg'] == 'y':
        return 'green'
    elif row['stir_fry_fruit'] == 'y':
        return 'orange'
    elif row['stir_fry_protein'] == 'y':
        return 'brown'
    elif row['stir_fry_grain'] == 'y':
        return 'tan'
    else:
        return 'lightgrey'
nodes_color = top_selected_ingredients.apply(get_color, axis=1).tolist()

top_net.add_nodes(
    nodes=nodes,
    color=nodes_color
)

for pair in top_lc_pairs:
    top_net.add_edge(pair[0], pair[1], physics=False, color='lightgrey')
for pair in top_ld_pairs:
    top_net.add_edge(pair[0], pair[1], physics=False, color='grey')
for pair in top_uc_pairs:
    top_net.add_edge(pair[0], pair[1], color='darkgrey')
for pair in top_ud_pairs:
    top_net.add_edge(pair[0], pair[1], color='black')

top_net.show('top_net.html')

NameError: name 'nx' is not defined

## 7.2. Black magic

### 7.2.1. Preparing subgraph data
(stuff that can be pre-calculated, pickled)

In [37]:
import networkx as nx
# import community
from networkx.algorithms.clique import enumerate_all_cliques, find_cliques, cliques_containing_node

In [38]:
selected_ingredients = stir_fry_flavor_data.copy()#.sample(200)
selected_names = selected_ingredients['name'].tolist()

lower_category_pairs = []
lower_direct_pairs = []
upper_category_pairs = []
upper_direct_pairs = []

for i_1, name_1 in enumerate(selected_names[:-1]):
    for i_2, name_2 in enumerate(selected_names[i_1+1:], i_1+1):
        connection = selected_ingredients[name_1][name_2] # this is what is finicky
        if connection[0] == 'c':
            lower_category_pairs.append((name_1, name_2,))
        elif connection[0] == 'd':
            lower_direct_pairs.append((name_1, name_2,))
        elif connection[0] == 'C':
            upper_category_pairs.append((name_1, name_2,))
        elif connection[0] == 'D':
            upper_direct_pairs.append((name_1, name_2,))
lower_pairs = lower_category_pairs + lower_direct_pairs
upper_pairs = upper_category_pairs + upper_direct_pairs
all_pairs = lower_pairs + upper_pairs


In [40]:
# %%time
# G = nx.Graph()
# G.add_nodes_from(selected_names)
# G.add_edges_from(upper_category_pairs)
# G.add_edges_from(upper_direct_pairs)

# cliques_gen = find_cliques(G)
# cliques_all = list(cliques_gen)
# cliques = [clique for clique in cliques_all if len(clique) > 1]
# clique_sets = [set(clique) for clique in cliques]

# data = {
#     s_name: ['y' if s_name in c_set else '' for c_set in clique_sets]
# for s_name in selected_names}
# clique_upper = pd.DataFrame(data)
# clique_upper['type'] = ['clique_upper'] * len(clique_upper)
# clique_upper['list'] = cliques
# clique_upper['set'] = clique_sets
# clique_upper['length'] = clique_upper['list'].apply(len)

# clique_upper.to_pickle(os.path.join(root_path, 'DATA/stir_fry_clique_upper.pickle'))
clique_upper = pd.read_pickle(os.path.join(root_path, 'DATA/stir_fry_clique_upper.pickle'))

CPU times: user 523 ms, sys: 12 ms, total: 535 ms
Wall time: 535 ms


In [59]:
# %%time
# G = nx.Graph()
# G.add_nodes_from(selected_names)
# G.add_edges_from(lower_direct_pairs)
# G.add_edges_from(upper_category_pairs)
# G.add_edges_from(upper_direct_pairs)

# cliques_gen = find_cliques(G)
# cliques_all = list(cliques_gen)
# cliques = [clique for clique in cliques_all if len(clique) > 1]
# clique_sets = [set(clique) for clique in cliques]

# data = {
#     s_name: ['y' if s_name in c_set else '' for c_set in clique_sets]
# for s_name in selected_names}
# clique_u_ld = pd.DataFrame(data)
# clique_u_ld['type'] = ['clique_u_ld'] * len(clique_u_ld)
# clique_u_ld['list'] = cliques
# clique_u_ld['set'] = clique_sets
# clique_u_ld['length'] = clique_u_ld['list'].apply(len)

# clique_u_ld.to_pickle(os.path.join(root_path, 'DATA/clique_u_ld.pickle'))

In [60]:
# selected_ingredients = stir_fry_flavor_data[stir_fry_flavor_data['strong'].isin(['y', 'Y'])]
# selected_names = selected_ingredients['name'].tolist()

# lower_category_pairs = []
# lower_direct_pairs = []
# upper_category_pairs = []
# upper_direct_pairs = []

# for i_1, name_1 in enumerate(selected_names[:-1]):
#     for i_2, name_2 in enumerate(selected_names[i_1+1:], i_1+1):
#         connection = selected_ingredients[name_1][name_2] # this is what is finicky
#         if connection[0] == 'c':
#             lower_category_pairs.append((name_1, name_2,))
#         elif connection[0] == 'd':
#             lower_direct_pairs.append((name_1, name_2,))
#         elif connection[0] == 'C':
#             upper_category_pairs.append((name_1, name_2,))
#         elif connection[0] == 'D':
#             upper_direct_pairs.append((name_1, name_2,))
# lower_pairs = lower_category_pairs + lower_direct_pairs
# upper_pairs = upper_category_pairs + upper_direct_pairs
# all_pairs = lower_pairs + upper_pairs


In [61]:
# %%time
# G = nx.Graph()
# G.add_nodes_from(selected_names)
# G.add_edges_from(upper_category_pairs)
# G.add_edges_from(upper_direct_pairs)

# cliques_gen = find_cliques(G)
# cliques_all = list(cliques_gen)
# cliques = [clique for clique in cliques_all if len(clique) > 1]
# clique_sets = [set(clique) for clique in cliques]

# data = {
#     s_name: ['y' if s_name in c_set else '' for c_set in clique_sets]
# for s_name in selected_names}
# clique_upper_strong = pd.DataFrame(data)
# clique_upper_strong['type'] = ['clique_upper_strong'] * len(clique_upper_strong)
# clique_upper_strong['list'] = cliques
# clique_upper_strong['set'] = clique_sets
# clique_upper_strong['length'] = clique_upper_strong['list'].apply(len)

# clique_upper_strong.to_pickle(os.path.join(root_path, 'DATA/clique_upper_strong.pickle'))

In [62]:
# %%time
# G = nx.Graph()
# G.add_nodes_from(selected_names)
# G.add_edges_from(lower_direct_pairs)
# G.add_edges_from(upper_category_pairs)
# G.add_edges_from(upper_direct_pairs)

# cliques_gen = find_cliques(G)
# cliques_all = list(cliques_gen)
# cliques = [clique for clique in cliques_all if len(clique) > 1]
# clique_sets = [set(clique) for clique in cliques]

# data = {
#     s_name: ['y' if s_name in c_set else '' for c_set in clique_sets]
# for s_name in selected_names}
# clique_u_ld_strong = pd.DataFrame(data)
# clique_u_ld_strong['type'] = ['clique_u_ld_strong'] * len(clique_u_ld_strong)
# clique_u_ld_strong['list'] = cliques
# clique_u_ld_strong['set'] = clique_sets
# clique_u_ld_strong['length'] = clique_u_ld_strong['list'].apply(len)

# clique_u_ld_strong.to_pickle(os.path.join(root_path, 'DATA/clique_u_ld_strong.pickle'))

In [63]:
# %%time
# G = nx.Graph()
# G.add_nodes_from(selected_names)
# G.add_edges_from(lower_category_pairs)
# G.add_edges_from(lower_direct_pairs)
# G.add_edges_from(upper_category_pairs)
# G.add_edges_from(upper_direct_pairs)

# cliques_gen = find_cliques(G)
# cliques_all = list(cliques_gen)
# cliques = [clique for clique in cliques_all if len(clique) > 1]
# clique_sets = [set(clique) for clique in cliques]

# data = {
#     s_name: ['y' if s_name in c_set else '' for c_set in clique_sets]
# for s_name in selected_names}
# clique_all_strong = pd.DataFrame(data)
# clique_all_strong['type'] = ['clique_all_strong'] * len(clique_all_strong)
# clique_all_strong['list'] = cliques
# clique_all_strong['set'] = clique_sets
# clique_all_strong['length'] = clique_all_strong['list'].apply(len)

# clique_all_strong.to_pickle(os.path.join(root_path, 'DATA/clique_all_strong.pickle'))

In [41]:
%%time

# sweet_upper_set = set(stir_fry_flavor_data[stir_fry_flavor_data['sweet'] == 'Y']['name'])
# sweet_lower_set = set(stir_fry_flavor_data[stir_fry_flavor_data['sweet'] == 'y']['name'])
# sour_upper_set = set(stir_fry_flavor_data[stir_fry_flavor_data['sour'] == 'Y']['name'])
# sour_lower_set = set(stir_fry_flavor_data[stir_fry_flavor_data['sour'] == 'y']['name'])
# salty_upper_set = set(stir_fry_flavor_data[stir_fry_flavor_data['salty'] == 'Y']['name'])
# salty_lower_set = set(stir_fry_flavor_data[stir_fry_flavor_data['salty'] == 'y']['name'])
# savory_upper_set = set(stir_fry_flavor_data[stir_fry_flavor_data['savory'] == 'Y']['name'])
# savory_lower_set = set(stir_fry_flavor_data[stir_fry_flavor_data['savory'] == 'y']['name'])
# spicy_upper_set = set(stir_fry_flavor_data[stir_fry_flavor_data['spicy'] == 'Y']['name'])
# spicy_lower_set = set(stir_fry_flavor_data[stir_fry_flavor_data['spicy'] == 'y']['name'])
# bitter_upper_set = set(stir_fry_flavor_data[stir_fry_flavor_data['bitter'] == 'Y']['name'])
# bitter_lower_set = set(stir_fry_flavor_data[stir_fry_flavor_data['bitter'] == 'y']['name'])
salt_set = set(stir_fry_flavor_data[stir_fry_flavor_data['stir_fry_salt'] == 'y']['name'])
fat_oil_set = set(stir_fry_flavor_data[stir_fry_flavor_data['stir_fry_fat_oil'] == 'y']['name'])
other_flavoring_set = set(stir_fry_flavor_data[stir_fry_flavor_data['stir_fry_flavoring'] == 'y']['name']) - salt_set
foodstuff_set = set(stir_fry_flavor_data['name']) - salt_set - fat_oil_set - other_flavoring_set
mushroom_set = set(stir_fry_flavor_data[stir_fry_flavor_data['stir_fry_mushroom'] == 'y']['name'])
# PROBS main_set = set(stir_fry_flavor_data['stir_fry_main'] == 'y')
bean_set = set(stir_fry_flavor_data[stir_fry_flavor_data['stir_fry_protein_bean'] == 'y'])
grain_set = set(stir_fry_flavor_data[stir_fry_flavor_data['stir_fry_grain'] == 'y'])

# clique_upper['n_sweet_upper'] = [len(sweet_upper_set.intersection(clique_set)) for clique_set in list(clique_upper['set'])]
# clique_upper['n_sweet_lower'] = [len(sweet_lower_set.intersection(clique_set)) for clique_set in list(clique_upper['set'])]
# clique_upper['n_sour_upper'] = [len(sour_upper_set.intersection(clique_set)) for clique_set in list(clique_upper['set'])]
# clique_upper['n_sour_lower'] = [len(sour_lower_set.intersection(clique_set)) for clique_set in list(clique_upper['set'])]
# clique_upper['n_salty_upper'] = [len(salty_upper_set.intersection(clique_set)) for clique_set in list(clique_upper['set'])]
# clique_upper['n_salty_lower'] = [len(salty_lower_set.intersection(clique_set)) for clique_set in list(clique_upper['set'])]
# clique_upper['n_savory_upper'] = [len(savory_upper_set.intersection(clique_set)) for clique_set in list(clique_upper['set'])]
# clique_upper['n_savory_lower'] = [len(savory_lower_set.intersection(clique_set)) for clique_set in list(clique_upper['set'])]
# clique_upper['n_spicy_upper'] = [len(spicy_upper_set.intersection(clique_set)) for clique_set in list(clique_upper['set'])]
# clique_upper['n_spicy_lower'] = [len(spicy_lower_set.intersection(clique_set)) for clique_set in list(clique_upper['set'])]
# clique_upper['n_bitter_upper'] = [len(bitter_upper_set.intersection(clique_set)) for clique_set in list(clique_upper['set'])]
# clique_upper['n_bitter_lower'] = [len(bitter_lower_set.intersection(clique_set)) for clique_set in list(clique_upper['set'])]
clique_upper['salts'] = [list(salt_set.intersection(clique_set)) for clique_set in list(clique_upper['set'])]
clique_upper['fat_oils'] = [list(fat_oil_set.intersection(clique_set)) for clique_set in list(clique_upper['set'])]
clique_upper['other_flavorings'] = [list(other_flavoring_set.intersection(clique_set)) for clique_set in list(clique_upper['set'])]
clique_upper['foodstuffs'] = [list(foodstuff_set.intersection(clique_set)) for clique_set in list(clique_upper['set'])]
clique_upper['mushrooms'] = [list(mushroom_set.intersection(clique_set)) for clique_set in list(clique_upper['set'])]
# clique_upper['mains'] = [list(main_set.intersection(clique_set)) for clique_set in list(clique_upper['set'])]
clique_upper['beans'] = [list(bean_set.intersection(clique_set)) for clique_set in list(clique_upper['set'])]
clique_upper['grains'] = [list(grain_set.intersection(clique_set)) for clique_set in list(clique_upper['set'])]

clique_upper['salts_set'] = [set(salt_set.intersection(clique_set)) for clique_set in list(clique_upper['set'])]
clique_upper['fat_oils_set'] = [set(fat_oil_set.intersection(clique_set)) for clique_set in list(clique_upper['set'])]
clique_upper['other_flavorings_set'] = [set(other_flavoring_set.intersection(clique_set)) for clique_set in list(clique_upper['set'])]
clique_upper['foodstuffs_set'] = [set(foodstuff_set.intersection(clique_set)) for clique_set in list(clique_upper['set'])]
clique_upper['mushrooms_set'] = [set(mushroom_set.intersection(clique_set)) for clique_set in list(clique_upper['set'])]
# clique_upper['mains_set'] = [set(main_set.intersection(clique_set)) for clique_set in list(clique_upper['set'])]
clique_upper['beans_set'] = [set(bean_set.intersection(clique_set)) for clique_set in list(clique_upper['set'])]
clique_upper['grains_set'] = [set(grain_set.intersection(clique_set)) for clique_set in list(clique_upper['set'])]

# clique_upper['n_sweet_upper'] = [len(sweet_upper_set.intersection(clique_set)) for clique_set in list(clique_upper['set'])]
# clique_upper['n_sweet_lower'] = [len(sweet_lower_set.intersection(clique_set)) for clique_set in list(clique_upper['set'])]
# clique_upper['n_sour_upper'] = [len(sour_upper_set.intersection(clique_set)) for clique_set in list(clique_upper['set'])]
# clique_upper['n_sour_lower'] = [len(sour_lower_set.intersection(clique_set)) for clique_set in list(clique_upper['set'])]
# clique_upper['n_salty_upper'] = [len(salty_upper_set.intersection(clique_set)) for clique_set in list(clique_upper['set'])]
# clique_upper['n_salty_lower'] = [len(salty_lower_set.intersection(clique_set)) for clique_set in list(clique_upper['set'])]
# clique_upper['n_savory_upper'] = [len(savory_upper_set.intersection(clique_set)) for clique_set in list(clique_upper['set'])]
# clique_upper['n_savory_lower'] = [len(savory_lower_set.intersection(clique_set)) for clique_set in list(clique_upper['set'])]
# clique_upper['n_spicy_upper'] = [len(spicy_upper_set.intersection(clique_set)) for clique_set in list(clique_upper['set'])]
# clique_upper['n_spicy_lower'] = [len(spicy_lower_set.intersection(clique_set)) for clique_set in list(clique_upper['set'])]
# clique_upper['n_bitter_upper'] = [len(bitter_upper_set.intersection(clique_set)) for clique_set in list(clique_upper['set'])]
# clique_upper['n_bitter_lower'] = [len(bitter_lower_set.intersection(clique_set)) for clique_set in list(clique_upper['set'])]
clique_upper['n_salts'] = [len(salt_set.intersection(clique_set)) for clique_set in list(clique_upper['set'])]
clique_upper['n_fat_oils'] = [len(fat_oil_set.intersection(clique_set)) for clique_set in list(clique_upper['set'])]
clique_upper['n_other_flavorings'] = [len(other_flavoring_set.intersection(clique_set)) for clique_set in list(clique_upper['set'])]
clique_upper['n_foodstuffs'] = [len(foodstuff_set.intersection(clique_set)) for clique_set in list(clique_upper['set'])]
clique_upper['n_mushrooms'] = [len(mushroom_set.intersection(clique_set)) for clique_set in list(clique_upper['set'])]
# clique_upper['n_mains'] = [len(main_set.intersection(clique_set)) for clique_set in list(clique_upper['set'])]
# clique_upper['n_beans'] = [len(bean_set.intersection(clique_set)) for clique_set in list(clique_upper['set'])]
# clique_upper['n_grains'] = [len(grain_set.intersection(clique_set)) for clique_set in list(clique_upper['set'])]

# clique_u_ld['n_sweet_upper'] = [len(sweet_upper_set.intersection(clique_set)) for clique_set in list(clique_u_ld['set'])]
# clique_u_ld['n_sweet_lower'] = [len(sweet_lower_set.intersection(clique_set)) for clique_set in list(clique_u_ld['set'])]
# clique_u_ld['n_sour_upper'] = [len(sour_upper_set.intersection(clique_set)) for clique_set in list(clique_u_ld['set'])]
# clique_u_ld['n_sour_lower'] = [len(sour_lower_set.intersection(clique_set)) for clique_set in list(clique_u_ld['set'])]
# clique_u_ld['n_salty_upper'] = [len(salty_upper_set.intersection(clique_set)) for clique_set in list(clique_u_ld['set'])]
# clique_u_ld['n_salty_lower'] = [len(salty_lower_set.intersection(clique_set)) for clique_set in list(clique_u_ld['set'])]
# clique_u_ld['n_savory_upper'] = [len(savory_upper_set.intersection(clique_set)) for clique_set in list(clique_u_ld['set'])]
# clique_u_ld['n_savory_lower'] = [len(savory_lower_set.intersection(clique_set)) for clique_set in list(clique_u_ld['set'])]
# clique_u_ld['n_spicy_upper'] = [len(spicy_upper_set.intersection(clique_set)) for clique_set in list(clique_u_ld['set'])]
# clique_u_ld['n_spicy_lower'] = [len(spicy_lower_set.intersection(clique_set)) for clique_set in list(clique_u_ld['set'])]
# clique_u_ld['n_bitter_upper'] = [len(bitter_upper_set.intersection(clique_set)) for clique_set in list(clique_u_ld['set'])]
# clique_u_ld['n_bitter_lower'] = [len(bitter_lower_set.intersection(clique_set)) for clique_set in list(clique_u_ld['set'])]
# clique_u_ld['n_salts'] = [len(salt_set.intersection(clique_set)) for clique_set in list(clique_u_ld['set'])]
# clique_u_ld['n_fat_oils'] = [len(fat_oil_set.intersection(clique_set)) for clique_set in list(clique_u_ld['set'])]
# clique_u_ld['n_other_flavorings'] = [len(other_flavoring_set.intersection(clique_set)) for clique_set in list(clique_u_ld['set'])]
# clique_u_ld['n_foodstuffs'] = [len(foodstuff_set.intersection(clique_set)) for clique_set in list(clique_u_ld['set'])]
# clique_u_ld['n_mushrooms'] = [len(mushroom_set.intersection(clique_set)) for clique_set in list(clique_u_ld['set'])]
# clique_u_ld['n_mains'] = [len(main_set.intersection(clique_set)) for clique_set in list(clique_u_ld['set'])]
# clique_u_ld['n_beans'] = [len(bean_set.intersection(clique_set)) for clique_set in list(clique_u_ld['set'])]
# clique_u_ld['n_grains'] = [len(grain_set.intersection(clique_set)) for clique_set in list(clique_u_ld['set'])]

# clique_upper_strong['n_sweet_upper'] = [len(sweet_upper_set.intersection(clique_set)) for clique_set in list(clique_upper_strong['set'])]
# clique_upper_strong['n_sweet_lower'] = [len(sweet_lower_set.intersection(clique_set)) for clique_set in list(clique_upper_strong['set'])]
# clique_upper_strong['n_sour_upper'] = [len(sour_upper_set.intersection(clique_set)) for clique_set in list(clique_upper_strong['set'])]
# clique_upper_strong['n_sour_lower'] = [len(sour_lower_set.intersection(clique_set)) for clique_set in list(clique_upper_strong['set'])]
# clique_upper_strong['n_salty_upper'] = [len(salty_upper_set.intersection(clique_set)) for clique_set in list(clique_upper_strong['set'])]
# clique_upper_strong['n_salty_lower'] = [len(salty_lower_set.intersection(clique_set)) for clique_set in list(clique_upper_strong['set'])]
# clique_upper_strong['n_savory_upper'] = [len(savory_upper_set.intersection(clique_set)) for clique_set in list(clique_upper_strong['set'])]
# clique_upper_strong['n_savory_lower'] = [len(savory_lower_set.intersection(clique_set)) for clique_set in list(clique_upper_strong['set'])]
# clique_upper_strong['n_spicy_upper'] = [len(spicy_upper_set.intersection(clique_set)) for clique_set in list(clique_upper_strong['set'])]
# clique_upper_strong['n_spicy_lower'] = [len(spicy_lower_set.intersection(clique_set)) for clique_set in list(clique_upper_strong['set'])]
# clique_upper_strong['n_bitter_upper'] = [len(bitter_upper_set.intersection(clique_set)) for clique_set in list(clique_upper_strong['set'])]
# clique_upper_strong['n_bitter_lower'] = [len(bitter_lower_set.intersection(clique_set)) for clique_set in list(clique_upper_strong['set'])]
# clique_upper_strong['n_salt'] = [len(salt_set.intersection(clique_set)) for clique_set in list(clique_upper_strong['set'])]
# clique_upper_strong['n_fat_oil'] = [len(fat_oil_set.intersection(clique_set)) for clique_set in list(clique_upper_strong['set'])]
# clique_upper_strong['n_other_flavoring'] = [len(other_flavoring_set.intersection(clique_set)) for clique_set in list(clique_upper_strong['set'])]

# clique_u_ld_strong['n_sweet_upper'] = [len(sweet_upper_set.intersection(clique_set)) for clique_set in list(clique_u_ld_strong['set'])]
# clique_u_ld_strong['n_sweet_lower'] = [len(sweet_lower_set.intersection(clique_set)) for clique_set in list(clique_u_ld_strong['set'])]
# clique_u_ld_strong['n_sour_upper'] = [len(sour_upper_set.intersection(clique_set)) for clique_set in list(clique_u_ld_strong['set'])]
# clique_u_ld_strong['n_sour_lower'] = [len(sour_lower_set.intersection(clique_set)) for clique_set in list(clique_u_ld_strong['set'])]
# clique_u_ld_strong['n_salty_upper'] = [len(salty_upper_set.intersection(clique_set)) for clique_set in list(clique_u_ld_strong['set'])]
# clique_u_ld_strong['n_salty_lower'] = [len(salty_lower_set.intersection(clique_set)) for clique_set in list(clique_u_ld_strong['set'])]
# clique_u_ld_strong['n_savory_upper'] = [len(savory_upper_set.intersection(clique_set)) for clique_set in list(clique_u_ld_strong['set'])]
# clique_u_ld_strong['n_savory_lower'] = [len(savory_lower_set.intersection(clique_set)) for clique_set in list(clique_u_ld_strong['set'])]
# clique_u_ld_strong['n_spicy_upper'] = [len(spicy_upper_set.intersection(clique_set)) for clique_set in list(clique_u_ld_strong['set'])]
# clique_u_ld_strong['n_spicy_lower'] = [len(spicy_lower_set.intersection(clique_set)) for clique_set in list(clique_u_ld_strong['set'])]
# clique_u_ld_strong['n_bitter_upper'] = [len(bitter_upper_set.intersection(clique_set)) for clique_set in list(clique_u_ld_strong['set'])]
# clique_u_ld_strong['n_bitter_lower'] = [len(bitter_lower_set.intersection(clique_set)) for clique_set in list(clique_u_ld_strong['set'])]
# clique_u_ld_strong['n_salts'] = [len(salt_set.intersection(clique_set)) for clique_set in list(clique_u_ld_strong['set'])]
# clique_u_ld_strong['n_fat_oils'] = [len(fat_oil_set.intersection(clique_set)) for clique_set in list(clique_u_ld_strong['set'])]
# clique_u_ld_strong['n_other_flavorings'] = [len(other_flavoring_set.intersection(clique_set)) for clique_set in list(clique_u_ld_strong['set'])]
# clique_u_ld_strong['n_foodstuffs'] = [len(foodstuff_set.intersection(clique_set)) for clique_set in list(clique_u_ld_strong['set'])]
# clique_u_ld_strong['n_mushrooms'] = [len(mushroom_set.intersection(clique_set)) for clique_set in list(clique_u_ld_strong['set'])]
# clique_u_ld_strong['n_mains'] = [len(main_set.intersection(clique_set)) for clique_set in list(clique_u_ld_strong['set'])]
# clique_u_ld_strong['n_beans'] = [len(bean_set.intersection(clique_set)) for clique_set in list(clique_u_ld_strong['set'])]
# clique_u_ld_strong['n_grains'] = [len(grain_set.intersection(clique_set)) for clique_set in list(clique_u_ld_strong['set'])]

# clique_all_strong['n_sweet_upper'] = [len(sweet_upper_set.intersection(clique_set)) for clique_set in list(clique_all_strong['set'])]
# clique_all_strong['n_sweet_lower'] = [len(sweet_lower_set.intersection(clique_set)) for clique_set in list(clique_all_strong['set'])]
# clique_all_strong['n_sour_upper'] = [len(sour_upper_set.intersection(clique_set)) for clique_set in list(clique_all_strong['set'])]
# clique_all_strong['n_sour_lower'] = [len(sour_lower_set.intersection(clique_set)) for clique_set in list(clique_all_strong['set'])]
# clique_all_strong['n_salty_upper'] = [len(salty_upper_set.intersection(clique_set)) for clique_set in list(clique_all_strong['set'])]
# clique_all_strong['n_salty_lower'] = [len(salty_lower_set.intersection(clique_set)) for clique_set in list(clique_all_strong['set'])]
# clique_all_strong['n_savory_upper'] = [len(savory_upper_set.intersection(clique_set)) for clique_set in list(clique_all_strong['set'])]
# clique_all_strong['n_savory_lower'] = [len(savory_lower_set.intersection(clique_set)) for clique_set in list(clique_all_strong['set'])]
# clique_all_strong['n_spicy_upper'] = [len(spicy_upper_set.intersection(clique_set)) for clique_set in list(clique_all_strong['set'])]
# clique_all_strong['n_spicy_lower'] = [len(spicy_lower_set.intersection(clique_set)) for clique_set in list(clique_all_strong['set'])]
# clique_all_strong['n_bitter_upper'] = [len(bitter_upper_set.intersection(clique_set)) for clique_set in list(clique_all_strong['set'])]
# clique_all_strong['n_bitter_lower'] = [len(bitter_lower_set.intersection(clique_set)) for clique_set in list(clique_all_strong['set'])]
# clique_all_strong['n_salt'] = [len(salt_set.intersection(clique_set)) for clique_set in list(clique_all_strong['set'])]
# clique_all_strong['n_fat_oil'] = [len(fat_oil_set.intersection(clique_set)) for clique_set in list(clique_all_strong['set'])]
# clique_all_strong['n_other_flavoring'] = [len(other_flavoring_set.intersection(clique_set)) for clique_set in list(clique_all_strong['set'])]


CPU times: user 290 ms, sys: 8 ms, total: 298 ms
Wall time: 298 ms


In [68]:
# %%time
# # FOREVERRRR
# # Something is definitely very wrong, here. I'm not gonna worry abt it, tho, cause I'm not using this data atm.

# def get_u_ld_superset_indices(clique_set):
#     return [i for i, value in clique_u_ld['set'].iteritems() if clique_set.issubset(value)]

# clique_upper['u_ld_supersets'] = clique_upper['set'].apply(get_u_ld_superset_indices)
# clique_upper.to_pickle(os.path.join(root_path, 'DATA/clique_upper_with_supersets.pickle'))

# clique_u_ld_strong['u_ld_supersets'] = clique_u_ld_strong['set'].apply(get_u_ld_superset_indices)
# clique_u_ld_strong.to_pickle(os.path.join(root_path, 'DATA/clique_u_ld_strong_with_supersets.pickle'))

In [69]:
# %%time
# # 47, good enough (tried other things, ugh)
# # This SHOULD be working, but (again) I don't need it so just not gonna bother.

# def get_upper_subset_data(clique_set):    
#     data_tuples = [(i, len(value)) for i, value in clique_upper['set'].iteritems() if value.issubset(clique_set)]
#     subset_indices = [data_tuple[0] for data_tuple in data_tuples]
#     subset_lengths = [data_tuple[1] for data_tuple in data_tuples]
#     subset_lengths_sum = sum(subset_lengths)
#     return (subset_indices, subset_lengths, subset_lengths_sum)

# upper_subset_data = clique_u_ld['set'].apply(get_upper_subset_data)
# clique_u_ld['upper_subset_indices'] = upper_subset_data.apply(lambda x: x[0])
# clique_u_ld['upper_subset_lengths'] = upper_subset_data.apply(lambda x: x[1])
# clique_u_ld['upper_subset_lengths_sum'] = upper_subset_data.apply(lambda x: x[2])

In [70]:
# %%time
# # 14s
# # Once again, not gonna bother.

# def get_strong_subset_data(clique_set):    
#     data_tuples = [(i, len(value)) for i, value in clique_u_ld_strong['set'].iteritems() if value.issubset(clique_set)]
#     subset_indices = [data_tuple[0] for data_tuple in data_tuples]
#     subset_lengths = [data_tuple[1] for data_tuple in data_tuples]
#     subset_lengths_sum = sum(subset_lengths)
#     return (subset_indices, subset_lengths, subset_lengths_sum)

# strong_subset_data = clique_u_ld['set'].apply(get_strong_subset_data)
# clique_u_ld['strong_subset_indices'] = strong_subset_data.apply(lambda x: x[0])
# clique_u_ld['strong_subset_lengths'] = strong_subset_data.apply(lambda x: x[1])
# clique_u_ld['strong_subset_lengths_sum'] = strong_subset_data.apply(lambda x: x[2])

In [71]:
# %%time
# # 1m 17s
# # Not rly using this I don't think, so not gonna bother

# def get_connection_density(clique_list):
#     connection_weighted_sum = 0
#     for i_1, name_1 in enumerate(clique_list[:-1]):
#         for i_2, name_2 in enumerate(clique_list[i_1+1:], i_1+1):
#             connection = stir_fry_flavor_data[name_1][name_2]
    
#             if connection[0] == 'c':
#                 pairs_with_score = .2
#             elif connection[0] == 'd':
#                 pairs_with_score = .4
#             elif connection[0] == 'C':
#                 pairs_with_score = .6
#             elif connection[0] == 'D':
#                 pairs_with_score = .8
#             else:
#                 print('OH NO! BAD PAIRING VALUE.')

#             if connection[1] == '_':
#                 strength_score = 0
#             elif connection[1] == 's':
#                 strength_score = .1
#             elif connection[1] == 'S':
#                 strength_score = .2
#             else:
#                 print('OH NO! BAD STRENGTH VALUE.')
#             connection_weight = pairs_with_score + strength_score
#             connection_weighted_sum += connection_weight
#     connection_density = connection_weighted_sum / math.factorial(len(clique_list)-1)
# #     print(connection_density)
#     return connection_density



# clique_u_ld['density'] = clique_u_ld['list'].apply(get_connection_density)

In [42]:
# THESE ARE THE ACTUAL CONTROLS FOR THE GENERATOR (I think? it's been a bit.)
n_salts = 1
n_fat_oils = 1
n_other_flavorings_min = 1
n_other_flavorings_max = 3
n_foodstuffs_min = 3
n_foodstuffs_max = 7
# n_mushrooms_min = 0
mushrooms_cap = 1 # possible to have more than this, if there are locked mushrooms (I don't update sets after locked/gen is established)
n_beans_max = 1
n_grains_max = 1
# n_beans_min = 0
# n_beans_max = 2
# n_grains_min = 0
# n_grains_max = 2

In [43]:
# Important, don't delete.

reasonable_clique_upper = clique_upper.copy()

In [74]:
# %%time
# # len10k: 500ms
# # len1k: 50ms

# # doing this before get_connections_data, cause filtering out nodes would screw w that

# def get_reasonable_mushrooms_data(row):
# #     reasonable_n_salts = min(row['n_salts'], n_salts)
# #     reasonable_n_fat_oils = min(row['n_fat_oils'], n_fat_oils)
# #     reasonable_n_other_flavorings = min(row['n_other_flavorings'], n_other_flavorings_max)
# #     reasonable_n_foodstuffs = min(row['n_foodstuffs'], n_foodstuffs_max)
#     reasonable_n_mushrooms = min(row['n_mushrooms'], mushrooms_cap)
# #     reasonable_n_beans = min(row['n_beans'], n_beans_max)
# #     reasonable_n_grains = min(row['n_grains'], n_grains_max)

#     # what if I need to remove a mushroom, so I do that, but then I also need to remove a foodstuff so I double remove??
#     # best not.
# #     salts_to_remove_set = set(random.sample(row['salts'], row['n_salts'] - reasonable_n_salts))
# #     fat_oils_to_remove_set = set(random.sample(row['fat_oils'], row['n_fat_oils'] - reasonable_n_fat_oils))
# #     other_flavorings_to_remove_set = set(random.sample(row['other_flavorings'], row['n_other_flavorings'] - reasonable_n_other_flavorings))
# #     foodstuffs_to_remove_set = set(random.sample(row['foodstuffs'], row['n_foodstuffs'] - reasonable_n_foodstuffs))
#     mushrooms_to_remove_set = set(random.sample(row['mushrooms'], row['n_mushrooms'] - reasonable_n_mushrooms))
# #     beans_to_remove_set = set(random.sample(row['beans'], row['n_beans'] - reasonable_n_beans))
# #     grains_to_remove_set = set(random.sample(row['grains'], row['n_grains'] - reasonable_n_grains))
    
# #     print(
# #         salts_to_remove_set,
# #         fat_oils_to_remove_set,
# #         other_flavorings_to_remove_set,
# #         foodstuffs_to_remove_set,
# #         mushrooms_to_remove_set,
# #         beans_to_remove_set,
# #         grains_to_remove_set
# #     )
    
# #     reasonable_salts_set = row['salts_set'] - salts_to_remove_set
# #     reasonable_fat_oils_set = row['fat_oils_set'] - fat_oils_to_remove_set
# #     reasonable_other_flavorings_set = row['other_flavorings_set'] - other_flavorings_to_remove_set
# #     reasonable_foodstuffs_set = row['foodstuffs_set'] - foodstuffs_to_remove_set
#     reasonable_mushrooms_set = row['mushrooms_set'] - mushrooms_to_remove_set
#     reasonable_foodstuffs_set = row['foodstuffs_set'] - mushrooms_to_remove_set
# #     reasonable_beans_set = row['beans_set'] - beans_to_remove_set
# #     reasonable_grains_set = row['grains_set'] - grains_to_remove_set
    
# #     reasonable_salts = list(reasonable_salts_set)
# #     reasonable_fat_oils = list(reasonable_fat_oils_set)
# #     reasonable_other_flavorings = list(reasonable_other_flavorings_set)
#     reasonable_foodstuffs = list(reasonable_foodstuffs_set)
#     reasonable_mushrooms = list(reasonable_mushrooms_set)
# #     reasonable_beans = list(reasonable_beans_set)
# #     reasonable_grains = list(reasonable_grains_set)
    
#     reasonable_set = row['set'].copy()
# #     reasonable_set -= salts_to_remove_set
# #     reasonable_set -= fat_oils_to_remove_set
# #     reasonable_set -= other_flavorings_to_remove_set
# #     reasonable_set -= foodstuffs_to_remove_set
#     reasonable_set -= mushrooms_to_remove_set
# #     reasonable_set -= beans_to_remove_set
# #     reasonable_set -= grains_to_remove_set
    
#     reasonable_list = list(reasonable_set)
#     reasonable_length = len(reasonable_list)

#     return (
# #         reasonable_n_salts,
# #         reasonable_n_fat_oils,
# #         reasonable_n_other_flavorings,
# #         reasonable_n_foodstuffs,
#         len(reasonable_foodstuffs),
#         reasonable_n_mushrooms,
# #         reasonable_n_beans,
# #         reasonable_n_grains,
# #         reasonable_salts_set,
# #         reasonable_fat_oils_set,
# #         reasonable_other_flavorings_set,
#         reasonable_foodstuffs_set,
#         reasonable_mushrooms_set,
# #         reasonable_beans_set,
# #         reasonable_grains_set,
# #         reasonable_salts,
# #         reasonable_fat_oils,
# #         reasonable_other_flavorings,
#         reasonable_foodstuffs,
#         reasonable_mushrooms,
# #         reasonable_beans,
# #         reasonable_grains,
#         reasonable_set,
#         reasonable_list,
#         reasonable_length
#     )

# reasonable_mushrooms_data = reasonable_clique_upper.apply(get_reasonable_mushrooms_data, axis=1)

In [75]:
# %%time

# # gotta fiflter out less than 2's

# # reasonable_clique_upper['reasonable_n_salts'] = reasonable_data.apply(lambda x: x[0])
# # reasonable_clique_upper['reasonable_n_fat_oils'] = reasonable_data.apply(lambda x: x[1])
# # reasonable_clique_upper['reasonable_n_other_flavorings'] = reasonable_data.apply(lambda x: x[2])
# reasonable_clique_upper['reasonable_n_foodstuffs'] = reasonable_mushrooms_data.apply(lambda x: x[0])
# reasonable_clique_upper['reasonable_n_mushrooms'] = reasonable_mushrooms_data.apply(lambda x: x[1])
# # reasonable_clique_upper['reasonable_n_beans'] = reasonable_data.apply(lambda x: x[5])
# # reasonable_clique_upper['reasonable_n_grains'] = reasonable_data.apply(lambda x: x[6])
# # reasonable_clique_upper['reasonable_salts_set'] = reasonable_data.apply(lambda x: x[4])
# # reasonable_clique_upper['reasonable_fat_oils_set'] = reasonable_data.apply(lambda x: x[5])
# # reasonable_clique_upper['reasonable_other_flavorings_set'] = reasonable_data.apply(lambda x: x[6])
# reasonable_clique_upper['reasonable_foodstuffs_set'] = reasonable_mushrooms_data.apply(lambda x: x[2])
# reasonable_clique_upper['reasonable_mushrooms_set'] = reasonable_mushrooms_data.apply(lambda x: x[3])
# # reasonable_clique_upper['reasonable_beans_set'] = reasonable_data.apply(lambda x: x[12])
# # reasonable_clique_upper['reasonable_grains_set'] = reasonable_data.apply(lambda x: x[13])
# # reasonable_clique_upper['reasonable_salts'] = reasonable_data.apply(lambda x: x[8])
# # reasonable_clique_upper['reasonable_fat_oils'] = reasonable_data.apply(lambda x: x[9])
# # reasonable_clique_upper['reasonable_other_flavorings'] = reasonable_data.apply(lambda x: x[10])
# reasonable_clique_upper['reasonable_foodstuffs'] = reasonable_mushrooms_data.apply(lambda x: x[4])
# reasonable_clique_upper['reasonable_mushrooms'] = reasonable_mushrooms_data.apply(lambda x: x[5])
# # reasonable_clique_upper['reasonable_beans'] = reasonable_data.apply(lambda x: x[19])
# # reasonable_clique_upper['reasonable_grains'] = reasonable_data.apply(lambda x: x[20])
# reasonable_clique_upper['reasonable_set'] = reasonable_mushrooms_data.apply(lambda x: x[6])
# reasonable_clique_upper['reasonable_list'] = reasonable_mushrooms_data.apply(lambda x: x[7])
# reasonable_clique_upper['reasonable_length'] = reasonable_mushrooms_data.apply(lambda x: x[8])

In [44]:
%%time
# len10k: 500ms
# len1k: 50ms

# doing this before get_connections_data, cause filtering out nodes would screw w that

def get_reasonable_data(row):
    # done first, since it messes w foodstuffs
    reasonable_n_mushrooms = min(row['n_mushrooms'], mushrooms_cap)
    mushrooms_to_remove_set = set(random.sample(row['mushrooms'], row['n_mushrooms'] - reasonable_n_mushrooms))
    reasonable_mushrooms_set = row['mushrooms_set'] - mushrooms_to_remove_set
    reasonable_mushrooms = list(reasonable_mushrooms_set)
#     print(reasonable_n_mushrooms, mushrooms_to_remove_set, reasonable_mushrooms_set, row['mushrooms'])
#     print(mushrooms_to_remove_set)
    deshroomed_foodstuffs_set = row['foodstuffs_set'] - mushrooms_to_remove_set
    deshroomed_foodstuffs = list(deshroomed_foodstuffs_set)
    deshroomed_n_foodstuffs = len(deshroomed_foodstuffs)

    deshroomed_set = row['set'].copy()
    deshroomed_set -= mushrooms_to_remove_set
    deshroomed_list = list(deshroomed_set)
    deshroomed_length = len(deshroomed_list)
    
    reasonable_n_salts = min(row['n_salts'], n_salts)
    reasonable_n_fat_oils = min(row['n_fat_oils'], n_fat_oils)
    reasonable_n_other_flavorings = min(row['n_other_flavorings'], n_other_flavorings_max)
    reasonable_n_foodstuffs = min(deshroomed_n_foodstuffs, n_foodstuffs_max)
#     reasonable_n_mushrooms = min(row['n_mushrooms'], n_mushrooms_max)
#     reasonable_n_beans = min(row['n_beans'], n_beans_max)
#     reasonable_n_grains = min(row['n_grains'], n_grains_max)

    # what if I need to remove a mushroom, so I do that, but then I also need to remove a foodstuff so I double remove??
    # best not.
    salts_to_remove_set = set(random.sample(row['salts'], row['n_salts'] - reasonable_n_salts))
    fat_oils_to_remove_set = set(random.sample(row['fat_oils'], row['n_fat_oils'] - reasonable_n_fat_oils))
    other_flavorings_to_remove_set = set(random.sample(row['other_flavorings'], row['n_other_flavorings'] - reasonable_n_other_flavorings))
    foodstuffs_to_remove_set = set(random.sample(deshroomed_foodstuffs, deshroomed_n_foodstuffs - reasonable_n_foodstuffs))
#     mushrooms_to_remove_set = set(random.sample(row['mushrooms'], row['n_mushrooms'] - reasonable_n_mushrooms))
#     beans_to_remove_set = set(random.sample(row['beans'], row['n_beans'] - reasonable_n_beans))
#     grains_to_remove_set = set(random.sample(row['grains'], row['n_grains'] - reasonable_n_grains))
    
#     print(
#         salts_to_remove_set,
#         fat_oils_to_remove_set,
#         other_flavorings_to_remove_set,
#         foodstuffs_to_remove_set,
#         mushrooms_to_remove_set,
#         beans_to_remove_set,
#         grains_to_remove_set
#     )
    
    reasonable_salts_set = row['salts_set'] - salts_to_remove_set
    reasonable_fat_oils_set = row['fat_oils_set'] - fat_oils_to_remove_set
    reasonable_other_flavorings_set = row['other_flavorings_set'] - other_flavorings_to_remove_set
    reasonable_foodstuffs_set = deshroomed_foodstuffs_set - foodstuffs_to_remove_set
#     reasonable_mushrooms_set = row['mushrooms_set'] - mushrooms_to_remove_set
#     reasonable_beans_set = row['beans_set'] - beans_to_remove_set
#     reasonable_grains_set = row['grains_set'] - grains_to_remove_set
    
    reasonable_salts = list(reasonable_salts_set)
    reasonable_fat_oils = list(reasonable_fat_oils_set)
    reasonable_other_flavorings = list(reasonable_other_flavorings_set)
    reasonable_foodstuffs = list(reasonable_foodstuffs_set)
#     reasonable_mushrooms = list(reasonable_mushrooms_set)
#     reasonable_beans = list(reasonable_beans_set)
#     reasonable_grains = list(reasonable_grains_set)
    
#     reasonable_set = row['set'].copy()
    reasonable_set = deshroomed_set # omitting the copy here (for negligible speed)
    reasonable_set -= salts_to_remove_set
    reasonable_set -= fat_oils_to_remove_set
    reasonable_set -= other_flavorings_to_remove_set
    reasonable_set -= foodstuffs_to_remove_set
#     reasonable_set -= mushrooms_to_remove_set
#     reasonable_set -= beans_to_remove_set
#     reasonable_set -= grains_to_remove_set
    
    reasonable_list = list(reasonable_set)
    reasonable_length = len(reasonable_list)

    return (
        reasonable_n_salts,
        reasonable_n_fat_oils,
        reasonable_n_other_flavorings,
        reasonable_n_foodstuffs,
#         reasonable_n_mushrooms,
#         reasonable_n_beans,
#         reasonable_n_grains,
        reasonable_salts_set,
        reasonable_fat_oils_set,
        reasonable_other_flavorings_set,
        reasonable_foodstuffs_set,
#         reasonable_mushrooms_set,
#         reasonable_beans_set,
#         reasonable_grains_set,
        reasonable_salts,
        reasonable_fat_oils,
        reasonable_other_flavorings,
        reasonable_foodstuffs,
#         reasonable_mushrooms,
#         reasonable_beans,
#         reasonable_grains,
        reasonable_set,
        reasonable_list,
        reasonable_length
    )

reasonable_data = reasonable_clique_upper.apply(get_reasonable_data, axis=1)

CPU times: user 3.59 s, sys: 48 ms, total: 3.64 s
Wall time: 3.64 s


In [45]:
%%time

# gotta filter out less than 2's

# reasonable_clique_upper = clique_upper.copy()

reasonable_clique_upper['reasonable_n_salts'] = reasonable_data.apply(lambda x: x[0])
reasonable_clique_upper['reasonable_n_fat_oils'] = reasonable_data.apply(lambda x: x[1])
reasonable_clique_upper['reasonable_n_other_flavorings'] = reasonable_data.apply(lambda x: x[2])
reasonable_clique_upper['reasonable_n_foodstuffs'] = reasonable_data.apply(lambda x: x[3])
# reasonable_clique_upper['reasonable_n_mushrooms'] = reasonable_data.apply(lambda x: x[4])
# reasonable_clique_upper['reasonable_n_beans'] = reasonable_data.apply(lambda x: x[5])
# reasonable_clique_upper['reasonable_n_grains'] = reasonable_data.apply(lambda x: x[6])
reasonable_clique_upper['reasonable_salts_set'] = reasonable_data.apply(lambda x: x[4])
reasonable_clique_upper['reasonable_fat_oils_set'] = reasonable_data.apply(lambda x: x[5])
reasonable_clique_upper['reasonable_other_flavorings_set'] = reasonable_data.apply(lambda x: x[6])
reasonable_clique_upper['reasonable_foodstuffs_set'] = reasonable_data.apply(lambda x: x[7])
# reasonable_clique_upper['reasonable_mushrooms_set'] = reasonable_data.apply(lambda x: x[11])
# reasonable_clique_upper['reasonable_beans_set'] = reasonable_data.apply(lambda x: x[12])
# reasonable_clique_upper['reasonable_grains_set'] = reasonable_data.apply(lambda x: x[13])
reasonable_clique_upper['reasonable_salts'] = reasonable_data.apply(lambda x: x[8])
reasonable_clique_upper['reasonable_fat_oils'] = reasonable_data.apply(lambda x: x[9])
reasonable_clique_upper['reasonable_other_flavorings'] = reasonable_data.apply(lambda x: x[10])
reasonable_clique_upper['reasonable_foodstuffs'] = reasonable_data.apply(lambda x: x[11])
# reasonable_clique_upper['reasonable_mushrooms'] = reasonable_data.apply(lambda x: x[18])
# reasonable_clique_upper['reasonable_beans'] = reasonable_data.apply(lambda x: x[19])
# reasonable_clique_upper['reasonable_grains'] = reasonable_data.apply(lambda x: x[20])
reasonable_clique_upper['reasonable_set'] = reasonable_data.apply(lambda x: x[12])
reasonable_clique_upper['reasonable_list'] = reasonable_data.apply(lambda x: x[13])
reasonable_clique_upper['reasonable_length'] = reasonable_data.apply(lambda x: x[14])

CPU times: user 123 ms, sys: 0 ns, total: 123 ms
Wall time: 123 ms


In [46]:
# IMPORTANT don't remove:

reasonable_clique_upper = reasonable_clique_upper[reasonable_clique_upper['reasonable_list'].apply(lambda x: len(x) >= 2)]

In [47]:
%%time
# 2m 47s
# fuuuuck not working (oh that's cause I blew connections up)
# anyway, with upper it's only 8s
  
G = nx.Graph()
def get_connections_data(clique_list):
    G.clear()
    G.add_nodes_from(clique_list)
    weighted_edges = []
    connections = []
#     print(clique_list)
    for i_1, name_1 in enumerate(clique_list[:-1]):
        for i_2, name_2 in enumerate(clique_list[i_1+1:], i_1+1):
            connection = stir_fry_flavor_data[name_1][name_2]
    
            # Really, since I'm using upper the only demerit values here will be .5333 and .4. but, I think I've adjusted variation to 0-1 anyway, so it should be ok (?)
            if connection[0] == 'c':
                pairs_with_demerit = .8
            elif connection[0] == 'd':
                pairs_with_demerit = .6666
            elif connection[0] == 'C':
                pairs_with_demerit = .5333
            elif connection[0] == 'D':
                pairs_with_demerit = .4
            else:
                print('OH NO! BAD PAIRING VALUE.')

            if connection[1] == '_':
                strength_demerit = .2
            elif connection[1] == 's':
                strength_demerit = .15
            elif connection[1] == 'S':
                strength_demerit = .1
            else:
                print('OH NO! BAD STRENGTH VALUE.')
                
            connection_length = pairs_with_demerit + strength_demerit
            weighted_edges.append((name_1, name_2, connection_length))
            connections.append((name_1, name_2, connection))
            
    G.add_weighted_edges_from(weighted_edges)
    path_length = nx.average_shortest_path_length(G, weight='weight')
    return (path_length, connections)

connections_data = reasonable_clique_upper['reasonable_list'].apply(get_connections_data)

CPU times: user 3.24 s, sys: 16 ms, total: 3.26 s
Wall time: 3.26 s


In [48]:
reasonable_clique_upper['path_length'] = connections_data.apply(lambda x: x[0])
reasonable_clique_upper['connections'] = connections_data.apply(lambda x: x[1])
reasonable_clique_upper['connection_values'] = connections_data.apply(lambda x: [c[2] for c in x[1]])

In [16]:
# !pip install networkx==2.4
# !pip install pyvis
# !pip install python-louvain
# !pip install matplotlib
import networkx as nx
from pyvis import network as net
import community
import matplotlib.cm as cm
import matplotlib.pyplot as plt

In [50]:
%%time

# THIS IS A CRUCIAL THING, PROLLY NEEDS FINE TUNING
# for example, I'm still getting some high-scored len-3's. good 3's, but stilll..
# ANOTHER OPTION
    # add filler data for remaining nodes, then score like I would in generator

fruit_set = set(stir_fry_flavor_data[stir_fry_flavor_data['stir_fry_fruit'] == 'y']['name'])
protein_set = set(stir_fry_flavor_data[stir_fry_flavor_data['stir_fry_protein'] == 'y']['name'])
bean_set = set(stir_fry_flavor_data[stir_fry_flavor_data['stir_fry_protein_bean'] == 'y']['name'])
grain_set = set(stir_fry_flavor_data[stir_fry_flavor_data['stir_fry_grain'] == 'y']['name'])
    
max_length_upper = max(reasonable_clique_upper['reasonable_length'].tolist())

def get_score(row):
    
    # NODE LENGTH BONUS
    # scale lengths so that max is 10, then square to skew sampling toward long cliques, then scale to 1 max
    nodes_length_score = (row['reasonable_length'] * 10/max_length_upper)**2 / 100

    # now that everything is upper, this is just selecting for ud and strong. so, I'm going to prioritize length for now.
    
    # FOOD GROUPS BONUS
    n_fruit = len(row['reasonable_set'].intersection(fruit_set))
    n_protein = len(row['reasonable_set'].intersection(protein_set))

#     0, .7 or 1
    protein_score = (n_protein/2)**.5

    # very few, but also 0, .7, 1
    fruit_score = (n_fruit/2)**.5
    
    food_group_score = protein_score + fruit_score
    
    # PATH LENGTH BONUS
    path_length_score = 2.3/row['path_length'] - 3.1
    
    # RICE BEANS DEMERIT
    # I'm thinking giving a fat demerit to cliques with too many rice, beans will be faster than what I did with mushrooms
    # Jeez tho, to have both is sooo kludgeeey
    n_beans = len(row['reasonable_set'].intersection(bean_set))
    n_grains = len(row['reasonable_set'].intersection(grain_set))

    # trying to keep this low, to tack onto 0-1 score
    rice_beans_demerit = 0
    if n_beans == 2:
        rice_beans_demerit += .1
    elif n_beans > 2:
        rice_beans_demerit += .25
    if n_grains == 2:
        rice_beans_demerit += .1
    elif n_grains > 2:
        rice_beans_demerit += .25
        
    score = (nodes_length_score*1.6 + path_length_score*1 + food_group_score*.4)/3 - rice_beans_demerit
#     score = path_length_score
#     print(score, row['path_length'], row['length'])
#     print(round(score, 2), round(nodes_length_score, 2), round(food_group_score, 2), round(path_length_score, 2))
    if score > 0:
        return score
    else:
        return 0

reasonable_clique_upper['score'] = reasonable_clique_upper.apply(get_score, axis=1)
reasonable_clique_upper.sort_values('score', ascending=False, inplace=True) #IMPORTANT
reasonable_clique_upper.reset_index(drop=True, inplace=True)

CPU times: user 1.26 s, sys: 40 ms, total: 1.3 s
Wall time: 1.29 s


In [60]:
# reasonable_clique_upper.to_pickle(os.path.join(root_path, 'DATA/stir_fry_reasonable_clique_upper.pickle'))
# reasonable_clique_upper.to_pickle(os.path.join(root_path, '../data/stir_fry_reasonable_clique_upper.pickle'))
# reasonable_clique_upper = pd.read_pickle(os.path.join(root_path, 'DATA/stir_fry_reasonable_clique_upper.pickle'))

In [52]:
%%time

average_score_for_length = dict(reasonable_clique_upper.groupby('reasonable_length')['score'].mean())
def get_average_score(length):
    if length >= 2:
        return average_score_for_length[length]
    else:
        return 0

CPU times: user 2.57 ms, sys: 0 ns, total: 2.57 ms
Wall time: 2.08 ms


In [53]:
# 10/15/2020: this is so I can select weak rando ingredients, that won't clash with clique.
# Should probably be fine-tuned at some pt, this is just top of my head

def get_weak_score(strong):
    if strong == 'Y':
        return 1
    elif strong == 'y':
        return 3
    else:
        return 9
                        
stir_fry_flavor_data['weak_score'] = stir_fry_flavor_data['strong'].apply(get_weak_score)

In [55]:
%%time

# 10/15/2020: this is creating a graph, weighted by how "good" pairings are?

# ugggggh 4.54s
# guess I can't rly do this for all..
stir_fry_names = stir_fry_flavor_data['name']

stir_fry_g = nx.Graph()
stir_fry_g.add_nodes_from(stir_fry_names)
weighted_edges = []
for i_1, name_1 in enumerate(stir_fry_names[:-1]):
    for i_2, name_2 in enumerate(stir_fry_names[i_1+1:], i_1+1):
        connection = stir_fry_flavor_data[name_1][name_2]

        # Really, since I'm using upper the only demerit values here will be .5333 and .4. but, I think I've adjusted variation to 0-1 anyway, so it should be ok (?)
        if connection[0] == 'c':
            pairs_with_demerit = .8
        elif connection[0] == 'd':
            pairs_with_demerit = .6666
        elif connection[0] == 'C':
            pairs_with_demerit = .5333
        elif connection[0] == 'D':
            pairs_with_demerit = .4

        if connection[1] == '_':
            strength_demerit = .2
        elif connection[1] == 's':
            strength_demerit = .15
        elif connection[1] == 'S':
            strength_demerit = .1

        connection_length = pairs_with_demerit + strength_demerit
        weighted_edges.append((name_1, name_2, connection_length))

stir_fry_g.add_weighted_edges_from(weighted_edges)

CPU times: user 374 ms, sys: 8 ms, total: 382 ms
Wall time: 381 ms


In [114]:
# %%time
# # 4.8s, yikes.. good thing I can do it beforehand

# stir_fry_shortest_path_lengths = nx.shortest_path_length(stir_fry_g, weight='weight')

CPU times: user 7 µs, sys: 0 ns, total: 7 µs
Wall time: 9.78 µs


In [115]:
# %%time
# # pretty fast, 6ms
# present_shortest_path_lengths = {outer_key: {inner_key: inner_item for inner_key, inner_item in outer_item.items() if inner_key in present_set}  for outer_key, outer_item in stir_fry_shortest_path_lengths if outer_key in present_set}

CPU times: user 4.18 s, sys: 0 ns, total: 4.18 s
Wall time: 4.18 s


In [117]:
# %%time
# # 4.15s
# # no idea why above stopped working. did I change it by accident?

# stir_fry_shortest_path_lengths = nx.shortest_path_length(stir_fry_g, weight='weight')
# present_shortest_path_lengths = {outer_key: {inner_key: inner_item for inner_key, inner_item in outer_item.items() if inner_key in present_set}  for outer_key, outer_item in stir_fry_shortest_path_lengths if outer_key in present_set}

CPU times: user 4.15 s, sys: 3.75 ms, total: 4.16 s
Wall time: 4.16 s


In [56]:
%%time
# 3.98s
# I do not understand what is happening. It has been a while. Things have changed?

stir_fry_shortest_path_lengths = {key: value for key, value in nx.shortest_path_length(stir_fry_g, weight='weight')}

CPU times: user 3.85 s, sys: 12 ms, total: 3.86 s
Wall time: 3.86 s


In [57]:
# stir_fry_shortest_path_lengths.to_pickle(os.path.join(root_path, 'DATA/stir_fry_shortest_path_lengths.pickle'))
# stir_fry_shortest_path_lengths = pd.read_pickle(os.path.join(root_path, 'DATA/stir_fry_shortest_path_lengths.pickle'))

AttributeError: 'dict' object has no attribute 'to_pickle'

In [61]:
# %%time

# import pickle
# with open(os.path.join(root_path, 'DATA/stir_fry_shortest_path_lengths.pickle'), 'wb') as handle:
#     pickle.dump(stir_fry_shortest_path_lengths, handle, protocol=pickle.HIGHEST_PROTOCOL)
# with open(os.path.join(root_path, '../data/stir_fry_shortest_path_lengths.pickle'), 'wb') as handle:
#     pickle.dump(stir_fry_shortest_path_lengths, handle, protocol=pickle.HIGHEST_PROTOCOL)
# with open(os.path.join(root_path, 'DATA/stir_fry_shortest_path_lengths.pickle'), 'rb') as handle:
#     stir_fry_shortest_path_lengths = pickle.load(handle)

### 7.2.2. Generator set-up
(stuff to do once/call)

In [451]:
%%time
# 30ms, could be a lil faster maybe

# TODO: go through these, comment out unused

# gonna start off assuming all are present
# also gonna start off w no locked
# n_present = random.randrange(len(stir_fry_flavor_data)/2, len(stir_fry_flavor_data)+1)
n_present = len(stir_fry_flavor_data) - 10
present = stir_fry_flavor_data.sample(n_present)
present_set = set(present['name'])
present_strong_set = set(present[present['strong'].isin(['Y', 'y'])]['name'])
# not_present = stir_fry_flavor_data[~stir_fry_flavor_data['name'].isin(present['name'])]
not_present_set = set(stir_fry_flavor_data['name']) - present_set
# not_present_set = set(not_present)
# present_shortest_path_lengths = {outer_key: {inner_key: inner_item for inner_key, inner_item in outer_item.items() if inner_key in present_set}  for outer_key, outer_item in stir_fry_shortest_path_lengths.items() if outer_key in present_set}
present_shortest_path_lengths = {key: stir_fry_shortest_path_lengths[key] for key in stir_fry_shortest_path_lengths if key in present_set}

# n_locked = random.randrange(0, 6)
n_locked = 3
locked = present.sample(n_locked) # random for now, but will be handed down
locked_set = set(locked['name'])
# locked_set_strong = set(locked[locked['strong'].isin(['Y', 'y'])]['name'])
locked_fat_oils = locked[locked['stir_fry_fat_oil'] == 'y']
locked_salts = locked[locked['stir_fry_salt'] == 'y']
locked_other_flavorings = locked[(locked['stir_fry_flavoring'] == 'y') & (locked['stir_fry_salt'] != 'y')]
locked_foodstuffs = locked[(locked['stir_fry_fat_oil'] != 'y') & (locked['stir_fry_salt'] != 'y') & (locked['stir_fry_flavoring'] != 'y')] # why am I selecting not salt? already selecting not flavoring. don't wanna mess tho..
locked_fat_oils_set = set(locked_fat_oils['name'])
locked_salts_set = set(locked_salts['name'])
locked_other_flavorings_set = set(locked_other_flavorings['name'])
locked_foodstuffs_set = set(locked_foodstuffs['name'])

# locked_mushrooms = locked[locked['stir_fry_mushroom'] == 'y']
# locked_beans = locked[locked['stir_fry_grain'] == 'y']
# locked_grains = locked[locked['stir_fry_protein_bean'] == 'y']

the_rest = present[~present['name'].isin(locked['name'])]
the_rest_set = set(the_rest['name'])
the_rest_fat_oils = the_rest[the_rest['stir_fry_fat_oil'] == 'y']
the_rest_salts = the_rest[the_rest['stir_fry_salt'] == 'y']
the_rest_other_flavorings = the_rest[(the_rest['stir_fry_flavoring'] == 'y') & (the_rest['stir_fry_salt'] != 'y')]
the_rest_foodstuffs = the_rest[(the_rest['stir_fry_fat_oil'] != 'y') & (the_rest['stir_fry_salt'] != 'y') & (the_rest['stir_fry_flavoring'] != 'y')]
the_rest_fat_oils_set = set(the_rest_fat_oils['name'])
the_rest_salts_set = set(the_rest_salts['name'])
the_rest_other_flavorings_set = set(the_rest_other_flavorings['name'])
the_rest_foodstuffs_set = set(the_rest_foodstuffs['name'])

the_rest_weak = present[~present['strong'].isin(['y', 'Y'])]
the_rest_weak_set = set(the_rest_weak['name'])
the_rest_weak_fat_oils = the_rest_weak[the_rest_weak['stir_fry_fat_oil'] == 'y']
the_rest_weak_salts = the_rest_weak[the_rest_weak['stir_fry_salt'] == 'y']
the_rest_weak_other_flavorings = the_rest_weak[(the_rest_weak['stir_fry_flavoring'] == 'y') & (the_rest_weak['stir_fry_salt'] != 'y')]
the_rest_weak_foodstuffs = the_rest_weak[(the_rest_weak['stir_fry_fat_oil'] != 'y') & (the_rest_weak['stir_fry_salt'] != 'y') & (the_rest_weak['stir_fry_flavoring'] != 'y')]

n_gen_salts = max(n_salts - len(locked_salts), 0)
n_gen_fat_oils = max(n_fat_oils - len(locked_fat_oils), 0)
n_gen_other_flavorings_min = max(n_other_flavorings_min - len(locked_other_flavorings), 0)
n_gen_other_flavorings_max = max(n_other_flavorings_max - len(locked_other_flavorings), 0) # yikes, I had this as min..
n_gen_foodstuffs_min = max(n_foodstuffs_min - len(locked_foodstuffs), 0)
n_gen_foodstuffs_max = max(n_foodstuffs_max - len(locked_foodstuffs), 0)

n_salts_actual = max(n_salts, len(locked_salts))
n_fat_oils_actual = max(n_fat_oils, len(locked_fat_oils))

# not totally sure thesse are right (that both min and max are compared to locked)
n_other_flavorings_min_actual = max(n_other_flavorings_min, len(locked_other_flavorings))
n_other_flavorings_max_actual = max(n_other_flavorings_max, len(locked_other_flavorings))
n_foodstuffs_min_actual = max(n_foodstuffs_min, len(locked_foodstuffs))
n_foodstuffs_max_actual = max(n_foodstuffs_max, len(locked_foodstuffs))

# n_gen_mushrooms_min = max(n_mushrooms_min - len(locked_mushrooms), 0)
# n_gen_mushrooms_max = max(n_mushrooms_max - len(locked_foodstuffs), 0)
# n_gen_beans_min = max(n_beans_min - len(locked_beans), 0)
# n_gen_beans_max = max(n_beans_max - len(locked_foodstuffs), 0)
# n_gen_grains_min = max(n_grains_min - len(locked_grains), 0)
# n_gen_grains_max = max(n_grains_max - len(locked_foodstuffs), 0)

CPU times: user 26 ms, sys: 3.87 ms, total: 29.9 ms
Wall time: 29.3 ms


In [452]:
%%time 
# 63ms, without slicing
# 15.5ms 1000

# IMPORTANT don't remove

ok_cliques = reasonable_clique_upper[:1000].copy()

CPU times: user 3.61 ms, sys: 40 µs, total: 3.65 ms
Wall time: 3.12 ms


In [453]:
%%time

# 1.5s, without slicing
#150ms, with slicing
#180ms, with the shit I am now pulling T.T

# A note on mushrooms:
    # as is it is possible for locked mushrooms to result in more mushrooms than mushroom_cap.
    # easy enough to update the sets below with fewer mushrooms,
    # but idk if its worth the trouble.
    
# FUCK, just did a lot of changes (accounting for locked in clique), but rly don't wanna go back check work rn..
    # so tireddd
    
def get_ok_data(row):
    updated_set = row['reasonable_set'] - not_present_set
    updated_salts_set = row['reasonable_salts_set'] - not_present_set
    updated_fat_oils_set = row['reasonable_fat_oils_set'] - not_present_set
    updated_other_flavorings_set = row['reasonable_other_flavorings_set'] - not_present_set
    updated_foodstuffs_set = row['reasonable_foodstuffs_set'] - not_present_set
#     updated_mushrooms_set = row['reasonable_mushrooms_set'] - not_present_set
#     updated_beans_set = row['reasonable_beans_set'] - not_present_set
#     updated_grains_set = row['reasonable_grains_set'] - not_present_set
    
    updated_n_salts = len(updated_salts_set)
    updated_n_fat_oils = len(updated_fat_oils_set)
    updated_n_other_flavorings = len(updated_other_flavorings_set)
    updated_n_foodstuffs = len(updated_foodstuffs_set)
#     updated_n_mushrooms = len(updated_mushrooms_set)
#     updated_n_beans = len(updated_beans_set)
#     updated_n_grains = len(updated_grains_set)

#     ok_n_salts = min(updated_n_salts, n_gen_salts)
#     ok_n_fat_oils = min(updated_n_fat_oils, n_gen_fat_oils)
#     ok_n_other_flavorings = min(updated_n_other_flavorings, n_gen_other_flavorings_max)
#     ok_n_foodstuffs = min(updated_n_foodstuffs, n_gen_foodstuffs_max)
#     ok_n_mushrooms = min(updated_n_mushrooms, n_gen_mushrooms_max)
#     ok_n_beans = min(updated_n_beans, n_gen_beans_max)
#     ok_n_grains = min(updated_n_grains, n_gen_grains_max)

    # lessee, to do this right I have to be counting locked + this sor sosmething :-PPPPPP bleh
    # the question is, how many salts do I need to take away from this clique
        # let's say there is 1 locked salt
            # if that locked salt is in clique, none, then
            # if that locked salt ISN"T in clique, another is, take that
            # if that locked salt AND another is in, take the other
    # so the union of set of salts in clique and locked salts cannot be more than n_salts_actual (which is at least as big as locked salts)

    n_salts_so_far = len(updated_salts_set.union(locked_salts_set))
    n_salts_to_remove = max(n_salts_so_far - n_salts_actual, 0)
    salts_to_remove_from_set = updated_salts_set - locked_salts_set
    more_salts_to_remove_set = set(random.sample(salts_to_remove_from_set, n_salts_to_remove))
    
    n_fat_oils_so_far = len(updated_fat_oils_set.union(locked_fat_oils_set))
    n_fat_oils_to_remove = max(n_fat_oils_so_far - n_fat_oils_actual, 0)
    fat_oils_to_remove_from_set = updated_fat_oils_set - locked_fat_oils_set
    more_fat_oils_to_remove_set = set(random.sample(fat_oils_to_remove_from_set, n_fat_oils_to_remove))

    n_other_flavorings_so_far = len(updated_other_flavorings_set.union(locked_other_flavorings_set))
    n_other_flavorings_to_remove = max(n_other_flavorings_so_far - n_other_flavorings_max_actual, 0) # is this right?
    other_flavorings_to_remove_from_set = updated_other_flavorings_set - locked_other_flavorings_set
    more_other_flavorings_to_remove_set = set(random.sample(salts_to_remove_from_set, n_salts_to_remove))

    n_foodstuffs_so_far = len(updated_foodstuffs_set.union(locked_foodstuffs_set))
    n_foodstuffs_to_remove = max(n_foodstuffs_so_far - n_foodstuffs_max_actual, 0) # is this right? thinks so - it's keeping foodstuffs from above max
    foodstuffs_to_remove_from_set = updated_foodstuffs_set - locked_foodstuffs_set
    more_foodstuffs_to_remove_set = set(random.sample(foodstuffs_to_remove_from_set, n_foodstuffs_to_remove))

    
    
#     more_salts_to_remove_set = set(random.sample(updated_salts_set, updated_n_salts - ok_n_salts))
#     more_fat_oils_to_remove_set = set(random.sample(updated_fat_oils_set, updated_n_fat_oils - ok_n_fat_oils))
#     more_other_flavorings_to_remove_set = set(random.sample(updated_other_flavorings_set, updated_n_other_flavorings - ok_n_other_flavorings))
#     more_foodstuffs_to_remove_set = set(random.sample(updated_foodstuffs_set, updated_n_foodstuffs - ok_n_foodstuffs))
#     more_mushrooms_to_remove_set = set(random.sample(updated_mushrooms_set, updated_n_mushrooms - ok_n_mushrooms))
#     more_beans_to_remove_set = set(random.sample(updated_beans_set, updated_n_beans - ok_n_beans))
#     more_grains_to_remove_set = set(random.sample(updated_grains_set, updated_n_grains - ok_n_grains))
    
#     ok_salts_set = updated_salts_set - more_salts_to_remove_set
#     ok_fat_oils_set = updated_fat_oils_set - more_fat_oils_to_remove_set
    ok_other_flavorings_set = updated_other_flavorings_set - more_other_flavorings_to_remove_set
    ok_foodstuffs_set = updated_foodstuffs_set - more_foodstuffs_to_remove_set
#     ok_mushrooms_set = updated_mushrooms_set - more_mushrooms_to_remove_set
#     ok_beans_set = updated_beans_set - more_beans_to_remove_set
#     ok_grains_set = updated_grains_set - more_grains_to_remove_set
    
    ok_set = updated_set # not bothering w copy, for (negligible) speed reasons
    ok_set -= more_salts_to_remove_set
    ok_set -= more_fat_oils_to_remove_set
    ok_set -= more_other_flavorings_to_remove_set
    ok_set -= more_foodstuffs_to_remove_set
#     ok_set -= more_mushrooms_to_remove_set
#     ok_set -= more_beans_to_remove_set
#     ok_set -= more_grains_to_remove_set
    
    ok_list = list(ok_set)
    ok_length = len(ok_list)
    ok_n_locked = len(ok_set.intersection(locked_set))
    ok_n_other_flavorings = len(ok_other_flavorings_set)
    ok_n_foodstuffs = len(ok_foodstuffs_set)
    
    old_score_bonus_factor = row['score']/get_average_score(row['reasonable_length']) # how above/below avg score was previously
    ok_score = get_average_score(ok_length)*old_score_bonus_factor
    ok_score_xtreme = ok_score**2 # skew scores way upward, for use as weights

    # damn but this is kludgily arbitrary
    ok_score_lockified = ok_score + ok_n_locked
    ok_score_lockified_xtreme = ok_score_lockified**2
    
    ok_strong_set = ok_set.intersection(present_strong_set)
    
    #
    return (
        ok_set,
        ok_list,
        ok_length,
        ok_score,
        ok_score_xtreme,
#         ok_n_salts,
#         ok_n_fat_oils,
        ok_n_locked,
        ok_score_lockified,
        ok_score_lockified_xtreme,
        ok_strong_set,
#         ok_n_mushrooms,
#         ok_n_beans,
#         ok_n_grains
        ok_n_other_flavorings,
        ok_n_foodstuffs,
    )
    
ok_data = ok_cliques.apply(get_ok_data, axis=1)

CPU times: user 152 ms, sys: 0 ns, total: 152 ms
Wall time: 152 ms


In [454]:
%%time

ok_cliques['ok_set'] = ok_data.apply(lambda x: x[0])
ok_cliques['ok_list'] = ok_data.apply(lambda x: x[1])
ok_cliques['ok_length'] = ok_data.apply(lambda x: x[2])
ok_cliques['ok_score'] = ok_data.apply(lambda x: x[3])
ok_cliques['ok_score_xtreme'] = ok_data.apply(lambda x: x[4])
# ok_cliques['ok_n_salts'] = ok_data.apply(lambda x: x[5])
# ok_cliques['ok_n_fat_oils'] = ok_data.apply(lambda x: x[6])
ok_cliques['ok_n_locked'] = ok_data.apply(lambda x: x[5])
ok_cliques['ok_score_lockified'] = ok_data.apply(lambda x: x[6])
ok_cliques['ok_score_lockified_xtreme'] = ok_data.apply(lambda x: x[7])
ok_cliques['ok_strong_set'] = ok_data.apply(lambda x: x[8])
# ok_cliques['ok_n_mushrooms'] = ok_data.apply(lambda x: x[9])
# ok_cliques['ok_n_beans'] = ok_data.apply(lambda x: x[10])
# ok_cliques['ok_n_grains'] = ok_data.apply(lambda x: x[11])
ok_cliques['ok_n_other_flavorings'] = ok_data.apply(lambda x: x[9])
ok_cliques['ok_n_foodstuffs'] = ok_data.apply(lambda x: x[10])

# UPDATE: don't think this is actually necessary, since I'll just be sampling going fwd? maybe not, idk
# ok_cliques.sort_values('ok_score', ascending=False, inplace=True) # this is smart to do here, but faster to replace the "copy" step above with this

CPU times: user 15.9 ms, sys: 0 ns, total: 15.9 ms
Wall time: 15.1 ms


In [455]:
ok_cliques['ok_n_locked'].sum()

112

In [456]:
len(ok_cliques)

1000

In [457]:
%%time

# IMPORTANT don't remove.
# UPDATE sorting by ok score, not lockified

ok_cliques = ok_cliques[ok_cliques['ok_list'].apply(lambda x: len(x) >= 2)]
ok_cliques = ok_cliques.sort_values('ok_score', ascending=False)

CPU times: user 14.8 ms, sys: 59 µs, total: 14.9 ms
Wall time: 13.7 ms


In [458]:
# goal: select from non_clique the_rest, using distance from cliques as weights AND fullfilling n_gen_xyz constraints (where possible)

A few things to note around here:
- I've decided I'm going to draw directly out of supersets, rather than core then supersets. few reasons:
    - simpler, faster
    - not much bigger necessarily (or at least increase in size is lost in size variation among cliques), and we'll be removing not_present stuff anyway to make them smaller so actually smaller is bad.

- I'll also be doing a kinda tricksy thing where, instead of filtering out cliques that contain not_present ingredients, I'll just be cutting out the not_present ingredients from those cliques. why:
    - the clique list doesn't include cliques that are subsets of larger cliques (that'd take enumerate_all_cliques, which gives too much
    - it's better to have a slightly smaller big clique than cut that out and choose a small one that's biggest it can be. (there's no award for having clique that's big as it can be)
- I guess also, I'll be scoring partially on core size
- also also, I might include u_ld with core of cliques where nodes are strong, rather than just cliques whose core consists of strong connections

Ok another pretty complicated thing i wanna remember for if doing a blog
- removing present nodes from cliques would fuck w score so
- if I did "enumerate_all_cliques", I could just remove all that didn't include locked and did include a not_present
- and then eliminate all with superset in same dataframe, since the (valid) superset would obviously be better
- so to make this
    - go through same motions but with enumerate_all_cliques?
    - we'll see as we go
- oof tho
    - I am trying this now, and enumerate all cliques for u_ld is taking loooong time
    - might be unworkable
    - I guess I could try recalculating scores, OR
        - just adjust score by some predetermined factor depending on how many not_present get taken away
    - OR
        - leave as is, don't even bother, so what if they don't get sampled perfectly in proportion to actual score
- what I might actually go with
    - calculate average score demerit for removing various numbers of present ingredients, then apply that ( {n_removed: score_demerit} ) according to n removed from each clique
- wait nvm this
    - get average scores for each clique length
    - if a clique has not_present nodes removed, set it's score equal to halfway between old score and average score for new n_nodes
    
I guess while I'm at it here's another funky thing:
- I'll be folding in umbrella ingredients, I think
- gonna do it by
    - calling all of its matches category?
    - then adding umbrella matches on to regular matches if the match is greater than regular match?
    - think so
    
Another thing
- had to do some bleh workarounds to remove extra stuff instead of filtering whole click. bleeehhh so much typing I'm not joing it was horrible

Another thing
- still cocming up w a soln for that beans, grains, mush are all in foodstuffs
- I *really* hope that hasn't already fucked me over (my assuming those were exclusive)
    - Update: was fucked over a lil
- soln: forgedaboudit
    - removing mush, bean, grain entirely. can always add it back in.
    
Some more weird stuff
- makin sure ok_data takes account of mix of locked, unlocked when it say s there's too muchhhh

In [459]:
# %%time
# selected_clique = ok_cliques.sample(1).iloc[0]
# the_rest_sans_clique_set = the_rest_set - selected_clique['ok_set']
# the_rest_sans_clique_names = list(the_rest_sans_clique_set)

In [460]:
# %%time
# #supa dupa fast

# center_name = random.sample(selected_clique['ok_strong_set'], k=1)[0]
# items_from_center = {key: value for key, value in present_shortest_path_lengths[strong_center_name].items() if key != center_name}
# names_from_center = [key for key, item in items_from_center.items()]
# lengths_from_center = [item for key, item in items_from_center.items()]

In [461]:
# %%time
# #supa dupa fast
# # NO IDEA why above wasn't working when I got back. Is this even the most recent nb???

# strong_center_name = random.sample(selected_clique['ok_strong_set'], k=1)[0]
# items_from_center = {key: value for key, value in present_shortest_path_lengths[strong_center_name].items() if key != center_name}
# names_from_center = [key for key, item in items_from_center.items()]
# lengths_from_center = [item for key, item in items_from_center.items()]

In [462]:
# %%time

# lengths_from_clique_strongs_tuples = []
# for present_name in list(present_set):
#     if not present_name in selected_clique['ok_set']:
#         lengths_from_each_strong = [present_shortest_path_lengths[center_name][present_name] for center_name in list(selected_clique['ok_strong_set'])]
#         lengths_from_clique_strongs_tuples.append((present_name, sum(lengths_from_each_strong) / len(lengths_from_each_strong)))

In [463]:
# %%time
# # fast: 8ms
# # do this w each addition? w each strong addition?

# lengths_from_clique_tuples = []
# for present_name in list(present_set):
#     if not present_name in selected_clique['ok_set']:
#         lengths_from_each_clique_node = [present_shortest_path_lengths[clique_node][present_name] for clique_node in selected_clique['ok_list']]
#         lengths_from_clique_tuples.append((present_name, sum(lengths_from_each_clique_node) / len(lengths_from_each_clique_node)))

In [464]:
# %%time

# selected_names = selected_clique['ok_list']


# def get_sort_key(length_tuple):
#     return length_tuple[1]

In [465]:
# %%time
# # fast: 8ms
# # just changed selected_clique['ok list'] out for selected_names - so fuckin fast why now?
# # and doing that outside is fast too? wtf?
# # huh, it's the clique thing in the comprehension - bizarre, but now I know!
# # so for sure do this each time, now
# # worth weighting by strong?
# # oh! you know what? Each value is already weighted :-D


# lengths_from_clique_tuples = []
# for present_name in list(present_set):
#     if not present_name in selected_names:
#         lengths_from_each_clique_node = [present_shortest_path_lengths[clique_node][present_name] for clique_node in selected_names]
#         lengths_from_clique_tuples.append((present_name, sum(lengths_from_each_clique_node) / len(lengths_from_each_clique_node)))
        
# lengths_from_clique_tuples.sort(key=get_sort_key)

In [466]:
# %%time
# # super duper fast

# def get_sort_key(length_tuple):
#     return length_tuple[1]

# lengths_from_clique_strongs_tuples.sort(key=get_sort_key)
# lengths_from_clique_tuples.sort(key=get_sort_key)

In [467]:
# already sorted, so I guess I could just find the closest eg. salt

In [468]:
# locked_salts_set

### 7.2.3. Generator
(contains code that will execute many times / call)

In [469]:
# # %%time
# # 60ms/iteration with no score
# # same for with pairing bonus, weirdly. I guess nx is crazy fast?
# # same with flavor bonus?? bizarre.
# # k, more like 45 without thing I forgot to cocmment

# # is lockified stuff enough to include locked? should I be doing multiple stages, 2 locked then 1 then none? or something?

# # WILL have to deal with disconnected graphs

# def get_sort_key(length_tuple):
#     return length_tuple[1]

# # pretty weird that I'm prolly gonna use density here, but average_shortest_path_length in scoring cliques..
# # I guess it's fair I'm using the fast one where it matters?
# # except with nodes, density would make *more* sense since it wouldn't be giving disconnected gaphs a too-high score
# def n_possible_edges(n_nodes):
#     return int((n_nodes*(n_nodes-1)) / 2)

# def get_first_name_in_set(sorted_tuples, food_set):
#     for sorted_tuple in sorted_tuples:
#         if sorted_tuple[0] in food_set:
#             return sorted_tuple[0]
   
# # n_other_flavorings_atual = max(n_other_flavorings_min)

# # n_gen_salts = max(n_salts - len(locked_salts), 0)
# # n_gen_fat_oils = max(n_fat_oils - len(locked_fat_oils), 0)
# # n_gen_other_flavorings_min = max(n_other_flavorings_min - len(locked_other_flavorings), 0)
# # n_gen_other_flavorings_max = max(n_other_flavorings_min - len(locked_other_flavorings), 0)
# # n_gen_foodstuffs_min = max(n_foodstuffs_min - len(locked_foodstuffs), 0)
# # n_gen_foodstuffs_max = max(n_foodstuffs_max - len(locked_foodstuffs), 0)

# # n_gen_other_flavorings = min(random.randrange(n_gen_other_flavorings_min, n_gen_other_flavorings_max+1), len(the_rest_other_flavorings))
# # n_gen_foodstuffs = min(random.randrange(n_gen_foodstuffs_min, n_gen_foodstuffs_max+1), len(the_rest_foodstuffs))

# n_other_flavorings_actual = random.randrange(n_other_flavorings_min_actual, n_other_flavorings_max_actual+1)
# n_foodstuffs_actual = random.randrange(n_foodstuffs_min_actual, n_foodstuffs_max_actual+1)

# n_gen_other_flavorings_actual = n_other_flavorings_actual - len(locked_other_flavorings)
# n_gen_foodstuffs_actual = n_foodstuffs_actual - len(locked_foodstuffs)
# print('N GEN OTHER FLAVORINGS ACTUAL', n_gen_other_flavorings_actual)
# print('N GEN FOODSTUFFS ACTUAL', n_gen_foodstuffs_actual)

# final_cliques = ok_cliques[(ok_cliques['ok_n_other_flavorings'] <= n_gen_other_flavorings_actual) & (ok_cliques['ok_n_foodstuffs'] <= n_gen_foodstuffs_actual)]

# # edge case: no cliques left (meh)
# # clique = ok_cliques[:100].sample(1, weights='ok_score_lockified_xtreme').iloc[0] # to skew sample toward top
# clique = ok_cliques.sample(1, weights='ok_score_xtreme').iloc[0] # to skew sample toward top

# try:
#     clique_ingredients = present.loc[clique['ok_list']]
# except:
#     print('MAJOR PROBLEM! Likely cause: clique_data contains not-present ingredients.')
#     clique_ingredients = stir_fry_flavor_data.loc[clique['ok_list']]

# salts_so_far_set = clique['ok_set'].intersection(salt_set).union(locked_salts_set)
# fat_oils_so_far_set = clique['ok_set'].intersection(fat_oil_set).union(locked_fat_oils_set)
# other_flavorings_so_far_set = clique['ok_set'].intersection(other_flavoring_set).union(locked_other_flavorings_set)
# foodstuffs_so_far_set = clique['ok_set'].intersection(foodstuff_set).union(locked_foodstuffs_set)
# ingredients_so_far_set = clique['ok_set'].union(locked_set)

# n_additional_salts = n_salts_actual - len(salts_so_far_set) # pretty sure there shouldn't be more salts so far than salts actual
# n_additional_fat_oils = n_fat_oils_actual - len(fat_oils_so_far_set) # pretty sure there shouldn't be more salts so far than salts actual
# n_additional_other_flavorings = max(n_other_flavorings_actual - len(other_flavorings_so_far_set), 0) # pretty sure there shouldn't be more salts so far than salts actual
# n_additional_foodstuffs = max(n_foodstuffs_actual - len(foodstuffs_so_far_set), 0) # pretty sure there shouldn't be more salts so far than salts actual
# # n_locked_clique_other_flavorings
# # n_locked_clique_foodstuffs
# # n_locked_clique_salts
# # n_locked_clique_fat_oils

# # or could I rethink things to just, deal with that clique is combined locked & not locked?
# # also, don't think I'm adding locked, below, to selected ingredients
# # how about, add clique foodstuffs to the locked foodstuffs NOT in clique, to get all foodstuffs except additional
# # or, make a set combining clique and locked, then use that to determine what's still needed

# # Kludgy hack to accomodate for possibility that clique has more of a thing than gen (though it should have less than gen_max)
# # says 'don't select more' in that case. there are worse things, I guess...
# # n_additional_other_flavorings = max(n_gen_other_flavorings - clique['ok_n_other_flavorings'], 0)
# # n_additional_foodstuffs = max(n_gen_foodstuffs - clique['ok_n_foodstuffs'], 0)
# print('foodstuffs min/max', n_foodstuffs_min, n_foodstuffs_max)
# print('actual foodstuffs min/max', n_foodstuffs_min_actual, n_foodstuffs_max_actual)
# print('locked foodstuffs', len(locked_foodstuffs))
# print('actual foodstuffs', n_foodstuffs_actual)#, clique['ok_n_foodstuffs'])
# print('foodstuffs_so_far_set_len', len(foodstuffs_so_far_set))
# print('n_additional_foodstuffs', n_additional_foodstuffs)
# print('n gen foodstuffs actual', n_gen_foodstuffs_actual)

# # For most of these, tho, we should have already assured that n <= gen
# # n_additional_salts = n_gen_salts - clique['ok_n_salts'] # take the number of salts we need and subtract the number of salts present in the clique we selected
# # n_additional_fat_oils = n_gen_fat_oils - clique['ok_n_fat_oils']

# # print('flav, food, salt, fat', n_additional_other_flavorings, n_additional_foodstuffs, n_additional_salts, n_additional_fat_oils)
# # selected_names = clique_ingredients['name'].tolist()
# # additional_foodstuff_names = []
# # for i in range(n_additional_foodstuffs):
# #     lengths_from_selected_tuples = []
# #     for present_name in list(present_set):
# #         if not present_name in selected_names:
# #             lengths_from_each_clique_node = [present_shortest_path_lengths[clique_node][present_name] for clique_node in selected_names]
# #             lengths_from_selected_tuples.append((present_name, sum(lengths_from_each_clique_node) / len(lengths_from_each_clique_node)))

# #     lengths_from_selected_tuples.sort(key=get_sort_key)    
# #     additional_foodstuff_names.append(get_first_name_in_set(lengths_from_selected_tuples, foodstuff_set))
# #     print(selected_names)
# #     print(additional_foodstuff_names)
# #     print()        
            
# # lengths_from_clique_tuples.sort(key=get_sort_key)

# # so even if I was gonna sample at random what I NEED to be sampling is unlocked - clique
# additional_salts = the_rest_salts[~the_rest_salts['name'].isin(clique_ingredients['name'])].sample(n_additional_salts, weights='weak_score')
# additional_fat_oils = the_rest_fat_oils[~the_rest_fat_oils['name'].isin(clique_ingredients['name'])].sample(n_additional_fat_oils, weights='weak_score')
# additional_other_flavorings = the_rest_other_flavorings[~the_rest_other_flavorings['name'].isin(clique_ingredients['name'])].sample(n_additional_other_flavorings, weights='weak_score')
# additional_foodstuffs = the_rest_foodstuffs[~the_rest_foodstuffs['name'].isin(clique_ingredients['name'])].sample(n_additional_foodstuffs, weights='weak_score')

# ingredients_so_far = present[present['name'].isin(ingredients_so_far_set)]

# # selected_salts = locked_salts.append(the_rest_salts[~the_rest_salts['name'].isin(clique_ingredients['name'])].sample(n_gen_salts, weights='weak_score'))
# # selected_fat_oils = locked_fat_oils.append(the_rest_fat_oils.sample(n_gen_fat_oils, weights='weak_score'))
# # selected_other_flavorings = locked_other_flavorings.append(the_rest_other_flavorings.sample(n_gen_other_flavorings, weights='weak_score'))
# # selected_foodstuffs = locked_foodstuffs.append(the_rest_foodstuffs.sample(n_gen_foodstuffs, weights='weak_score'))
# # selected_ingredients = clique_ingredients.append(additional_salts).append(additional_fat_oils).append(additional_other_flavorings).append(additional_foodstuffs)
# selected_ingredients = ingredients_so_far.append(additional_salts).append(additional_fat_oils).append(additional_other_flavorings).append(additional_foodstuffs)
# # selected_ingredients = clique_ingredients
# selected_names = selected_ingredients['name'].values.tolist()

# #     lower_category_pairs = []
# #     lower_direct_pairs = []
# #     upper_category_pairs = []
# #     upper_direct_pairs = []
# connections = []
# weighted_edges = []

# for i_1, name_1 in enumerate(selected_names[:-1]):
#     for i_2, name_2 in enumerate(selected_names[i_1+1:], i_1+1):
#         connection = selected_ingredients[name_1][name_2]
#         if connection[0] != '_':
# #                 print(connection)
    
#                 # Really, since I'm using upper the only demerit values here will be .5333 and .4. but, I think I've adjusted variation to 0-1 anyway, so it should be ok (?)
#             if connection[0] == 'c':
#                 pairs_with_demerit = .8
#             elif connection[0] == 'd':
#                 pairs_with_demerit = .6666
#             elif connection[0] == 'C':
#                 pairs_with_demerit = .5333
#             elif connection[0] == 'D':
#                 pairs_with_demerit = .4
#             else:
#                 print('OH NO! BAD PAIRING VALUE.')

#             if connection[1] == '_':
#                 strength_demerit = .2
#             elif connection[1] == 's':
#                 strength_demerit = .15
#             elif connection[1] == 'S':
#                 strength_demerit = .1
#             else:
#                 print('OH NO! BAD STRENGTH VALUE.')
                
                
#             connection_weight = pairs_with_demerit + strength_demerit
#             weighted_edges.append((name_1, name_2, connection_weight))
#             connections.append((name_1, name_2, connection))

# selected_g = nx.Graph()
# selected_g.add_nodes_from(selected_names)
# selected_g.add_weighted_edges_from(weighted_edges)
# score = 0

# # PAIRING BONUS ============================================================================================
# # ranges from roughly (0 to 1) * 3, tho could be a lil over or under that range
# average_shortest_path_length = nx.average_shortest_path_length(selected_g, weight='weight')
# average_shortest_path_score = 1 / average_shortest_path_length * 4 - 1
# score += average_shortest_path_score * 3

# # # FLAVOR BALANCE BONUS =============================================================================================
# # # ranges from roughly (0 to 1) * 1 (could be a lil over/under)
# # n_sweet_lower = (selected_ingredients['sweet'] == 'y').sum()
# # n_sweet_upper = (selected_ingredients['sweet'] == 'Y').sum()
# # n_salty_lower = (selected_ingredients['salty'] == 'y').sum()
# # n_salty_upper = (selected_ingredients['salty'] == 'Y').sum()
# # n_sour_lower = (selected_ingredients['sour'] == 'y').sum()
# # n_sour_upper = (selected_ingredients['sour'] == 'Y').sum()
# # n_savory_lower = (selected_ingredients['savory'] == 'y').sum()
# # n_savory_upper = (selected_ingredients['savory'] == 'Y').sum()
# # n_bitter_lower = (selected_ingredients['bitter'] == 'y').sum()
# # n_bitter_upper = (selected_ingredients['bitter'] == 'Y').sum()
# # n_spicy_lower = (selected_ingredients['spicy'] == 'y').sum()
# # n_spicy_upper = (selected_ingredients['spicy'] == 'Y').sum()

# # # each varies from roughly .5 to 1 (normalized to the average flavor score)
# # sweet_score = (n_sweet_lower/2 + n_sweet_upper)/6
# # salty_score = (n_salty_lower/2 + n_salty_upper)/4
# # sour_score = (n_sour_lower/2 + n_sour_upper)/2.5
# # savory_score = (n_savory_lower/2 + n_savory_upper)/3
# # bitter_score = (n_bitter_lower/2 + n_bitter_upper)/3
# # spicy_score = (n_spicy_lower/2 + n_spicy_upper)/3
# # print(sweet_score, salty_score, sour_score, savory_score, bitter_score, spicy_score)

# # # damn but this seems arbitrary 7 kludgy
# # flavor_balance_score = 1 / (1 + abs(1-sweet_score) + abs(1-salty_score) + abs(1-sour_score) + abs(1-savory_score) + abs(1-spicy_score))
# # # print(flavor_balance_score)
# # score += flavor_balance_score
# #     print()

    
# # FOOD GROUPS BONUS ==========================================================================================
# n_fruit = (selected_ingredients['stir_fry_fruit'] == 'y').sum()
# n_protein = (selected_ingredients['stir_fry_protein'] == 'y').sum()

# # very very few, so realistically max .7
# fruit_score = (n_fruit/2)**.5

# #     0, .7, occaaaasionally 1
# protein_score = (n_protein/2)**.5

# # print(fruit_score, protein_score)
# food_group_score = protein_score + fruit_score
# score += food_group_score

# # # will bias toward larger stir_frys, slightly
# # # PROTEIN BONUS =========================================================================================
# # # ranges from roughly (0 to 1) * .5 (mostly balanced on its own)
# #     n_protein = (selected_ingredients['stir_fry_protein'] == 'y').sum()
    
# #     # /2 for steep diminishing returns (?)
# #     protein_score = (n_protein/2)**.5 * .75
# # #     print(protein_score)
# #     score += protein_score * .5
    
# #     if score > top_score:
# #         top_average_shortest_path_score = average_shortest_path_score
# #         top_flavor_balance_score = flavor_balance_score
# #         top_protein_score = protein_score
# #         top_score = score
# #         top_lc_pairs = lower_category_pairs
# #         top_ld_pairs = lower_direct_pairs
# #         top_uc_pairs = upper_category_pairs
# #         top_ud_pairs = upper_direct_pairs
# #         top_selected_ingredients = selected_ingredients
# #         top_G = G
# # print('RAW PAIRING BONUS', top_average_shortest_path_score)
# # print('RAW FLAVOR BALANCE BONUS', top_flavor_balance_score)
# # print('RAW PROTEIN BONUS', top_protein_score)
# # print('SCORE', top_score)

# cliqued_net = net.Network(notebook=True)

# nodes = selected_ingredients['name'].tolist()

# def get_color(row):
# #     return 'grey'
#     if row['stir_fry_veg'] == 'y':
#         return 'green'
#     elif row['stir_fry_fruit'] == 'y':
#         return 'orange'
#     elif row['stir_fry_protein'] == 'y':
#         return 'brown'
#     elif row['stir_fry_grain'] == 'y':
#         return 'tan'
#     else:
#         return 'lightgrey'
# nodes_color = selected_ingredients.apply(get_color, axis=1).tolist()

# cliqued_net.add_nodes(
#     nodes=nodes,
#     color=nodes_color
# )

# for connection in connections:
#     if connection[2][0] == 'c':
#         cliqued_net.add_edge(connection[0], connection[1], physics=False, color='lightgrey')
#     if connection[2][0] == 'd':
#         cliqued_net.add_edge(connection[0], connection[1], physics=False, color='grey')
#     if connection[2][0] == 'C':
#         cliqued_net.add_edge(connection[0], connection[1], color='darkgrey')
#     if connection[2][0] == 'D':
#         cliqued_net.add_edge(connection[0], connection[1], color='black')
 
# # print(score, average_shortest_path_length)
# cliqued_net.show('cliqued_net.html')

# # ISSUES
#     # now that clique can include locked, is that messing me up?
#         # cause I was assuming clique to be gen, but really it's a combo of gen and locked..
#     # clique seems to be reliably more than gen_foodstuffs. like, by big margin.
#         # could I take notes on how many foodstuffs each clique has, then draw from ones where it's smaller???
#         # pity it'd be filtering (not smallening), buuut hey fuck it
#         # but then, if person locks a bunch, wouldn't they want more foodstuffs?
#         # MOAR IDEAS
#             # it's really just foodstuffs and gen flavorings. would be no problem if there wasn't a range bn min/max, so that's an option.
#             # could also generate sm, md, lg stir fry sizes (pair flavorings & regulars?)
#             # or even make versions for each possible n foodstuffs
#             # split into multiple rows?
#             # idk man. suuure seems nice to just narrow the min-max
#             # OR discard more than is needed?
#                 # let's say it's 3-7, with 3 locked already
#                 # wait, tho. that means there's more than usual shorties, but it's still pulled at random from high scored
#             # OR discard random num?
#             # OR sample more from small, if num is small? tho that still omits goodies inside of big ones.
#             # radical - what if aim was more like, find clique that encompasses locked, then draw from that?????
#             # THIS^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
#                 # & in absence of locked?
#                     # I guess it's, like, do I pick big then whittle it down? or filter to small then pick one? which would lead to higher score?
#                     # I guess I'd want to pick one that has at least enough of everything, then discard excess
#                     # should think more about what alg leads to big pic big score

N GEN OTHER FLAVORINGS ACTUAL 0
N GEN FOODSTUFFS ACTUAL 3
foodstuffs min/max 3 7
actual foodstuffs min/max 3 7
locked foodstuffs 0
actual foodstuffs 3
foodstuffs_so_far_set_len 3
n_additional_foodstuffs 0
n gen foodstuffs actual 3


In [490]:
ok_cliques.sample(0)

,"ALMONDS (and UNSWEETENED ALMOND BUTTER; see also MILK, ALMOND)",ANISE SEEDS,"APPLES (and APPLE CIDER, APPLE JUICE and/or APPLESAUCE)",ARTICHOKE HEARTS (see also ARTICHOKES),"ARTICHOKES, JERUSALEM (aka SUNCHOKES)",ARUGULA (aka ROCKET),ASPARAGUS,BASIL,BAY LEAF,"BEANS, BLACK (aka TURTLE BEANS)",...,ok_list,ok_length,ok_score,ok_score_xtreme,ok_n_locked,ok_score_lockified,ok_score_lockified_xtreme,ok_strong_set,ok_n_other_flavorings,ok_n_foodstuffs


In [522]:
# # %%time

# # FILTERING FOR FINAL CLIQUES IN THIS ONE (cliques that don't have more other_flavorings or foodsteffs than the generated actual)

# # 60ms/iteration with no score
# # same for with pairing bonus, weirdly. I guess nx is crazy fast?
# # same with flavor bonus?? bizarre.
# # k, more like 45 without thing I forgot to cocmment

# # is lockified stuff enough to include locked? should I be doing multiple stages, 2 locked then 1 then none? or something?

# # WILL have to deal with disconnected graphs

# def get_sort_key(length_tuple):
#     return length_tuple[1]

# # pretty weird that I'm prolly gonna use density here, but average_shortest_path_length in scoring cliques..
# # I guess it's fair I'm using the fast one where it matters?
# # except with nodes, density would make *more* sense since it wouldn't be giving disconnected gaphs a too-high score
# def n_possible_edges(n_nodes):
#     return int((n_nodes*(n_nodes-1)) / 2)

# def get_first_name_in_set(sorted_tuples, food_set):
#     for sorted_tuple in sorted_tuples:
#         if sorted_tuple[0] in food_set:
#             return sorted_tuple[0]
   
# # n_other_flavorings_atual = max(n_other_flavorings_min)

# # n_gen_salts = max(n_salts - len(locked_salts), 0)
# # n_gen_fat_oils = max(n_fat_oils - len(locked_fat_oils), 0)
# # n_gen_other_flavorings_min = max(n_other_flavorings_min - len(locked_other_flavorings), 0)
# # n_gen_other_flavorings_max = max(n_other_flavorings_min - len(locked_other_flavorings), 0)
# # n_gen_foodstuffs_min = max(n_foodstuffs_min - len(locked_foodstuffs), 0)
# # n_gen_foodstuffs_max = max(n_foodstuffs_max - len(locked_foodstuffs), 0)

# # n_gen_other_flavorings = min(random.randrange(n_gen_other_flavorings_min, n_gen_other_flavorings_max+1), len(the_rest_other_flavorings))
# # n_gen_foodstuffs = min(random.randrange(n_gen_foodstuffs_min, n_gen_foodstuffs_max+1), len(the_rest_foodstuffs))

# n_other_flavorings_actual = random.randrange(n_other_flavorings_min_actual, n_other_flavorings_max_actual+1)
# n_foodstuffs_actual = random.randrange(n_foodstuffs_min_actual, n_foodstuffs_max_actual+1)

# n_gen_other_flavorings_actual = n_other_flavorings_actual - len(locked_other_flavorings)
# n_gen_foodstuffs_actual = n_foodstuffs_actual - len(locked_foodstuffs)

# final_cliques = ok_cliques[(ok_cliques['ok_n_other_flavorings'] <= n_gen_other_flavorings_actual) & (ok_cliques['ok_n_foodstuffs'] <= n_gen_foodstuffs_actual)]

# # edge case: no cliques left (meh)
# # clique = ok_cliques[:100].sample(1, weights='ok_score_lockified_xtreme').iloc[0] # to skew sample toward top
# if len(final_cliques) > 0:
#     clique = final_cliques.sample(1, weights='ok_score_xtreme').iloc[0] # to skew sample toward top
# else:
#     clique = final_cliques.sample(0)
    
# try:
#     clique_ingredients = present.loc[clique['ok_list']]
# except:
#     print('MAJOR PROBLEM! Likely cause: clique_data contains not-present ingredients.')
#     clique_ingredients = stir_fry_flavor_data.loc[clique['ok_list']]

# salts_so_far_set = clique['ok_set'].intersection(salt_set).union(locked_salts_set)
# fat_oils_so_far_set = clique['ok_set'].intersection(fat_oil_set).union(locked_fat_oils_set)
# other_flavorings_so_far_set = clique['ok_set'].intersection(other_flavoring_set).union(locked_other_flavorings_set)
# foodstuffs_so_far_set = clique['ok_set'].intersection(foodstuff_set).union(locked_foodstuffs_set)
# ingredients_so_far_set = clique['ok_set'].union(locked_set)

# n_additional_salts = n_salts_actual - len(salts_so_far_set) # pretty sure there shouldn't be more salts so far than salts actual
# n_additional_fat_oils = n_fat_oils_actual - len(fat_oils_so_far_set) # pretty sure there shouldn't be more salts so far than salts actual
# n_additional_other_flavorings = max(n_other_flavorings_actual - len(other_flavorings_so_far_set), 0) # pretty sure there shouldn't be more salts so far than salts actual
# n_additional_foodstuffs = max(n_foodstuffs_actual - len(foodstuffs_so_far_set), 0) # pretty sure there shouldn't be more salts so far than salts actual
# # n_locked_clique_other_flavorings
# # n_locked_clique_foodstuffs
# # n_locked_clique_salts
# # n_locked_clique_fat_oils

# # or could I rethink things to just, deal with that clique is combined locked & not locked?
# # also, don't think I'm adding locked, below, to selected ingredients
# # how about, add clique foodstuffs to the locked foodstuffs NOT in clique, to get all foodstuffs except additional
# # or, make a set combining clique and locked, then use that to determine what's still needed

# # Kludgy hack to accomodate for possibility that clique has more of a thing than gen (though it should have less than gen_max)
# # says 'don't select more' in that case. there are worse things, I guess...
# # n_additional_other_flavorings = max(n_gen_other_flavorings - clique['ok_n_other_flavorings'], 0)
# # n_additional_foodstuffs = max(n_gen_foodstuffs - clique['ok_n_foodstuffs'], 0)
# print('foodstuffs min/max', n_foodstuffs_min, n_foodstuffs_max)
# print('actual foodstuffs min/max', n_foodstuffs_min_actual, n_foodstuffs_max_actual)
# print('locked foodstuffs', len(locked_foodstuffs))
# print('actual foodstuffs', n_foodstuffs_actual)#, clique['ok_n_foodstuffs'])
# print('foodstuffs_so_far_set_len', len(foodstuffs_so_far_set))
# print('n_additional_foodstuffs', n_additional_foodstuffs)
# print('n gen foodstuffs actual', n_gen_foodstuffs_actual)

# # For most of these, tho, we should have already assured that n <= gen
# # n_additional_salts = n_gen_salts - clique['ok_n_salts'] # take the number of salts we need and subtract the number of salts present in the clique we selected
# # n_additional_fat_oils = n_gen_fat_oils - clique['ok_n_fat_oils']

# # print('flav, food, salt, fat', n_additional_other_flavorings, n_additional_foodstuffs, n_additional_salts, n_additional_fat_oils)
# # selected_names = clique_ingredients['name'].tolist()
# # additional_foodstuff_names = []
# # for i in range(n_additional_foodstuffs):
# #     lengths_from_selected_tuples = []
# #     for present_name in list(present_set):
# #         if not present_name in selected_names:
# #             lengths_from_each_clique_node = [present_shortest_path_lengths[clique_node][present_name] for clique_node in selected_names]
# #             lengths_from_selected_tuples.append((present_name, sum(lengths_from_each_clique_node) / len(lengths_from_each_clique_node)))

# #     lengths_from_selected_tuples.sort(key=get_sort_key)    
# #     additional_foodstuff_names.append(get_first_name_in_set(lengths_from_selected_tuples, foodstuff_set))
# #     print(selected_names)
# #     print(additional_foodstuff_names)
# #     print()        
            
# # lengths_from_clique_tuples.sort(key=get_sort_key)

# # so even if I was gonna sample at random what I NEED to be sampling is unlocked - clique
# additional_salts = the_rest_salts[~the_rest_salts['name'].isin(clique_ingredients['name'])].sample(n_additional_salts, weights='weak_score')
# additional_fat_oils = the_rest_fat_oils[~the_rest_fat_oils['name'].isin(clique_ingredients['name'])].sample(n_additional_fat_oils, weights='weak_score')
# additional_other_flavorings = the_rest_other_flavorings[~the_rest_other_flavorings['name'].isin(clique_ingredients['name'])].sample(n_additional_other_flavorings, weights='weak_score')
# additional_foodstuffs = the_rest_foodstuffs[~the_rest_foodstuffs['name'].isin(clique_ingredients['name'])].sample(n_additional_foodstuffs, weights='weak_score')

# ingredients_so_far = present[present['name'].isin(ingredients_so_far_set)]

# # selected_salts = locked_salts.append(the_rest_salts[~the_rest_salts['name'].isin(clique_ingredients['name'])].sample(n_gen_salts, weights='weak_score'))
# # selected_fat_oils = locked_fat_oils.append(the_rest_fat_oils.sample(n_gen_fat_oils, weights='weak_score'))
# # selected_other_flavorings = locked_other_flavorings.append(the_rest_other_flavorings.sample(n_gen_other_flavorings, weights='weak_score'))
# # selected_foodstuffs = locked_foodstuffs.append(the_rest_foodstuffs.sample(n_gen_foodstuffs, weights='weak_score'))
# # selected_ingredients = clique_ingredients.append(additional_salts).append(additional_fat_oils).append(additional_other_flavorings).append(additional_foodstuffs)
# selected_ingredients = ingredients_so_far.append(additional_salts).append(additional_fat_oils).append(additional_other_flavorings).append(additional_foodstuffs)
# # selected_ingredients = clique_ingredients
# selected_names = selected_ingredients['name'].values.tolist()

# #     lower_category_pairs = []
# #     lower_direct_pairs = []
# #     upper_category_pairs = []
# #     upper_direct_pairs = []
# connections = []
# weighted_edges = []

# for i_1, name_1 in enumerate(selected_names[:-1]):
#     for i_2, name_2 in enumerate(selected_names[i_1+1:], i_1+1):
#         connection = selected_ingredients[name_1][name_2]
#         if connection[0] != '_':
# #                 print(connection)
    
#                 # Really, since I'm using upper the only demerit values here will be .5333 and .4. but, I think I've adjusted variation to 0-1 anyway, so it should be ok (?)
#             if connection[0] == 'c':
#                 pairs_with_demerit = .8
#             elif connection[0] == 'd':
#                 pairs_with_demerit = .6666
#             elif connection[0] == 'C':
#                 pairs_with_demerit = .5333
#             elif connection[0] == 'D':
#                 pairs_with_demerit = .4
#             else:
#                 print('OH NO! BAD PAIRING VALUE.')

#             if connection[1] == '_':
#                 strength_demerit = .2
#             elif connection[1] == 's':
#                 strength_demerit = .15
#             elif connection[1] == 'S':
#                 strength_demerit = .1
#             else:
#                 print('OH NO! BAD STRENGTH VALUE.')
                
                
#             connection_weight = pairs_with_demerit + strength_demerit
#             weighted_edges.append((name_1, name_2, connection_weight))
#             connections.append((name_1, name_2, connection))

# selected_g = nx.Graph()
# selected_g.add_nodes_from(selected_names)
# selected_g.add_weighted_edges_from(weighted_edges)
# score = 0

# # PAIRING BONUS ============================================================================================
# # ranges from roughly (0 to 1) * 3, tho could be a lil over or under that range
# average_shortest_path_length = nx.average_shortest_path_length(selected_g, weight='weight')
# average_shortest_path_score = 1 / average_shortest_path_length * 4 - 1
# score += average_shortest_path_score * 3

# # # FLAVOR BALANCE BONUS =============================================================================================
# # # ranges from roughly (0 to 1) * 1 (could be a lil over/under)
# # n_sweet_lower = (selected_ingredients['sweet'] == 'y').sum()
# # n_sweet_upper = (selected_ingredients['sweet'] == 'Y').sum()
# # n_salty_lower = (selected_ingredients['salty'] == 'y').sum()
# # n_salty_upper = (selected_ingredients['salty'] == 'Y').sum()
# # n_sour_lower = (selected_ingredients['sour'] == 'y').sum()
# # n_sour_upper = (selected_ingredients['sour'] == 'Y').sum()
# # n_savory_lower = (selected_ingredients['savory'] == 'y').sum()
# # n_savory_upper = (selected_ingredients['savory'] == 'Y').sum()
# # n_bitter_lower = (selected_ingredients['bitter'] == 'y').sum()
# # n_bitter_upper = (selected_ingredients['bitter'] == 'Y').sum()
# # n_spicy_lower = (selected_ingredients['spicy'] == 'y').sum()
# # n_spicy_upper = (selected_ingredients['spicy'] == 'Y').sum()

# # # each varies from roughly .5 to 1 (normalized to the average flavor score)
# # sweet_score = (n_sweet_lower/2 + n_sweet_upper)/6
# # salty_score = (n_salty_lower/2 + n_salty_upper)/4
# # sour_score = (n_sour_lower/2 + n_sour_upper)/2.5
# # savory_score = (n_savory_lower/2 + n_savory_upper)/3
# # bitter_score = (n_bitter_lower/2 + n_bitter_upper)/3
# # spicy_score = (n_spicy_lower/2 + n_spicy_upper)/3
# # print(sweet_score, salty_score, sour_score, savory_score, bitter_score, spicy_score)

# # # damn but this seems arbitrary 7 kludgy
# # flavor_balance_score = 1 / (1 + abs(1-sweet_score) + abs(1-salty_score) + abs(1-sour_score) + abs(1-savory_score) + abs(1-spicy_score))
# # # print(flavor_balance_score)
# # score += flavor_balance_score
# #     print()

    
# # FOOD GROUPS BONUS ==========================================================================================
# n_fruit = (selected_ingredients['stir_fry_fruit'] == 'y').sum()
# n_protein = (selected_ingredients['stir_fry_protein'] == 'y').sum()

# # very very few, so realistically max .7
# fruit_score = (n_fruit/2)**.5

# #     0, .7, occaaaasionally 1
# protein_score = (n_protein/2)**.5

# # print(fruit_score, protein_score)
# food_group_score = protein_score + fruit_score
# score += food_group_score

# # # will bias toward larger stir_frys, slightly
# # # PROTEIN BONUS =========================================================================================
# # # ranges from roughly (0 to 1) * .5 (mostly balanced on its own)
# #     n_protein = (selected_ingredients['stir_fry_protein'] == 'y').sum()
    
# #     # /2 for steep diminishing returns (?)
# #     protein_score = (n_protein/2)**.5 * .75
# # #     print(protein_score)
# #     score += protein_score * .5
    
# #     if score > top_score:
# #         top_average_shortest_path_score = average_shortest_path_score
# #         top_flavor_balance_score = flavor_balance_score
# #         top_protein_score = protein_score
# #         top_score = score
# #         top_lc_pairs = lower_category_pairs
# #         top_ld_pairs = lower_direct_pairs
# #         top_uc_pairs = upper_category_pairs
# #         top_ud_pairs = upper_direct_pairs
# #         top_selected_ingredients = selected_ingredients
# #         top_G = G
# # print('RAW PAIRING BONUS', top_average_shortest_path_score)
# # print('RAW FLAVOR BALANCE BONUS', top_flavor_balance_score)
# # print('RAW PROTEIN BONUS', top_protein_score)
# # print('SCORE', top_score)

# cliqued_net = net.Network(notebook=True)

# nodes = selected_ingredients['name'].tolist()

# def get_color(row):
# #     return 'grey'
#     if row['stir_fry_veg'] == 'y':
#         return 'green'
#     elif row['stir_fry_fruit'] == 'y':
#         return 'orange'
#     elif row['stir_fry_protein'] == 'y':
#         return 'brown'
#     elif row['stir_fry_grain'] == 'y':
#         return 'tan'
#     else:
#         return 'lightgrey'
# nodes_color = selected_ingredients.apply(get_color, axis=1).tolist()

# cliqued_net.add_nodes(
#     nodes=nodes,
#     color=nodes_color
# )

# for connection in connections:
#     if connection[2][0] == 'c':
#         cliqued_net.add_edge(connection[0], connection[1], physics=False, color='lightgrey')
#     if connection[2][0] == 'd':
#         cliqued_net.add_edge(connection[0], connection[1], physics=False, color='grey')
#     if connection[2][0] == 'C':
#         cliqued_net.add_edge(connection[0], connection[1], color='darkgrey')
#     if connection[2][0] == 'D':
#         cliqued_net.add_edge(connection[0], connection[1], color='black')
 
# # print(score, average_shortest_path_length)
# cliqued_net.show('cliqued_net.html')

# # ISSUES
#     # now that clique can include locked, is that messing me up?
#         # cause I was assuming clique to be gen, but really it's a combo of gen and locked..
#     # clique seems to be reliably more than gen_foodstuffs. like, by big margin.
#         # could I take notes on how many foodstuffs each clique has, then draw from ones where it's smaller???
#         # pity it'd be filtering (not smallening), buuut hey fuck it
#         # but then, if person locks a bunch, wouldn't they want more foodstuffs?
#         # MOAR IDEAS
#             # it's really just foodstuffs and gen flavorings. would be no problem if there wasn't a range bn min/max, so that's an option.
#             # could also generate sm, md, lg stir fry sizes (pair flavorings & regulars?)
#             # or even make versions for each possible n foodstuffs
#             # split into multiple rows?
#             # idk man. suuure seems nice to just narrow the min-max
#             # OR discard more than is needed?
#                 # let's say it's 3-7, with 3 locked already
#                 # wait, tho. that means there's more than usual shorties, but it's still pulled at random from high scored
#             # OR discard random num?
#             # OR sample more from small, if num is small? tho that still omits goodies inside of big ones.
#             # radical - what if aim was more like, find clique that encompasses locked, then draw from that?????
#             # THIS^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
#                 # & in absence of locked?
#                     # I guess it's, like, do I pick big then whittle it down? or filter to small then pick one? which would lead to higher score?
#                     # I guess I'd want to pick one that has at least enough of everything, then discard excess
#                     # should think more about what alg leads to big pic big score

foodstuffs min/max 3 7
actual foodstuffs min/max 3 7
locked foodstuffs 0
actual foodstuffs 3
foodstuffs_so_far_set_len 3
n_additional_foodstuffs 0
n gen foodstuffs actual 3


In [62]:
%%time

def get_sort_key(length_tuple):
    return length_tuple[1]

def n_possible_edges(n_nodes):
    return int((n_nodes*(n_nodes-1)) / 2)

def get_first_name_in_set(sorted_tuples, food_set):
    for sorted_tuple in sorted_tuples:
        if sorted_tuple[0] in food_set:
            return sorted_tuple[0]

n_iterations = 100
keep_iterating = True

n_attempts_before_deciding = 10
n_attempts_before_giving_up = 5

scoring_method = 'tbd'
top_score = 0

for iteration in range(n_iterations):
    connection_attempt = 1
    while True:
        n_other_flavorings_actual = random.randrange(n_other_flavorings_min_actual, n_other_flavorings_max_actual+1)
        n_foodstuffs_actual = random.randrange(n_foodstuffs_min_actual, n_foodstuffs_max_actual+1)

        n_gen_other_flavorings_actual = n_other_flavorings_actual - len(locked_other_flavorings)
        n_gen_foodstuffs_actual = n_foodstuffs_actual - len(locked_foodstuffs)

        final_cliques = ok_cliques[(ok_cliques['ok_n_other_flavorings'] <= n_gen_other_flavorings_actual) & (ok_cliques['ok_n_foodstuffs'] <= n_gen_foodstuffs_actual)]

        if len(final_cliques) > 0:
            clique = final_cliques.sample(1, weights='ok_score_xtreme').iloc[0] # to skew sample toward top
        else:
            clique = final_cliques.sample(0)

        try:
            clique_ingredients = present.loc[clique['ok_list']]
        except:
            print('MAJOR PROBLEM! Likely cause: clique_data contains not-present ingredients.')
            clique_ingredients = stir_fry_flavor_data.loc[clique['ok_list']]

        salts_so_far_set = clique['ok_set'].intersection(salt_set).union(locked_salts_set)
        fat_oils_so_far_set = clique['ok_set'].intersection(fat_oil_set).union(locked_fat_oils_set)
        other_flavorings_so_far_set = clique['ok_set'].intersection(other_flavoring_set).union(locked_other_flavorings_set)
        foodstuffs_so_far_set = clique['ok_set'].intersection(foodstuff_set).union(locked_foodstuffs_set)
        ingredients_so_far_set = clique['ok_set'].union(locked_set)

        n_additional_salts = n_salts_actual - len(salts_so_far_set) # pretty sure there shouldn't be more salts so far than salts actual
        n_additional_fat_oils = n_fat_oils_actual - len(fat_oils_so_far_set) # pretty sure there shouldn't be more salts so far than salts actual
        n_additional_other_flavorings = max(n_other_flavorings_actual - len(other_flavorings_so_far_set), 0) # pretty sure there shouldn't be more salts so far than salts actual
        n_additional_foodstuffs = max(n_foodstuffs_actual - len(foodstuffs_so_far_set), 0) # pretty sure there shouldn't be more salts so far than salts actual

        additional_salts = the_rest_salts[~the_rest_salts['name'].isin(clique_ingredients['name'])].sample(n_additional_salts, weights='weak_score')
        additional_fat_oils = the_rest_fat_oils[~the_rest_fat_oils['name'].isin(clique_ingredients['name'])].sample(n_additional_fat_oils, weights='weak_score')
        additional_other_flavorings = the_rest_other_flavorings[~the_rest_other_flavorings['name'].isin(clique_ingredients['name'])].sample(n_additional_other_flavorings, weights='weak_score')
        additional_foodstuffs = the_rest_foodstuffs[~the_rest_foodstuffs['name'].isin(clique_ingredients['name'])].sample(n_additional_foodstuffs, weights='weak_score')

        ingredients_so_far = present[present['name'].isin(ingredients_so_far_set)]

        selected_ingredients = ingredients_so_far.append(additional_salts).append(additional_fat_oils).append(additional_other_flavorings).append(additional_foodstuffs)
        selected_names = selected_ingredients['name'].values.tolist()

        connections = []
        weighted_edges = []

        for i_1, name_1 in enumerate(selected_names[:-1]):
            for i_2, name_2 in enumerate(selected_names[i_1+1:], i_1+1):
                connection = selected_ingredients[name_1][name_2]
                if connection[0] != '_':
                    if connection[0] == 'c':
                        pairs_with_demerit = .8
                    elif connection[0] == 'd':
                        pairs_with_demerit = .6666
                    elif connection[0] == 'C':
                        pairs_with_demerit = .5333
                    elif connection[0] == 'D':
                        pairs_with_demerit = .4
                    else:
                        print('OH NO! BAD PAIRING VALUE.')

                    if connection[1] == '_':
                        strength_demerit = .2
                    elif connection[1] == 's':
                        strength_demerit = .15
                    elif connection[1] == 'S':
                        strength_demerit = .1
                    else:
                        print('OH NO! BAD STRENGTH VALUE.')


                    connection_weight = pairs_with_demerit + strength_demerit
                    weighted_edges.append((name_1, name_2, connection_weight))
                    connections.append((name_1, name_2, connection))

        selected_g = nx.Graph()
        selected_g.add_nodes_from(selected_names)
        selected_g.add_weighted_edges_from(weighted_edges)
        connected_components = list(nx.connected_components(selected_g))
        
        if scoring_method == 'tbd':
            if len(connected_components) == 1:
                print('Scoring method set to "connected"')
                scoring_method = 'connected'
                break
            elif connection_attempt >= n_attempts_before_deciding:
                print('Scoring method set to "disconnected"')
                scoring_method = 'disconnected'
                break
        elif scoring_method == 'connected':
            if len(connected_components) == 1:
                break
            elif connection_attempt >= n_attempts_before_giving_up:
                print('Giving up')
                keep_iterating = False
                break
        elif scoring_method == 'disconnected':
            break
            
        connection_attempt += 1
        
    if not keep_iterating:
        break # Just go with the best iteration so far (rather than slogging through disconnected graphs)
    
    score = 0

    if scoring_method == 'connected':
        # CONNECTED PAIRING BONUS ============================================================================================
        # ranges from roughly (0 to 1) * 3, tho could be a lil over or under that range
        average_shortest_path_length = nx.average_shortest_path_length(selected_g, weight='weight')
        average_shortest_path_score = 1 / average_shortest_path_length * 2 - 1.75
#         print('AVERAGE SHORTEST PATH SCORE', average_shortest_path_score)
        score += average_shortest_path_score * 3
    else:
        # DISCONNECTED PAIRING BONUS ============================================================================================
        # not really sure how this sranges. hopefully (0 - 1) * 3? Hard to test.
        largest_cc = max(connected_components, key=len)
        largest_subgraph = selected_g.subgraph(largest_cc) # .copy()?
        largest_subgraph_g = nx.Graph(largest_subgraph)
        
        average_shortest_path_length = nx.average_shortest_path_length(largest_subgraph_g, weight='weight')
        average_shortest_path_score = 1 / average_shortest_path_length * 2 - 1.75
#         print('DISCONNECTED AVERAGE SHORTEST PATH SCORE', average_shortest_path_score)
        score += average_shortest_path_score * 3
        
    # FLAVOR BALANCE BONUS =============================================================================================
    # ranges from roughly (0 to 1) * 1 (could be a lil over/under)
    # keeping this the same from ye olde stir fry gen (app version)
    n_sweet_lower = (selected_ingredients['sweet'] == 'y').sum()
    n_sweet_upper = (selected_ingredients['sweet'] == 'Y').sum()
    n_salty_lower = (selected_ingredients['salty'] == 'y').sum()
    n_salty_upper = (selected_ingredients['salty'] == 'Y').sum()
    n_sour_lower = (selected_ingredients['sour'] == 'y').sum()
    n_sour_upper = (selected_ingredients['sour'] == 'Y').sum()
    n_savory_lower = (selected_ingredients['savory'] == 'y').sum()
    n_savory_upper = (selected_ingredients['savory'] == 'Y').sum()
    n_bitter_lower = (selected_ingredients['bitter'] == 'y').sum()
    n_bitter_upper = (selected_ingredients['bitter'] == 'Y').sum()
    n_spicy_lower = (selected_ingredients['spicy'] == 'y').sum()
    n_spicy_upper = (selected_ingredients['spicy'] == 'Y').sum()

#     # each varies from roughly .5 to 1 (normalized to the average flavor score)
#     sweet_score = (n_sweet_lower/2 + n_sweet_upper)/6
#     salty_score = (n_salty_lower/2 + n_salty_upper)/4
#     sour_score = (n_sour_lower/2 + n_sour_upper)/2.5
#     savory_score = (n_savory_lower/2 + n_savory_upper)/3
#     bitter_score = (n_bitter_lower/2 + n_bitter_upper)/3
#     spicy_score = (n_spicy_lower/2 + n_spicy_upper)/3
# #     print(sweet_score, salty_score, sour_score, savory_score, bitter_score, spicy_score)

# #     damn but this seems arbitrary & kludgy
# #     flavor_balance_score = 1 / (1 + abs(1-sweet_score) + abs(1-salty_score) + abs(1-sour_score) + abs(1-savory_score) + abs(1-spicy_score))
    
#     flavor_balance_score = 1 / (1 + abs(1-sweet_score) + abs(1-salty_score) + abs(1-sour_score) + abs(1-savory_score) + abs(1-spicy_score)) * 8 - 1.9
#     print('FLAVOR BALANCE SCORE', flavor_balance_score)
#     score += flavor_balance_score

# SIMPLIFYING THIS, cause at end of the day it's up to user to balance shit    

    # TWO OBJECTIVES: reward having a little of every flavor, maaaybe punish for unbalanced flavors
    sweet_score = min(n_sweet_lower/2 + n_sweet_upper, 1)
    salty_score = min(n_salty_lower/2 + n_salty_upper, 1)
    sour_score = min(n_sour_lower/2 + n_sour_upper, 1)
    savory_score = min(n_savory_lower/2 + n_savory_upper, 1)
    bitter_score = min(n_bitter_lower/2 + n_bitter_upper, 1)
    spicy_score = min(n_spicy_lower/2 + n_spicy_upper, 1)

    flavor_balance_score = 0
    flavor_balance_score += sweet_score*3 # rly want something sweet in there
    flavor_balance_score += salty_score*.5 # can always use salt
    flavor_balance_score += sour_score*2 # like me some sour
    flavor_balance_score += savory_score*3 # LOVE me some savory
    flavor_balance_score += bitter_score # idk
    flavor_balance_score += spicy_score*2 # can be nice
    flavor_balance_score = flavor_balance_score * .2 - 1.3
    
#     print('FLAVOR BALANCE SCORE', flavor_balance_score)
    score += flavor_balance_score

    # FOOD GROUPS BONUS ==========================================================================================
#     n_fruit = (selected_ingredients['stir_fry_fruit'] == 'y').sum()
#     n_protein = (selected_ingredients['stir_fry_protein'] == 'y').sum()

#     # very very few, so realistically max .7
#     fruit_score = (n_fruit/2)**.5
#     print('FRUIT SCORE', fruit_score)

#     #     0, .7, occaaaasionally 1
#     protein_score = (n_protein/2)**.5
#     print('PROTEIN SCORE', protein_score)

    if 'y' in selected_ingredients['stir_fry_protein'].values:
        protein_score = .5
    else:
        protein_score = 0
    
    if 'y' in selected_ingredients['stir_fry_fruit'].values:
        fruit_score = .5
    else:
        fruit_score = 0

    # print(fruit_score, protein_score)
    food_group_score = protein_score + fruit_score
#     print('PROTEIN FRUIT', protein_score, fruit_score)
    score += food_group_score

    # # will bias toward larger stir_frys, slightly
    # # PROTEIN BONUS =========================================================================================
    # # ranges from roughly (0 to 1) * .5 (mostly balanced on its own)
    #     n_protein = (selected_ingredients['stir_fry_protein'] == 'y').sum()

    #     # /2 for steep diminishing returns (?)
    #     protein_score = (n_protein/2)**.5 * .75
    # #     print(protein_score)
    #     score += protein_score * .5

    if score > top_score:
        top_average_shortest_path_score = average_shortest_path_score
        top_flavor_balance_score = flavor_balance_score
        top_food_group_score = food_group_score
#         top_protein_score = protein_score
        top_score = score
#         top_lc_pairs = lower_category_pairs
#         top_ld_pairs = lower_direct_pairs
#         top_uc_pairs = upper_category_pairs
        top_ud_pairs = upper_direct_pairs
        top_connections = connections
        top_selected_ingredients = selected_ingredients
#         top_selected_g = selected_g
#     print('RAW PAIRING BONUS', top_average_shortest_path_score)
#     print('RAW FLAVOR BALANCE BONUS', top_flavor_balance_score)
#     print('RAW PROTEIN BONUS', top_protein_score)
#     print('SCORE', top_score)
#     print()
print()
print('RAW PAIRING BONUS', top_average_shortest_path_score)
print('RAW FLAVOR BALANCE BONUS', top_flavor_balance_score)
print('RAW FOOD GROUP BONUS', top_food_group_score)
print('SCORE', top_score)

cliqued_net = net.Network(notebook=True)

nodes = top_selected_ingredients['name'].tolist()

def get_color(row):
#     return 'grey'
    if row['stir_fry_veg'] == 'y':
        return 'green'
    elif row['stir_fry_fruit'] == 'y':
        return 'orange'
    elif row['stir_fry_protein'] == 'y':
        return 'brown'
    elif row['stir_fry_grain'] == 'y':
        return 'tan'
    else:
        return 'lightgrey'
nodes_color = top_selected_ingredients.apply(get_color, axis=1).tolist()

cliqued_net.add_nodes(
    nodes=nodes,
    color=nodes_color
)

for connection in top_connections:
    if connection[2][0] == 'c':
        cliqued_net.add_edge(connection[0], connection[1], physics=False, color='lightgrey')
    if connection[2][0] == 'd':
        cliqued_net.add_edge(connection[0], connection[1], physics=False, color='grey')
    if connection[2][0] == 'C':
        cliqued_net.add_edge(connection[0], connection[1], color='darkgrey')
    if connection[2][0] == 'D':
        cliqued_net.add_edge(connection[0], connection[1], color='black')
 
cliqued_net.show('cliqued_net.html')

NameError: name 'n_other_flavorings_min_actual' is not defined

In [439]:
foodstuffs_so_far_set

{'GARLIC',
 'MUSHROOMS, SHIITAKE—DRIED and FRESH',
 'ONIONS, PEARL',
 'PARSNIPS',
 'POTATOES, BLUE (or PURPLE)'}

In [367]:
len(ok_cliques)

842

In [492]:
len(final_cliques)

61

In [624]:
list(final_cliques.sample(1)['ok_list'])

[['CORN',
  'TOMATOES, TOMATO JUICE, TOMATO PASTE, and TOMATO SAUCE',
  'CILANTRO (aka CHINESE PARSLEY or FRESH CORIANDER LEAF)',
  'CHILES, SERRANO',
  'BEANS, BLACK (aka TURTLE BEANS)',
  'GARLIC, GREEN (aka BABY GARLIC or SPRING GARLIC)']]

In [306]:
ingredients_so_far_set

{'CABBAGE, RED',
 'CHEESE, CHEDDAR',
 'LENTILS, YELLOW',
 'MUSHROOMS, BUTTON (aka WHITE MUSHROOMS)',
 'MUSHROOMS, CREMINI (aka CRIMINI or ITALIAN BROWN MUSHROOMS)',
 'MUSHROOMS, OYSTER',
 'MUSHROOMS, SHIITAKE—DRIED and FRESH',
 'ONIONS, RED (see also ONIONS)',
 'PARSNIPS',
 'POTATOES, THICK-SKINNED (e.g., IDAHO, RUSSET)',
 'SQUASH, BUTTERNUT (see also SQUASH, WINTER)',
 'THYME'}

In [625]:
locked_set

{'BASIL', 'THYME', 'VINEGAR, APPLE CIDER (aka VINEGAR, CIDER)'}

In [470]:
locked_set

{'BASIL', 'THYME', 'VINEGAR, APPLE CIDER (aka VINEGAR, CIDER)'}

In [1066]:
%%time

# Am I going to have to adjust n_foodstuffs in cliques each iteration, if it's below n_foodstuffs of clique?
    # cause cliques are only filtered to include fewer foodstuffs than max
    # alternative: adjust n_gen_foodstuffs based on what's present in clique (easier)
# Ugh, gotta sample from the_rest except clique

top_score = 0
for try_i in range(100):   
    n_gen_other_flavorings = min(random.randrange(n_gen_other_flavorings_min, n_gen_other_flavorings_max+1), len(the_rest_other_flavorings))
    n_gen_foodstuffs = min(random.randrange(n_gen_foodstuffs_min, n_gen_foodstuffs_max+1), len(the_rest_foodstuffs))

    # edge case: no cliques left (meh)
    clique = ok_cliques[:100].sample(1, weights='ok_score_xtreme').iloc[0] # to skew sample toward top
#     cliques_xtreme = ok_cliques[:100].sample(10, weights='adjusted_score_xtreme')
#     cliques = ok_cliques[:100].sample(10, weights='adjusted_score')
#     print(cliques_xtreme['score'].mean())
#     print(clique['adjusted_score'].iloc[0])
#     print(clique['list'].iloc[0])
#     print(type(clique.iloc[0]))

    # hacky hack to account for cliques with more ingredients than actual gen (even though there's fewer than max gen)
    # might replace it with something better later
#     if clique['ok_n_other_flavorings'] > n_gen_other_flavorings:
#         n_gen_other_flavorings = clique['ok_n_other_flavorings']
#     if clique['']
    
    try:
        clique_ingredients = present.loc[clique['ok_list']]
    except:
        print('MAJOR PROBLEM! Likely cause: clique_data contains not-present ingredients.')
        clique_ingredients = stir_fry_flavor_data.loc[clique['ok_list']]
    
    # Kludgy hack to accomodate for possibility that clique has more of a thing than gen (though it should have less than gen_max)
    # says 'don't select more' in that case. there are worse things, I guess...
    n_additional_other_flavorings = max(n_gen_other_flavorings - clique['ok_n_other_flavorings'], 0)
    n_additional_foodstuffs = max(n_gen_foodstuffs - clique['ok_n_foodstuffs'], 0)
        
    # For most of these, tho, we should have already assured that n <= gen
    n_additional_salts = n_gen_salts - clique['ok_n_salts'] # take the number of salts we need and subtract the number of salts present in the clique we selected
    n_additional_fat_oils = n_gen_fat_oils - clique['ok_n_fat_oils']
    
    additional_salts = the_rest_salts[~the_rest_salts['name'].isin(clique_ingredients['name'])].sample(n_additional_salts, weights='weak_score')
    additional_fat_oils = the_rest_fat_oils[~the_rest_fat_oils['name'].isin(clique_ingredients['name'])].sample(n_additional_fat_oils, weights='weak_score')
    additional_other_flavorings = the_rest_other_flavorings[~the_rest_other_flavorings['name'].isin(clique_ingredients['name'])].sample(n_additional_other_flavorings, weights='weak_score')
    additional_foodstuffs = the_rest_foodstuffs[~the_rest_foodstuffs['name'].isin(clique_ingredients['name'])].sample(n_additional_foodstuffs, weights='weak_score')
    
#     selected_salts = locked_salts.append(the_rest_salts[~the_rest_salts['name'].isin(clique_ingredients['name'])].sample(n_gen_salts, weights='weak_score'))
#     selected_fat_oils = locked_fat_oils.append(the_rest_fat_oils.sample(n_gen_fat_oils, weights='weak_score'))
#     selected_other_flavorings = locked_other_flavorings.append(the_rest_other_flavorings.sample(n_gen_other_flavorings, weights='weak_score'))
#     selected_foodstuffs = locked_foodstuffs.append(the_rest_foodstuffs.sample(n_gen_foodstuffs, weights='weak_score'))
    selected_ingredients = clique_ingredients.append(additional_salts).append(additional_fat_oils).append(additional_other_flavorings).append(additional_foodstuffs)
    selected_names = selected_ingredients['name'].values.tolist()

# #     lower_category_pairs = []
# #     lower_direct_pairs = []
# #     upper_category_pairs = []
# #     upper_direct_pairs = []
#     connections = []

#     for i_1, name_1 in enumerate(selected_names[:-1]):
#         for i_2, name_2 in enumerate(selected_names[i_1+1:], i_1+1):
#             connection = selected_ingredients[name_1][name_2]
#             if connection[0] != '_':
# #                 print(connection)
#                 connections.append((name_1, name_2, connection))
# #             if connection[0] == 'c':
# #                 lower_category_pairs.append((name_1, name_2,))
# #             elif connection[0] == 'd':
# #                 lower_direct_pairs.append((name_1, name_2,))
# #             elif connection[0] == 'C':
# #                 upper_category_pairs.append((name_1, name_2,))
# #             elif connection[0] == 'D':
# #                 upper_direct_pairs.append((name_1, name_2,))

# #     lower_pairs = lower_category_pairs + lower_direct_pairs
# #     upper_pairs = upper_category_pairs + upper_direct_pairs
# #     all_pairs = lower_pairs + upper_pairs
  
# #     G = nx.Graph()
# #     G.add_nodes_from(selected_names)
# #     G.add_edges_from(lower_category_pairs, length=2)
# #     G.add_edges_from(lower_direct_pairs, length=1.5)
# #     G.add_edges_from(upper_category_pairs, length=1.2)
# #     G.add_edges_from(upper_direct_pairs, length=1)
#     score = 0

# # # PAIRING BONUS ============================================================================================
# # # ranges from roughly (0 to 1) * 3, tho could be a lil over or under that range
# #     average_shortest_path_length = nx.average_shortest_path_length(G, weight='length')
# #     average_shortest_path_score = 1 / average_shortest_path_length * 4 - 1
# # #     print(average_shortest_path_score)
# #     score += average_shortest_path_score * 3
  
# # PAIRING BONUS ===============================================================================================
#     connection_weighted_sum = 0
#     for connection in connections:
#         if connection[2][0] == 'c':
#             pairs_with_score = .2
#         elif connection[2][0] == 'd':
#             pairs_with_score = .4
#         elif connection[2][0] == 'C':
#             pairs_with_score = .6
#         elif connection[2][0] == 'D':
#             pairs_with_score = .8
#         else:
#             print('OH NO! BAD PAIRING VALUE.')
            
#         if connection[2][1] == '_':
#             strength_score = 0
#         elif connection[2][1] == 's':
#             strength_score = .1
#         elif connection[2][1] == 'S':
#             strength_score = .2
#         else:
#             print('OH NO! BAD STRENGTH VALUE.')
#         connection_weight = pairs_with_score + strength_score
#         connection_weighted_sum += connection_weight
#     connection_density = connection_weighted_sum / math.factorial(len(selected_names)-1)
#     print(connection_density)
            
# # COMMUNITY BONUS 



# # FLAVOR BALANCE BONUS =============================================================================================
# # ranges from roughly (0 to 1) * 1 (could be a lil over/under)
#     n_sweet_lower = (selected_ingredients['sweet'] == 'y').sum()
#     n_sweet_upper = (selected_ingredients['sweet'] == 'Y').sum()
#     n_salty_lower = (selected_ingredients['salty'] == 'y').sum()
#     n_salty_upper = (selected_ingredients['salty'] == 'Y').sum()
#     n_sour_lower = (selected_ingredients['sour'] == 'y').sum()
#     n_sour_upper = (selected_ingredients['sour'] == 'Y').sum()
#     n_savory_lower = (selected_ingredients['savory'] == 'y').sum()
#     n_savory_upper = (selected_ingredients['savory'] == 'Y').sum()
#     n_bitter_lower = (selected_ingredients['bitter'] == 'y').sum()
#     n_bitter_upper = (selected_ingredients['bitter'] == 'Y').sum()
#     n_spicy_lower = (selected_ingredients['spicy'] == 'y').sum()
#     n_spicy_upper = (selected_ingredients['spicy'] == 'Y').sum()

#     # each varies from roughly .5 to 1 (normalized to the average flavor score)
#     sweet_score = (n_sweet_lower/2 + n_sweet_upper)/6
#     salty_score = (n_salty_lower/2 + n_salty_upper)/4
#     sour_score = (n_sour_lower/2 + n_sour_upper)/2.5
#     savory_score = (n_savory_lower/2 + n_savory_upper)/3
#     bitter_score = (n_bitter_lower/2 + n_bitter_upper)/3
#     spicy_score = (n_spicy_lower/2 + n_spicy_upper)/3

#     flavor_balance_score = 5 / (1 + abs(1-sweet_score) + abs(1-salty_score) + abs(1-sour_score) + abs(1-savory_score) + abs(1-spicy_score)) - .9
# #     print(flavor_balance_score)
#     score += flavor_balance_score
# #     print()

# # will bias toward larger stir_frys, slightly
# # PROTEIN BONUS =========================================================================================
# # ranges from roughly (0 to 1) * .5 (mostly balanced on its own)
#     n_protein = (selected_ingredients['stir_fry_protein'] == 'y').sum()
    
#     # /2 for steep diminishing returns (?)
#     protein_score = (n_protein/2)**.5 * .75
# #     print(protein_score)
#     score += protein_score * .5
    
#     if score > top_score:
#         top_average_shortest_path_score = average_shortest_path_score
#         top_flavor_balance_score = flavor_balance_score
#         top_protein_score = protein_score
#         top_score = score
#         top_lc_pairs = lower_category_pairs
#         top_ld_pairs = lower_direct_pairs
#         top_uc_pairs = upper_category_pairs
#         top_ud_pairs = upper_direct_pairs
#         top_selected_ingredients = selected_ingredients
#         top_G = G
# print('RAW PAIRING BONUS', top_average_shortest_path_score)
# print('RAW FLAVOR BALANCE BONUS', top_flavor_balance_score)
# print('RAW PROTEIN BONUS', top_protein_score)
# print('SCORE', top_score)

# top_net = net.Network(notebook=True)

# nodes = top_selected_ingredients['name'].tolist()

# def get_color(row):
# #     return 'grey'
#     if row['stir_fry_veg'] == 'y':
#         return 'green'
#     elif row['stir_fry_fruit'] == 'y':
#         return 'orange'
#     elif row['stir_fry_protein'] == 'y':
#         return 'brown'
#     elif row['stir_fry_grain'] == 'y':
#         return 'tan'
#     else:
#         return 'lightgrey'
# nodes_color = top_selected_ingredients.apply(get_color, axis=1).tolist()

# top_net.add_nodes(
#     nodes=nodes,
#     color=nodes_color
# )

# for pair in top_lc_pairs:
#     top_net.add_edge(pair[0], pair[1], physics=False, color='lightgrey')
# for pair in top_ld_pairs:
#     top_net.add_edge(pair[0], pair[1], physics=False, color='grey')
# for pair in top_uc_pairs:
#     top_net.add_edge(pair[0], pair[1], color='darkgrey')
# for pair in top_ud_pairs:
#     top_net.add_edge(pair[0], pair[1], color='black')

# top_net.show('top_net.html')

CPU times: user 2.11 s, sys: 0 ns, total: 2.11 s
Wall time: 2.12 s


In [572]:
thing_net = net.Network(notebook=True)
        
nodes = []
edges = []

for i, clique in enumerate(found_cliques):
    if len(clique) > 6:
        nodes.append(str(i) + '_node')
        for node in clique:
            node_name = str(i) + '_' + node
            nodes.append(node_name)
            edges.append((str(i) + '_node', node_name))
        
# print(nodes)
thing_net.add_nodes(nodes=nodes)

for edge in edges:
    thing_net.add_edge(edge[0], edge[1])
    
# def get_color(row):
# #     return 'grey'
#     if row['stir_fry_veg'] == 'y':
#         return 'green'
#     elif row['stir_fry_fruit'] == 'y':
#         return 'orange'
#     elif row['stir_fry_protein'] == 'y':
#         return 'brown'
#     elif row['stir_fry_grain'] == 'y':
#         return 'tan'
#     else:
#         return 'lightgrey'
# nodes_color = top_selected_ingredients.apply(get_color, axis=1).tolist()

# thing_net.add_nodes(
#     nodes=nodes + list(range(100))
# #     color=nodes_color
# )

    

# for k, v in part.items():
#     thing_net.add_edge(k, v)

# for pair in top_lc_pairs:
#     top_net.add_edge(pair[0], pair[1], physics=False, color='lightgrey')
# for pair in top_ld_pairs:
#     top_net.add_edge(pair[0], pair[1], physics=False, color='grey')
# for pair in top_uc_pairs:
#     top_net.add_edge(pair[0], pair[1], color='darkgrey')
# for pair in top_ud_pairs:
#     top_net.add_edge(pair[0], pair[1], color='black')

thing_net.show('thing_net.html')

NameError: name 'found_cliques' is not defined

In [153]:
%%time
containing_cliques = cliques_containing_node(G, nodes=['WALNUTS', 'YAMS'])

CPU times: user 14.4 ms, sys: 0 ns, total: 14.4 ms
Wall time: 14.3 ms


In [175]:
%%time
lengths = nx.all_pairs_dijkstra_path_length(G)

CPU times: user 441 ms, sys: 3.88 ms, total: 444 ms
Wall time: 444 ms


## 7.2. Random control

In [95]:
n_subgraphs = 2
while n_subgraphs > 1: # keep shuffling until you get a well connected graph

    n_gen_greens = random.randrange(n_gen_greens_min, n_gen_greens_max+1)

    n_gen_extras = random.randrange(n_gen_extras_min, n_gen_extras_max+1)

    n_gen_dressing_oils = max(1-len(locked_dressing_oils), 0)
    n_gen_dressing_vinegars = max(1-len(locked_dressing_vinegars), 0)
    n_gen_dressing_salts = max(1-len(locked_dressing_salts), 0)
    n_gen_dressing_peppers = max(1-len(locked_dressing_oils), 0)

    selected_greens = locked_greens.append(the_rest_greens.sample(n_gen_greens))
    selected_extras = locked_extras.append(the_rest_extras.sample(n_gen_extras))
    selected_dressing_oils = locked_dressing_oils.append(the_rest_dressing_oils.sample(n_gen_dressing_oils))
    selected_dressing_vinegars = locked_dressing_vinegars.append(the_rest_dressing_vinegars.sample(n_gen_dressing_vinegars))
    selected_dressing_salts = locked_dressing_salts.append(the_rest_dressing_salts.sample(n_gen_dressing_salts))
    selected_dressing_peppers = locked_dressing_peppers.append(the_rest_dressing_peppers.sample(n_gen_dressing_peppers))
    selected_ingredients = selected_greens.append(selected_extras).append(selected_dressing_oils).append(selected_dressing_vinegars).append(selected_dressing_salts).append(selected_dressing_peppers)
    selected_names = selected_ingredients['name'].values.tolist()

    lower_category_pairs = []
    lower_direct_pairs = []
    upper_category_pairs = []
    upper_direct_pairs = []
    lower_clashing_pairs = []
    upper_clashing_pairs = []

    # finicky but pretty fast
    for i, col_name in enumerate(selected_names):
        for j, row_name in enumerate(selected_names[i+1:]):
            connection = selected_ingredients[col_name].tolist()[i+1+j] # this is what is finicky
            if connection == 'c':
                lower_category_pairs.append((col_name, row_name,))
            elif connection == 'd':
                lower_direct_pairs.append((col_name, row_name,))
            elif connection == 'C':
                upper_category_pairs.append((col_name, row_name,))
            elif connection == 'D':
                upper_direct_pairs.append((col_name, row_name,))
            elif connection == 'n':
                lower_clashing_pairs.append((col_name, row_name,))
            elif connection == 'N':
                upper_clashing_pairs.append((col_name, row_name,))
    lower_pairs = lower_category_pairs + lower_direct_pairs
    upper_pairs = upper_category_pairs + upper_direct_pairs
    all_pairs = lower_pairs + upper_pairs
    all_clashing_pairs = lower_clashing_pairs + upper_clashing_pairs

    G = nx.Graph()
    G.add_nodes_from(selected_names)
    G.add_edges_from(lower_category_pairs, length=2)
    G.add_edges_from(lower_direct_pairs, length=1.5)
    G.add_edges_from(upper_category_pairs, length=1.2)
    G.add_edges_from(upper_direct_pairs, length=1)
    n_subgraphs = len(list(nx.connected_component_subgraphs(G)))

random_net = net.Network(notebook=True)

nodes = selected_ingredients['name'].tolist()

def get_color(row):
    if row['stir_fry_green'] == 'y':
        return 'lightgreen'
    elif row['stir_fry_extra'] == 'y':
        if row['veg'] == 'y':
            return 'green'
        elif row['fruit'] == 'y':
            return 'orange'
        elif row['protein_nut_seed'] == 'y':
            return 'brown'
        else:
            return 'lightblue'
    elif row['stir_fry_dressing'] == 'y':
        return 'lightgrey'
nodes_color = selected_ingredients.apply(get_color, axis=1).tolist()

random_net.add_nodes(
    nodes=nodes,
    color=nodes_color
)

for pair in lower_category_pairs:
    random_net.add_edge(pair[0], pair[1], physics=False, color='lightgrey')
for pair in lower_direct_pairs:
    random_net.add_edge(pair[0], pair[1], physics=False, color='grey')
for pair in upper_category_pairs:
    random_net.add_edge(pair[0], pair[1], color='darkgrey')
for pair in upper_direct_pairs:
    random_net.add_edge(pair[0], pair[1], color='black')

vegan = selected_ingredients['not_vegan'].sum() == ''
gluten_free = selected_ingredients['gluten'].sum() == ''

random_net.show('random_net.html')

# 8. Recording recipes

## 8.1. Recording with flavor tool generator

In [ ]:
# try:
#     stir_fry_recipe_data = pd.read_pickle(os.path.join(root_path, 'DATA/stir_fry_recipe_data_latest.pickle'))
#     print('stir_fry RECIPE DATA IMPORT SUCCESSFUL')
# except:
#     stir_fry_recipe_data = pd.DataFrame({
#         'vegan': [],
#         'gluten_free': [],
#         'basic': [],
#         'best_of': [],
#         'score': [],
#         'pairing_density_bonus': [],
#         'pair_strength_bonus': [],
#         'clash_penalty': [],
#         'flavor_balance_bonus': [],
#         'texture_balance_bonus': [],
#         'food_group_balance_bonus': [],
#         'lc_pairs': [],
#         'ld_pairs': [],
#         'uc_pairs': [],
#         'ud_pairs':[],
#         'clashing_pairs': [],
#         'ingredient_names': [],
#         'leafy_green_names': [],
#         'extra_names': [],
#         'dressing_names': [],
#     })
#     print('IMPORT FAILED, CREATING NEW stir_fry RECIPE DATAFRAME')

In [ ]:
# stir_fry_greens = stir_fry_data[stir_fry_data['stir_fry_green'] == 'y']

# stir_fry_extras = stir_fry_data[stir_fry_data['stir_fry_extra'] == 'y']
# stir_fry_extra_veg = stir_fry_data[(stir_fry_data['veg'] == 'y') & (stir_fry_data['stir_fry_extra'] == 'y')]
# stir_fry_extra_fruits = stir_fry_data[(stir_fry_data['fruit'] == 'y') & (stir_fry_data['stir_fry_extra'] == 'y')]
# stir_fry_extra_nuts = stir_fry_data[(stir_fry_data['protein_seed'] == 'y') & (stir_fry_data['stir_fry_extra'] == 'y')]
# stir_fry_extra_seeds = stir_fry_data[(stir_fry_data['protein_nut'] == 'y') & (stir_fry_data['stir_fry_extra'] == 'y')]
# stir_fry_extra_tomatoes = stir_fry_data[stir_fry_data['stir_fry_extra_tomato'] == 'y']
# stir_fry_extra_olives = stir_fry_data[stir_fry_data['stir_fry_extra_olive'] == 'y']
# stir_fry_extra_cheeses = stir_fry_data[stir_fry_data['stir_fry_extra_cheese'] == 'y']
# stir_fry_extra_eggs = stir_fry_data[stir_fry_data['stir_fry_extra_egg'] == 'y']
# stir_fry_extra_croutons = stir_fry_data[stir_fry_data['stir_fry_extra_crouton'] == 'y']

# stir_fry_dressing_oils = stir_fry_data[stir_fry_data['stir_fry_dressing_oil'] == 'y']
# stir_fry_dressing_vinegars = stir_fry_data[stir_fry_data['stir_fry_dressing_vinegar'] == 'y']
# stir_fry_dressing_salts = stir_fry_data[stir_fry_data['stir_fry_dressing_salt'] == 'y']
# stir_fry_dressing_peppers = stir_fry_data[stir_fry_data['stir_fry_dressing_pepper'] == 'y']
# stir_fry_dressing_garlics = stir_fry_data[stir_fry_data['stir_fry_dressing_garlic'] == 'y']

In [ ]:
# # !pip install networkx

# import networkx as nx

In [ ]:
# best_of = 300
# for recipe_count in range(500):
#     top_score = 0
#     for i in range(best_of):
#         n_subgraphs = 2
#         while n_subgraphs > 1: # keep shuffling until you get a well connected graph
#             n_greens = random.randrange(2, 4)
#             n_extras = random.randrange(2, 6)
#             n_dressing_oils = 1
#             n_dressing_vinegars = 1
#             n_dressing_salts = 1
#             n_dressing_peppers = 1
#             # n_dressing_garlics = random.randrange(0, 2) # maybe make presence dependent on the rest. or, just leave out for now.

#             selected_greens = stir_fry_greens.sample(n_greens)
#             selected_extras = stir_fry_extras.sample(n_extras)
#             selected_dressing_oils = stir_fry_dressing_oils.sample(n_dressing_oils)
#             selected_dressing_vinegars = stir_fry_dressing_vinegars.sample(n_dressing_vinegars)
#             selected_dressing_salts = stir_fry_dressing_salts.sample(n_dressing_salts)
#             selected_dressing_peppers = stir_fry_dressing_peppers.sample(n_dressing_peppers)
#             selected_ingredients = selected_greens.append(selected_extras).append(selected_dressing_oils).append(selected_dressing_vinegars).append(selected_dressing_salts).append(selected_dressing_peppers)

#             lower_category_pairs = []
#             lower_direct_pairs = []
#             upper_category_pairs = []
#             upper_direct_pairs = []
#             ingredients_list = selected_ingredients['name'].values.tolist()
#             already_checked = []
#             for ingredient_name in ingredients_list:
#                 for lc_name in pairing_data['lower_category_names'][pairing_data['name'] == ingredient_name].iloc[0]:
#                     if lc_name in ingredients_list and not lc_name in already_checked:
#                         lower_category_pairs.append([ingredient_name, lc_name])
#                 for ld_name in pairing_data['lower_direct_names'][pairing_data['name'] == ingredient_name].iloc[0]:
#                     if ld_name in ingredients_list and not ld_name in already_checked:
#                         lower_direct_pairs.append([ingredient_name, ld_name])
#                 for uc_name in pairing_data['upper_category_names'][pairing_data['name'] == ingredient_name].iloc[0]:
#                     if uc_name in ingredients_list and not uc_name in already_checked:
#                         upper_category_pairs.append([ingredient_name, uc_name])
#                 for ud_name in pairing_data['upper_direct_names'][pairing_data['name'] == ingredient_name].iloc[0]:
#                     if ud_name in ingredients_list and not ud_name in already_checked:
#                         upper_direct_pairs.append([ingredient_name, ud_name])
#                 already_checked.append(ingredient_name)

#             lower_pairs = lower_category_pairs + lower_direct_pairs
#             upper_pairs = upper_category_pairs + upper_direct_pairs
#             all_pairs = lower_pairs + upper_pairs

#     #         all_pairs_sp = [tuple(sorted(pair)) for pair in all_pairs]
#     #         print(len(all_pairs_sp), len(list(set(all_pairs_sp))))

#     #         print('INGREDIENTS', ingredients_list)
#     #         print()
#     #         print('LC PAIRS', lower_category_pairs)
#     #         print()
#     #         print()

#             G = nx.Graph()
#             G.add_nodes_from(selected_ingredients['name'].values.tolist())
#             G.add_edges_from(all_pairs)
#             n_subgraphs = len(list(nx.connected_component_subgraphs(G)))

#         score = 0

#     # SIMPLER ALTERNATIVE: bonus for proportion of actual pairs over possible pairs? could then combine with pair strength bonus?
#     # PAIRING DENSITY BONUS ============================================================================================
#         # ranges from roughly (.1 to 1) * 4
#         average_shortest_path_length = nx.average_shortest_path_length(G)
#         average_shortest_path_score = 2.5 / average_shortest_path_length - 1.1
#     #     print(average_shortest_path_score)
#         score += average_shortest_path_score * 4    

#     # # UPPER PAIRING BONUS ==============================================================================================
#     #     # ranges from roughly (.25 to 1) * 3
#     #     upper_proportion_score = len(upper_pairs) / len(all_pairs) * 2.5 # messed with this, not sure if it still works
#     # #     print(upper_proportion_score)
#     # #     print(len(upper_pairs), len(lower_pairs), len(all_pairs))
#     # #     print()
#     #     score += upper_proportion_score * 3

#     # PAIR STRENGTH BONUS ==============================================================================================
#         # ranges from roughly (0 to 1) * 3

#         # I'm thinking of 'lower category' as default, and awarding points for steps up from that
#         ld_bonus = 1*len(lower_direct_pairs)
#         uc_bonus = 2*len(upper_category_pairs)
#         ud_bonus = 5*len(upper_direct_pairs)
#         pair_strength_score = .6*(ld_bonus + uc_bonus + ud_bonus)/len(all_pairs) # otherwise would tend toward large stir_frys
#     #     print('LC', len(lower_category_pairs))
#     #     print('LD', len(lower_direct_pairs))
#     #     print('UC', len(upper_category_pairs))
#     #     print('UD', len(upper_direct_pairs))
#     #     print('SCORE', pair_strength_score)
#     #     print()
#         score += pair_strength_score * 3

#     # important but easy to avoid, so not weighted too heavily
#     # CLASH PENALTY ====================================================================================================
#         # ranges from roughly (0 to 1) * -1.5
#         all_clashing_pairs = []
#         selected_ingredients_list = selected_ingredients['name'].values.tolist()
#         for name in selected_ingredients_list:
#             names_that_clash_with_name = clashes_with_data['all_clashes_with_names'][clashes_with_data['name'] == name].iloc[0]
#             all_clashing_names = set(selected_ingredients_list).intersection(set(names_that_clash_with_name)) # selected names that clash with this selected name
#             all_clashing_pairs += [tuple(sorted([name, all_clashing_name])) for all_clashing_name in all_clashing_names]

#         all_clashing_pairs = list(set(all_clashing_pairs))
#         all_clashing_pairs_score = len(list(all_clashing_pairs)) / 4
#         score += len(all_clashing_pairs) * -1.5

#     # # FRUIT BONUS ======================================================================================================
#     #     # ranges from roughly (0 to 3) * .1
#     #     n_fruit = len(selected_ingredients[selected_ingredients['fruit'] == 'y'])
#     #     score += n_fruit * .1

#     # # NUT SEED BONUS ===================================================================================================    
#     #     # ranges from roughly (0 to 2) * .15
#     #     n_nut_seed = len(selected_ingredients[selected_ingredients['protein_nut_seed'] == 'y'])
#     #     score += n_nut_seed * .15

#     # FLAVOR BALANCE BONUS =============================================================================================
#         # ranges from roughly (0 to 1) * 1
#         n_sweet_lower = len(selected_ingredients[selected_ingredients['sweet'] == 'y'])
#         n_sweet_upper = len(selected_ingredients[selected_ingredients['sweet'] == 'Y'])
#         n_salty_lower = len(selected_ingredients[selected_ingredients['salty'] == 'y'])
#         n_salty_upper = len(selected_ingredients[selected_ingredients['salty'] == 'Y'])
#         n_sour_lower = len(selected_ingredients[selected_ingredients['sour'] == 'y'])
#         n_sour_upper = len(selected_ingredients[selected_ingredients['sour'] == 'Y'])
#         n_savory_lower = len(selected_ingredients[selected_ingredients['savory'] == 'y'])
#         n_savory_upper = len(selected_ingredients[selected_ingredients['savory'] == 'Y'])
#         n_bitter_lower = len(selected_ingredients[selected_ingredients['bitter'] == 'y'])
#         n_bitter_upper = len(selected_ingredients[selected_ingredients['bitter'] == 'Y'])
#         n_spicy_lower = len(selected_ingredients[selected_ingredients['spicy'] == 'y'])
#         n_spicy_upper = len(selected_ingredients[selected_ingredients['spicy'] == 'Y'])

#         # each varies from roughly .5 to 1
#         sweet_score = (n_sweet_lower/2 + n_sweet_upper)/5
#         salty_score = (n_salty_lower/2 + n_salty_upper)*2/5
#         sour_score = (n_sour_lower/2 + n_sour_upper)*2/5
#         savory_score = (n_savory_lower/2 + n_savory_upper)*3/5
#         bitter_score = (n_bitter_lower/2 + n_bitter_upper)*3/5
#         spicy_score = (n_spicy_lower/2 + n_spicy_upper)*2/5

#         flavor_balance_score = 5 / (1 + abs(1-sweet_score) + abs(1-salty_score) + abs(1-sour_score) + abs(1-savory_score) + abs(1-spicy_score)) - 1.25
#     #     print(flavor_balance_score)

#         score += flavor_balance_score
#     #     print(sweet_score, salty_score, sour_score, savory_score, bitter_score, spicy_score)
#     #     print()

#     #     print(n_sweet_lower, n_sweet_upper)
#     #     print(n_salty_lower, n_salty_upper)
#     #     print(n_sour_lower, n_sour_upper)
#     #     print(n_savory_lower, n_savory_upper)
#     #     print(n_bitter_lower, n_bitter_upper)
#     #     print(n_spicy_lower, n_spicy_upper)
#     #     print()

#     # TEXTURE BALANCE BONUS ============================================================================================
#         # ranges from roughly (0 to 1) * .75
#         n_crunchy_lower = len(selected_ingredients[selected_ingredients['stir_fry_crunchy'] == 'y'])
#         n_crunchy_upper = len(selected_ingredients[selected_ingredients['stir_fry_crunchy'] == 'Y'])
#         n_chewy_lower = len(selected_ingredients[selected_ingredients['stir_fry_chewy'] == 'y'])
#         n_chewy_upper = len(selected_ingredients[selected_ingredients['stir_fry_chewy'] == 'Y'])
#         n_juicy_lower = len(selected_ingredients[selected_ingredients['stir_fry_juicy'] == 'y'])
#         n_juicy_upper = len(selected_ingredients[selected_ingredients['stir_fry_juicy'] == 'Y'])

#         # each ranges from roughly 0 to 1
#         crunchy_score = (n_crunchy_lower/2 + n_crunchy_upper)/3
#         chewy_score = (n_chewy_lower/2 + n_chewy_upper)
#         juicy_score = (n_juicy_lower/2 + n_juicy_upper)/3
#     #     print(crunchy_score, chewy_score, juicy_score)

#         texture_balance_score = 4 / (1 + abs(1-crunchy_score) + abs(1-chewy_score) + abs(1-juicy_score)) - 1
#     #     print(texture_balance_score)
#     #     print()

#         score += texture_balance_score * .75

#     # seems like it's hard to balance food groups on top of everything else. pity the scores aren't more independent
#     # FOOD GROUP BALANCE BONUS =========================================================================================
#         # ranges from roughly (25 to 1) * 2
#         n_fruit = len(selected_ingredients[selected_ingredients['fruit'] == 'y'])
#         n_veg = len(selected_ingredients[selected_ingredients['veg'] == 'y'])
#         n_protein = len(selected_ingredients[selected_ingredients['protein'] == 'y'])

#         # each varies from roughly 0 to 1 (sometimes a little over)
#         fruit_score = n_fruit / 3
#         veg_score = n_veg / 5
#         protein_score = n_protein / 3
#     #     print(fruit_score, veg_score, protein_score)

#         food_group_balance_score = 3 / (1 + abs(1-fruit_score) + abs(1-veg_score) + abs(1-protein_score)) - .75
#     #     print(food_group_balance_score)
#     #     print()

#         score += food_group_balance_score * 2

#         if score > top_score:
#             top_score = score
#             top_food_group_balance_score = food_group_balance_score
#             top_average_shortest_path_score = average_shortest_path_score
#             top_flavor_balance_score = flavor_balance_score
#             top_texture_balance_score = texture_balance_score
#             top_all_clashing_pairs_score = all_clashing_pairs_score
#             top_pair_strength_score = pair_strength_score
#     #         top_upper_pairs = upper_pairs
#     #         top_lower_pairs = lower_pairs
#             top_lc_pairs = lower_category_pairs
#             top_ld_pairs = lower_direct_pairs
#             top_uc_pairs = upper_category_pairs
#             top_ud_pairs = upper_direct_pairs
#             top_selected_ingredients = selected_ingredients
#             top_average_shortest_path_length = average_shortest_path_length
#             top_all_clashing_pairs = all_clashing_pairs
#     #         top_upper_proportion = len(upper_pairs) / (len(upper_pairs) + len(lower_pairs))
#     # print('TOP AVG SHORTEST PATH LENGTH', top_average_shortest_path_length)
#     # print('TOP UPPER PROPORTION', top_upper_proportion)
# #     print('TOP AVERAGE SHORTEST PATH SCORE', top_average_shortest_path_score * 4)
# #     print('TOP PAIR STRENGTH SCORE', top_pair_strength_score * 3)
# #     print('TOP ALL CLASHING PAIRS SCORE', top_all_clashing_pairs_score * -1.5)
# #     print('TOP FLAVOR BALANCE SCORE', top_flavor_balance_score)
# #     print('TOP TEXTURE BALANCE SCORE', top_texture_balance_score * .75)
# #     print('TOP FOOD GROUP BALANCE SCORE', top_food_group_balance_score * 2)
# #     print('TOP_SCORE', top_score)  
    
#     vegan = top_selected_ingredients['not_vegan'].sum() == ''
#     gluten_free = top_selected_ingredients['gluten'].sum() == ''
    
#     recipe_greens = top_selected_ingredients[top_selected_ingredients['stir_fry_green'] == 'y']
#     recipe_extras = top_selected_ingredients[top_selected_ingredients['stir_fry_extra'] == 'y']
#     recipe_dressing_oils = top_selected_ingredients[top_selected_ingredients['stir_fry_dressing_oil'] == 'y']
#     recipe_dressing_vinegars = top_selected_ingredients[top_selected_ingredients['stir_fry_dressing_vinegar'] == 'y']
#     recipe_dressing_salts = top_selected_ingredients[top_selected_ingredients['stir_fry_dressing_salt'] == 'y']
#     recipe_dressing_peppers = top_selected_ingredients[top_selected_ingredients['stir_fry_dressing_pepper'] == 'y']
#     recipe_dressing_garlics = top_selected_ingredients[top_selected_ingredients['stir_fry_dressing_garlic'] == 'y']
#     # could just select 'stir_fry_dressing', but this includes garlics
#     recipe_dressing = recipe_dressing_oils.append(recipe_dressing_vinegars).append(recipe_dressing_salts).append(recipe_dressing_peppers)

#     new_recipe = pd.DataFrame({
#         'vegan': [vegan],
#         'gluten_free': [gluten_free],
#         'basic': [stir_fry_data_is_basic],
#         'best_of': [best_of],
#         'score': [top_score],
#         'pairing_density_bonus': [top_average_shortest_path_score * 4],
#         'pair_strength_bonus': [top_pair_strength_score * 3],
#         'clash_penalty': [top_all_clashing_pairs_score * 4],
#         'flavor_balance_bonus': [top_flavor_balance_score],
#         'texture_balance_bonus': [top_texture_balance_score * 75],
#         'food_group_balance_bonus': [top_food_group_balance_score * 2],
#         'lc_pairs': [top_lc_pairs],
#         'ld_pairs': [top_ld_pairs],
#         'uc_pairs': [top_uc_pairs],
#         'ud_pairs':[top_ud_pairs],
#         'clashing_pairs': [top_all_clashing_pairs],
#         'ingredient_names': [top_selected_ingredients['name'].values.tolist()],
#         'leafy_green_names': [recipe_greens['name'].values.tolist()],
#         'extra_names': [recipe_extras['name'].values.tolist()],
#         'dressing_names': [recipe_dressing['name'].values.tolist()],
#     })
#     stir_fry_recipe_data = stir_fry_recipe_data.append(new_recipe, sort=False)
#     print('stir_fry RECIPE RECORDED. SCORE:', top_score)

In [ ]:
# from datetime import date
# today = date.today()
# date_string = f'{str(today.year)}_{str(today.month)}_{str(today.day)}'

# stir_fry_recipe_data.to_pickle(os.path.join(root_path, 'DATA/stir_fry_recipe_data_latest.pickle'))
# stir_fry_recipe_data.to_pickle(os.path.join(root_path, f'DATA/stir_fry_recipe_data_{date_string}.pickle'))

## 8.2. Displaying flavor tool generator records

In [ ]:
# # !pip install pyvis

# from pyvis import network as net

In [ ]:
# recipe = stir_fry_recipe_data[(stir_fry_recipe_data['basic'] == True) & (stir_fry_recipe_data['vegan'] == True) & stir_fry_recipe_data['gluten_free'] == True].sample(1).iloc[0]
# recipe_net = net.Network(notebook=True)

# nodes = recipe['ingredient_names']

# nodes_color = []
# for name in recipe['ingredient_names']:
#     ingredient = stir_fry_data[stir_fry_data['name'] == name].iloc[0]
#     if ingredient['stir_fry_green'] == 'y':
#         nodes_color.append('lightgreen')
#     elif ingredient['stir_fry_extra'] == 'y':
#         if ingredient['veg'] == 'y':
#             nodes_color.append('green')
#         elif ingredient['fruit'] == 'y':
#             nodes_color.append('orange')
#         elif ingredient['protein_nut_seed'] == 'y':
#             nodes_color.append('brown')
#         else:
#             nodes_color.append('lightblue')
#     elif ingredient['stir_fry_dressing'] == 'y':
#         nodes_color.append('lightgrey')  
        
# recipe_net.add_nodes(
#     nodes=nodes,
#     color=nodes_color
# )

# for pair in recipe['lc_pairs']:
#     recipe_net.add_edge(pair[0], pair[1], physics=False, color='lightgrey')

# for pair in recipe['ld_pairs']:
#     recipe_net.add_edge(pair[0], pair[1], physics=False, color='grey')
    
# for pair in recipe['uc_pairs']:
#     recipe_net.add_edge(pair[0], pair[1], color='darkgrey')
    
# for pair in recipe['ud_pairs']:
#     recipe_net.add_edge(pair[0], pair[1], color='black')

# if not recipe['vegan']:
#     print('NOT VEGAN')
# if not recipe['gluten_free']:
#     print('CONTAINS GLUTEN')
# print('SCORE:', recipe['score'])
# recipe_net.show('recipe_net.html')

In [62]:
%%time

n_iterations = 100
keep_iterating = True

n_attempts_before_deciding = 10
n_attempts_before_giving_up = 5

scoring_method = 'tbd'
top_score = 0

while True:
    selected_ingredients = stir_fry_flavor_data.sample(6)
    selected_names = selected_ingredients['name'].values.tolist()

    connections = []
    weighted_edges = []

    for i_1, name_1 in enumerate(selected_names[:-1]):
        for i_2, name_2 in enumerate(selected_names[i_1+1:], i_1+1):
            connection = selected_ingredients[name_1][name_2]
            if connection[0] != '_':
                if connection[0] == 'c':
                    pairs_with_demerit = .8
                elif connection[0] == 'd':
                    pairs_with_demerit = .6666
                elif connection[0] == 'C':
                    pairs_with_demerit = .5333
                elif connection[0] == 'D':
                    pairs_with_demerit = .4
                else:
                    print('OH NO! BAD PAIRING VALUE.')

                if connection[1] == '_':
                    strength_demerit = .2
                elif connection[1] == 's':
                    strength_demerit = .15
                elif connection[1] == 'S':
                    strength_demerit = .1
                else:
                    print('OH NO! BAD STRENGTH VALUE.')


                connection_weight = pairs_with_demerit + strength_demerit
                weighted_edges.append((name_1, name_2, connection_weight))
                connections.append((name_1, name_2, connection))

    selected_g = nx.Graph()
    selected_g.add_nodes_from(selected_names)
    selected_g.add_weighted_edges_from(weighted_edges)
    connected_components = list(nx.connected_components(selected_g))
    print(selected_g.degree())

    if scoring_method == 'tbd':
        if len(connected_components) == 1:
            print('Scoring method set to "connected"')
            scoring_method = 'connected'
            break
        elif connection_attempt >= n_attempts_before_deciding:
            print('Scoring method set to "disconnected"')
            scoring_method = 'disconnected'
            break
    elif scoring_method == 'connected':
        if len(connected_components) == 1:
            break
        elif connection_attempt >= n_attempts_before_giving_up:
            print('Giving up')
            keep_iterating = False
            break
    elif scoring_method == 'disconnected':
        break

    connection_attempt += 1

score = 0

if scoring_method == 'connected':
    # CONNECTED PAIRING BONUS ============================================================================================
    # ranges from roughly (0 to 1) * 3, tho could be a lil over or under that range
    average_shortest_path_length = nx.average_shortest_path_length(selected_g, weight='weight')
    average_shortest_path_score = 1 / average_shortest_path_length * 2 - 1.75
#         print('AVERAGE SHORTEST PATH SCORE', average_shortest_path_score)
    score += average_shortest_path_score * 3
else:
    # DISCONNECTED PAIRING BONUS ============================================================================================
    # not really sure how this sranges. hopefully (0 - 1) * 3? Hard to test.
    largest_cc = max(connected_components, key=len)
    largest_subgraph = selected_g.subgraph(largest_cc) # .copy()?
    largest_subgraph_g = nx.Graph(largest_subgraph)

    average_shortest_path_length = nx.average_shortest_path_length(largest_subgraph_g, weight='weight')
    average_shortest_path_score = 1 / average_shortest_path_length * 2 - 1.75
#         print('DISCONNECTED AVERAGE SHORTEST PATH SCORE', average_shortest_path_score)
    score += average_shortest_path_score * 3
    
# Used for both strength and locked bonus:
node_degrees = list(selected_g.degree())
average_degree = sum([node_degree[1] for node_degree in node_degrees]) / len(node_degrees)
print('AVERAGE DEGREE', average_degree)

# STRENGTH BONUS =======================================================
above_average = 0
for node_degree in node_degrees:
    if selected_ingredients['strong'][node_degree[0]] == 'Y':
        above_average += (node_degree[1] - average_degree)
    elif selected_ingredients['strong'][node_degree[0]] == 'y':
        above_average += (node_degree[1] - average_degree) * .5
print(above_average)


# FLAVOR BALANCE BONUS =============================================================================================
# ranges from roughly (0 to 1) * 1 (could be a lil over/under)
# keeping this the same from ye olde stir fry gen (app version)
n_sweet_lower = (selected_ingredients['sweet'] == 'y').sum()
n_sweet_upper = (selected_ingredients['sweet'] == 'Y').sum()
n_salty_lower = (selected_ingredients['salty'] == 'y').sum()
n_salty_upper = (selected_ingredients['salty'] == 'Y').sum()
n_sour_lower = (selected_ingredients['sour'] == 'y').sum()
n_sour_upper = (selected_ingredients['sour'] == 'Y').sum()
n_savory_lower = (selected_ingredients['savory'] == 'y').sum()
n_savory_upper = (selected_ingredients['savory'] == 'Y').sum()
n_bitter_lower = (selected_ingredients['bitter'] == 'y').sum()
n_bitter_upper = (selected_ingredients['bitter'] == 'Y').sum()
n_spicy_lower = (selected_ingredients['spicy'] == 'y').sum()
n_spicy_upper = (selected_ingredients['spicy'] == 'Y').sum()

#     # each varies from roughly .5 to 1 (normalized to the average flavor score)
#     sweet_score = (n_sweet_lower/2 + n_sweet_upper)/6
#     salty_score = (n_salty_lower/2 + n_salty_upper)/4
#     sour_score = (n_sour_lower/2 + n_sour_upper)/2.5
#     savory_score = (n_savory_lower/2 + n_savory_upper)/3
#     bitter_score = (n_bitter_lower/2 + n_bitter_upper)/3
#     spicy_score = (n_spicy_lower/2 + n_spicy_upper)/3
# #     print(sweet_score, salty_score, sour_score, savory_score, bitter_score, spicy_score)

# #     damn but this seems arbitrary & kludgy
# #     flavor_balance_score = 1 / (1 + abs(1-sweet_score) + abs(1-salty_score) + abs(1-sour_score) + abs(1-savory_score) + abs(1-spicy_score))

#     flavor_balance_score = 1 / (1 + abs(1-sweet_score) + abs(1-salty_score) + abs(1-sour_score) + abs(1-savory_score) + abs(1-spicy_score)) * 8 - 1.9
#     print('FLAVOR BALANCE SCORE', flavor_balance_score)
#     score += flavor_balance_score

# SIMPLIFYING THIS, cause at end of the day it's up to user to balance shit    

# TWO OBJECTIVES: reward having a little of every flavor, maaaybe punish for unbalanced flavors
sweet_score = min(n_sweet_lower/2 + n_sweet_upper, 1)
salty_score = min(n_salty_lower/2 + n_salty_upper, 1)
sour_score = min(n_sour_lower/2 + n_sour_upper, 1)
savory_score = min(n_savory_lower/2 + n_savory_upper, 1)
bitter_score = min(n_bitter_lower/2 + n_bitter_upper, 1)
spicy_score = min(n_spicy_lower/2 + n_spicy_upper, 1)

flavor_balance_score = 0
flavor_balance_score += sweet_score*3 # rly want something sweet in there
flavor_balance_score += salty_score*.5 # can always use salt
flavor_balance_score += sour_score*2 # like me some sour
flavor_balance_score += savory_score*3 # LOVE me some savory
flavor_balance_score += bitter_score # idk
flavor_balance_score += spicy_score*2 # can be nice
flavor_balance_score = flavor_balance_score * .2 - 1.3

#     print('FLAVOR BALANCE SCORE', flavor_balance_score)
score += flavor_balance_score

# FOOD GROUPS BONUS ==========================================================================================
#     n_fruit = (selected_ingredients['stir_fry_fruit'] == 'y').sum()
#     n_protein = (selected_ingredients['stir_fry_protein'] == 'y').sum()

#     # very very few, so realistically max .7
#     fruit_score = (n_fruit/2)**.5
#     print('FRUIT SCORE', fruit_score)

#     #     0, .7, occaaaasionally 1
#     protein_score = (n_protein/2)**.5
#     print('PROTEIN SCORE', protein_score)

if 'y' in selected_ingredients['stir_fry_protein'].values:
    protein_score = .5
else:
    protein_score = 0

if 'y' in selected_ingredients['stir_fry_fruit'].values:
    fruit_score = .5
else:
    fruit_score = 0

# print(fruit_score, protein_score)
food_group_score = protein_score + fruit_score
#     print('PROTEIN FRUIT', protein_score, fruit_score)
score += food_group_score

# # will bias toward larger stir_frys, slightly
# # PROTEIN BONUS =========================================================================================
# # ranges from roughly (0 to 1) * .5 (mostly balanced on its own)
#     n_protein = (selected_ingredients['stir_fry_protein'] == 'y').sum()

#     # /2 for steep diminishing returns (?)
#     protein_score = (n_protein/2)**.5 * .75
# #     print(protein_score)
#     score += protein_score * .5

top_average_shortest_path_score = average_shortest_path_score
top_flavor_balance_score = flavor_balance_score
top_food_group_score = food_group_score
#         top_protein_score = protein_score
top_score = score
#         top_lc_pairs = lower_category_pairs
#         top_ld_pairs = lower_direct_pairs
#         top_uc_pairs = upper_category_pairs
top_connections = connections
top_selected_ingredients = selected_ingredients
#         top_selected_g = selected_g
#     print('RAW PAIRING BONUS', top_average_shortest_path_score)
#     print('RAW FLAVOR BALANCE BONUS', top_flavor_balance_score)
#     print('RAW PROTEIN BONUS', top_protein_score)
#     print('SCORE', top_score)
#     print()

print()
print('RAW PAIRING BONUS', top_average_shortest_path_score)
print('RAW FLAVOR BALANCE BONUS', top_flavor_balance_score)
print('RAW FOOD GROUP BONUS', top_food_group_score)
print('SCORE', top_score)

cliqued_net = net.Network(notebook=True)

nodes = top_selected_ingredients['name'].tolist()

def get_color(row):
#     return 'grey'
    if row['stir_fry_veg'] == 'y':
        return 'green'
    elif row['stir_fry_fruit'] == 'y':
        return 'orange'
    elif row['stir_fry_protein'] == 'y':
        return 'brown'
    elif row['stir_fry_grain'] == 'y':
        return 'tan'
    else:
        return 'lightgrey'
nodes_color = top_selected_ingredients.apply(get_color, axis=1).tolist()

cliqued_net.add_nodes(
    nodes=nodes,
    color=nodes_color
)

for connection in top_connections:
    if connection[2][0] == 'c':
        cliqued_net.add_edge(connection[0], connection[1], physics=False, color='lightgrey')
    if connection[2][0] == 'd':
        cliqued_net.add_edge(connection[0], connection[1], physics=False, color='grey')
    if connection[2][0] == 'C':
        cliqued_net.add_edge(connection[0], connection[1], color='darkgrey')
    if connection[2][0] == 'D':
        cliqued_net.add_edge(connection[0], connection[1], color='black')
 
cliqued_net.show('cliqued_net.html')

[('SEITAN (see also tips for NAMA-FU)', 2), ('ALMONDS (and UNSWEETENED ALMOND BUTTER; see also MILK, ALMOND)', 3), ('ORANGES, ORANGE JUICE, and ORANGE ZEST', 4), ('LAVENDER', 4), ('VINEGAR, BALSAMIC', 5), ('POTATOES, BLUE (or PURPLE)', 2)]
Scoring method set to "connected"
AVERAGE DEGREE 3.3333333333333335
1.4999999999999998

RAW PAIRING BONUS 0.11539322489180726
RAW FLAVOR BALANCE BONUS -0.050000000000000044
RAW FOOD GROUP BONUS 1.0
SCORE 1.2961796746754217
CPU times: user 16.8 ms, sys: 0 ns, total: 16.8 ms
Wall time: 16.9 ms


In [63]:
selected_ingredients['strong']

name_index
SEITAN (see also tips for NAMA-FU)                                 
ALMONDS (and UNSWEETENED ALMOND BUTTER; see also MILK, ALMOND)     
ORANGES, ORANGE JUICE, and ORANGE ZEST                            y
LAVENDER                                                          y
VINEGAR, BALSAMIC                                                 y
POTATOES, BLUE (or PURPLE)                                         
Name: strong, dtype: object